In [1]:
import os
import pickle
import string
import argparse

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1

REBUILDDICTIONARYFROMPICKLEFILE = 0

path = 'F:\LaTeX\LatexWork\Bible'
os.chdir(path)
print (os.getcwd())

import plotly

F:\LaTeX\LatexWork\Bible


In [2]:
# %load BibleWordList2.py
'''
Update 21 Deceber 2018, 0004
'''


import os
import pickle
import collections
import operator
import re

DEBUG = 0

SAVEDICTIONARYTOPICKLEFILE = 1
USEPICKLEFILE = 1
REBUILDDICTIONARYFROMPICKLEFILE = 0

punctuations = '''!()_;:,<>?@#$^.'''

exceptions = [
    'A', 'Accept', 'After', 'Aha', 'All','Alas','Also','Arise',
    'Amen', 'An', 'And', 'As', 'At', 'Awake','According',
    'Be', 'Because', 'Before', 'Behold', 'Blessed','Build','Better',
    'Blessings', 'Blind', 'Both', 'Bring', 'But','Beautiful',
    'By', 'Command', 'Concerning', 'Consider', 'Cut','Come',"Children's",
    'Deal', 'Deceit', 'Deep', 'Deliver', 'Depart','Did',
    'Doubtless', 'Each', 'Either', 'Even', 'Every','Excellent','Fear',
    'For', 'From', 'Get', 'Give', 'Go', 'Grace',
    'Great', 'Hatred', 'Having', 'He', 'Hear',
    'Hearing', 'Heaviness', 'Help', 'His', 'Hold',
    'Horror', 'How', 'If', 'In', 'Incline', 'Judge',
    'Leave', 'Let', 'Lo', 'Look', 'Lying', 'Lodge','Make','Must','Mark',
    'Many', 'Mine', 'My', 'Neither', 'Nevertheless','Nay',
    'Not', 'Now', 'Number', 'O', 'Of', 'On', 'One','Out',
    'Only', 'Open', 'Or', 'Order', 'Our', 'Pardon',
    'Plead', 'Princes', 'Quicken', 'Red', 'Remember',
    'Remove', 'Riches', 'Righteous', 'Rise', 'Rivers','Return',
    'Sacrifice', 'Salvation', 'Say', 'Seven', 'Shall','Spake',
    'So', 'Speak', 'Sprinkle', 'Stablish', 'Stand','Strong',
    'Surely', 'Take', 'Teach', 'That', 'The', 'Their',
    'Then', 'There', 'Therefore', 'These', 'They',
    'This', 'Those', 'Thou', 'Though', 'Through',
    'Thus', 'Thy', 'To', 'Treasures', 'Trouble','Told',
    'Turn', 'Two', 'Unless', 'Uphold', 'We',
    'What', 'Whatsoever', 'When', 'Where', 'Wherefore',
    'Wherewithal', 'Which', 'Who', 'Whom', 'Whoso',
    'Whosoever', 'Why', 'Wise', 'With', 'Withhold','Walk','Wisdom',
    'Would', 'Ye', 'Yea', 'Yet', 'Your']

properNameList = [
    'Aaron', 'Abelmizraim', 'Abiasaph', 'Abib', 'Abihu', 'Abraham', 'Abram',
    'Aceldama', 'Achaia', 'Achaicus', 'Achor', 'Achzib', 'Adam', 'Adramyttium',
    'Adria', 'Adullam', 'Agrippa', 'Ahab', 'Ahaz', 'Ahikam', 'Ahisamach',
    'Aholiab', 'Alexandria', 'Alexandrians', 'Alphaeus', 'Amalek', 'Amittai',
    'Ammi ', 'Amminadab', 'Amon', 'Amorite', 'Amorites', 'Amram', 'Anathoth',
    'Andrew', 'Angel', 'Antioch', 'Aphrah', 'Apollos', 'Apphia', 'Aquila',
    'Arabia ', 'Arabian', 'Archippus', 'Ard', 'Areli', 'Aretas', 'Aristarchus',
    'Arnon', 'Arodi', 'Aroer ', 'Asenath', 'Asenath', 'Ashbel', 'Asher',
    'Ashkelon', 'Asia', 'Assir', 'Assyria ', 'Assyrian', 'Atad', 'Augustus',
    'Azariah', 'Azur', 'BRANCH', 'Baal ', 'Baali ', 'Baalim', 'Baalzephon',
    'Bablyon', 'Balaam', 'Balak', 'Barbarian', 'Barjona', 'Barnabas',
    'Barsabas', 'Bartholomew', 'Baruch', 'Becher', 'Beeri', 'Beerlahairoi',
    'Beersheba', 'Belah', 'Belial', 'Belteshazzar', 'Benjamin', 'Beor',
    'Bered', 'Beriah', 'Bernice', 'Bethaven', 'Bethdiblathaim ', 'Bethel',
    'Bethezel', 'Bethgamul', 'Bethlehem', 'Bethmeon', 'Bethshemesh',
    'Bezaleel', 'Bilhah', 'Bozrah', 'Caesar ', 'Caesarea', 'Cainan', 'Cana',
    'Canaan', 'Canaanite', 'Canaanites', 'Canaanitish', 'Capernaum', 'Caphtor',
    'Carchemish', 'Carmi', 'Castor', 'Cephas', 'Chaldeans', 'Chaldeans',
    'Chemosh', 'Chittim', 'Chloe', 'Christ', 'Cilicia', 'Clauda', 'Cnidus',
    'Crete', 'Cretians', 'Crispus', 'Cyprus', 'Cyrenians', 'Cyrus',
    'Damascenes', 'Damascus', 'Dan', 'Daniel', 'Darius', 'David', 'Demas',
    'Diblaim', 'Dibon', 'Didymus', 'Dinah', 'Edom', 'Egypt', 'Egyptian',
    'Egyptians', 'Ehi', 'Elasah', 'Eleazar', 'Elias', 'Eliezer', 'Elim',
    'Elisheba', 'Elkanah', 'Elon', 'Elzaphan', 'Enoch', 'Enos', 'Epaphras',
    'Ephesus', 'Ephraim', 'Ephrath', 'Ephron', 'Er', 'Eri', 'Esaias', 'Esau',
    'Etham', 'Euphrates', 'Euroclydon', 'Ezbon', 'Father', 'Felix', 'Festus',
    'Fortunatus', 'Gad', 'Gadarenes', 'Gaius', 'Galatia', 'Galilee', 'Gath',
    'Gaza', 'Gedaliah', 'Gemariah', 'Gentiles', 'Gera', 'Gershom', 'Gershon',
    'Gibeah', 'Gibeon', 'Gilead', 'Gilgal', 'God', 'Gomer', 'Goshen',
    'Grecians', 'Greek', 'Guni', 'Hagar', 'Haggi', 'Ham', 'Hananiah', 'Hanoch',
    'Heber', 'Hebrew', 'Hebrews', "Hebrews'", 'Hebron', 'Heldai', 'Helem',
    'Hen', 'Heshbon', 'Hezekiah', 'Hezron', 'High', 'Hilkiah', 'Hinnom',
    'Hittite', 'Hittites', 'Hivite', 'Hivites', 'Horeb', 'Horonaim', 'Hosea',
    'Hoshaiah', 'Huppim', 'Hur', 'Hushim', 'I', 'Isaac', 'Ishi', 'Ishmael',
    'Ishmeelites', 'Ishuah', 'Israel', 'Issachar', 'Issachar', 'Isui', 'Italy',
    'Ithamar', 'Ithiel', 'Izhar', 'JEHOVAH', 'Jachin', 'Jacob', 'Jahleel',
    'Jahzeel', 'Jairus', 'Jakeh', 'James', 'Jamin', 'Japheth', 'Jareb:',
    'Jared', 'Jebusite', 'Jebusites', 'Jeconiah', 'Jedaiah', 'Jehoiada',
    'Jehoiakim', 'Jehoshapha', 'Jehovahnissi', 'Jehu', 'Jemuel', 'Jeremiah',
    'Jerusalem', 'Jesus', 'Jethro', 'Jews', 'Jezer', 'Jezreel', 'Jimnah',
    'Joash', 'Job', 'Jochebed', 'Joel', 'Johanan', 'John', 'Jonah', 'Jonas',
    'Joppa', 'Jordan', 'Josedech', 'Joseph', 'Josiah', 'Jotham', 'Judaea',
    'Judah', 'Judas', 'Julius', 'Just ', 'Justus ', 'Kadesh', 'Kareah',
    'Kedar', 'Kerioth', 'King', 'Kiriathaim ', 'Kohath', 'Korah', 'Korhites',
    'LORD', 'Laban', 'Laban', 'Lachis', 'Lamech', 'Laodicea', 'Lasea', 'Leah',
    'Lebanon', 'Legion', 'Levi', 'Libertines', 'Libni', 'Libyans', 'Loammi',
    'Lord', 'Lord Jesus Christ', 'Loruhamah', 'Lucas', 'Luhith', 'Luz', 'Lycia',
    'Lydians', 'Maaseiah', 'Macedonia', 'Macedonian', 'Machir', 'Machpelah',
    'Mahalaleel', 'Mahali', 'Malachi', 'Malchiel', 'Mamre', 'Manasseh',
    'Marcus', 'Mareshah', 'Maroth', 'Massah', 'Matthew', 'Matthias',
    'Melchizedek', 'Melita', 'Melzar', 'Merari', 'Meribah', 'Messias',
    'Methuselah', 'Micah', 'Midian', 'Migdol', 'Miriam', 'Misgab', 'Mishael',
    'Mizpah', 'Moab', 'Morasthite', 'Moreshethgath', 'Moses', 'Muppim', 'Mushi',
    'Myra', 'Naaman', 'Naashon', 'Nadab', 'Naphtali', 'Nathanael', 'Nazareth',
    'Nebo', 'Nebuchadnezzar', 'Nebuchadrezzar', 'Nebuzaradan', 'Nehelamite',
    'Nepheg', 'Neriah', 'Nicanor', 'Nicolas', 'Nicopolis', 'Nineveh', 'No ',
    'Noah', 'Noph', 'Ohad', 'Olivet', 'Omri', 'Onan', 'Onesimus', 'Padan',
    'Padanaram', 'Palestine', 'Pallu', 'Pamphylia', 'Parmenas', 'Pashur',
    'Pathros', 'Paul', 'Pentecost', 'Pergamos', 'Perizzite', 'Perizzites',
    'Peter', 'Pethuel', 'Phallu', 'Pharaoh', "Pharaoh's", 'Pharaohhophra ',
    'Pharaohnecho', 'Pharez', 'Pharisee ', 'Pharisees', 'Phenice',
    'Philadelphia', 'Philemon', 'Philip', 'Philippi', 'Philistines',
    'Phinehas', 'Phuvah', 'Pihahiroth', 'Pilate', 'Pithom', 'Pollux',
    'Potiphar', 'Potipherah', 'Prince', 'Priscilla', 'Prochorus', 'Puah',
    'Publius', 'Puteoli', 'Putiel', 'Raamses', 'Rachel', 'Ramah', 'Rameses',
    'Rephidim', 'Reuben', 'Reuel', 'Rhegium', 'Rock', 'Rome', 'Rosh', 'Ruhamah',
    'Sabeans', 'Sadducees', 'Salem', 'Salmone', 'Samaria', 'Samaritans',
    'Samuel', 'Saphir', 'Sara', 'Sarah', 'Sarai', 'Sardis', 'Satan', 'Saul',
    'Saviour', 'Scythian', 'Selah', 'Sepharad', 'Serah', 'Sered', 'Seth',
    'Shaphan', 'Shaul', 'Shelah', 'Shem', 'Shemaiah', 'Shillem', 'Shiloh',
    'Shimi', 'Shimron', 'Shiphrah', 'Shittim', 'Shuni', 'Sidon', 'Sihon',
    'Sihor', 'Simeon', 'Simon ', 'Simon Zelotes', 'Sinai', 'Sirion', 'Smyrna',
    'Solomon', 'Sosthenes', 'Stephanas', 'Stephen', 'Succoth', 'Sychar',
    'Syracuse', 'Tabor', 'Tahpanhes', 'Tarshish', 'Teman', 'Theophilus',
    'Thessalonica', 'Thomas', 'Thummim', 'Thyatira', 'Tiberias', 'Timon',
    'Timotheus', 'Timothy', 'Titus', 'Tobijah', 'Tola', 'Tophet', 'Tychicus',
    'Tyre', 'Tyrus', 'Ucal', 'Uri', 'Urim', 'Uzziah', 'Uzziel', 'Zaanan',
    'Zaphnathpaaneah', 'Zarephath', 'Zebedee', 'Zebulun', 'Zedekiah',
    'Zephaniah', 'Zerah', 'Zichri', 'Zidon', 'Zilpah', 'Zion', 'Ziphion',
    'Zipporah', 'Zithri', 'Zohar']

##########
def removePunc(input):
    no_punc = ''
    for char in input:
        if char not in punctuations:
            no_punc = no_punc + char
    return no_punc


##########
def generatedLatexFormattedText(dict):
    BUILDAWIP = 1
    BUILDNWIV = 1
    BUILDPNIP = 1
    BUILDNWIC = 1
    BUILDNLIW = 1
    BUILDBPIP = 1
    MINISTRY = 1

    topString = "\\footnote{\\textcolor[cmyk]{0.99998,1,0,0}{\\hyperlink{TOC}{Return to end " \
                "of Table of Contents.}}}"
    textColorString = "\\textcolor[cmyk]{0.99998,1,0,0}{"
    endString = "}"

    '''
    Print plain text with Latex color header:
        \textcolor[rgb]{0.00,0.00,1.00} == BLUE
        \textcolor[cmyk]{0.99998,1,0,0} == A DEEPER BLUE
    '''
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Chapter with Latex format\n%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')

    linesWith13Words = []
    linesWith18Words = []
    for line in dict:
        booknm = line[0]
        chapnm = line[1]
        chapno = line[2]
        versno = line[3]
        versn2 = line[4]
        newline = line[5]
        if versn2 == '1':
            print(topString + textColorString + newline + endString)
        else:
            print('[' + versn2 + '] ' + textColorString + newline + endString)
        words = newline.split()
        wordLength = len(words)
        if wordLength == 13:
            linesWith13Words.append(line)
        if wordLength == 18:
            linesWith18Words.append(line)
            
        
    '''
    NOW print the verses with LaTeX index items:
        AWIP - All Words in Passage
        NWIV - NUmber of Words in Verse
        PNIP - Proper noun in passage
        ... more later
    '''
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('Chapter with indexed items\n%%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')
    for line in dict:
        booknm = line[0]
        chapnm = line[1]
        chapno = line[2]
        versno = line[3]
        versn2 = line[4]
        verseRef = "!" + booknm + "!" + chapnm + " " + chapno + ":" + versno 
        line = line[5]
        if versn2 == '1':
            verseString = topString + textColorString + line + endString
        else:
            verseString = '[' + versn2 + '] ' + textColorString + line + endString
        #print(booknm,chapnm,chapno,versno,versn2) 
        '''
        NWIV
        '''
        words = line.split()
        numWords = len(words)
        if DEBUG == 1:
            print(numWords,'Words in verse',versn2)
        extraString="\\index[NWIV]{" + str(numWords) + verseRef + "}"
        '''
        AWIP - here, Passage = verse
        '''
        verseDict = {}
        wordString = ''
        for word in words:
            word = removePunc(word)
            if word not in verseDict:
                verseDict[word] = 1
                wordString = wordString + "\\index[AWIP]{" + word + verseRef + "}"
            else:
                verseDict[word] = verseDict[word] + 1
                wordString = wordString + "\\index[AWIP]{" + word + verseRef 
                wordString = wordString + " (" + str(verseDict[word]) + ")}"
        '''
        PNIP - Proper names in passage
        '''
        propernameStrng = ''
        for item in verseDict:
            if item in properNameList:
                if item not in exceptions:
                    propernameStrng = propernameStrng + "\\index[PNIP]{" + item + verseRef + "}"

        builtVerseString = verseString + extraString + wordString  + propernameStrng
        print(builtVerseString)
    return linesWith18Words, linesWith13Words

##########
def showWordsByFrequency(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    print('\\subsection{Words by Frequency}')
    print ("\\scriptsize")
    print ("\\begin{center}")
    print ("\\begin{longtable}{l|r}")
    print ("\caption[Word Frequencies in "+m.group(1)+" "+str(int(m.group(2)))+"]{Word Frequencies in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    sortedDictionary = sorted(dictionary.items(), key=lambda x: x[1], reverse=True)
    for item in sortedDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
    print ("\\end{center}")
    print ("\n\n")
    print ("\\normalsize")
    print ("\n\n")    

##########
def showWordsAlphabetically(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    print('\\subsection{Words Alphabetically}')
    print ("\\scriptsize")
    print ("\\begin{center}")
    print ("\\begin{longtable}{l|r}")
    print ("\caption[Word Alphabetically in "+m.group(1)+" "+str(int(m.group(2)))+"]{Word Alphabetically in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Word}} & \multicolumn{1}{c|}{\\textbf{Frequency}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    sortedDictionary = sorted(dictionary.items(), key=lambda x: x[0], reverse=False)
    for item in sortedDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
    print ("\\end{center}")
    print ("\n\n")
    print ("\\normalsize")
    print ("\n\n")    

##########
def generateWordStatisticsForChapter(label,verses):
    '''
    Need to simplify this code! ONLY count number of words, number of unique 
    words, number of itallic words, number of unique itallic words
    
    verseDictionary, rest for each verse
    chapterDictionary
    '''
    global properNameList
    wordCount = 0
    emphasizedwordCount = 0
    wordDict = {}
    emphasizedwordDict = {}
    verseString = ''
    A = re.findall(r'\d+', label)
    A = " ".join(map(str, A))
    digit = int(A)

    word1 = " ".join(re.findall("[a-zA-Z]+", label))
    
    print('\n\n%%%%%%%%%%%%%%%%%%%%%%%%%%%')
    print('%%%%% Word Statistics\n%%%%%%%%%%%%%%%%%%%%%%%%%%\n\n')
    
    print('\\normalsize')

    verseStats = ["\\subsection{Chapter Word Statistics}",
                  "\n",
                  "%%%%%%%%%%",
                  "%%%%%%%%%%",
                  " ",
                  "\\begin{center}",
                  "\\begin{longtable}{l|c|c|c|c}",
                  "\\caption[Stats for "+word1+" "+str(digit) +"]{Stats for "+word1+" "+str(digit) +"} \\label{table:Stats for "+word1+" "+str(digit) +"} \\\ ",
                  "\\hline \multicolumn{1}{|c|}{\\textbf{Verse(s)}} & \multicolumn{1}{|c|}{\\textbf{Count}} & \multicolumn{1}{|c|}{\\textbf{Unique}} & \multicolumn{1}{|c|}{\\textbf{Italics}} & \multicolumn{1}{|c|}{\\textbf{Uniq Italic}}  \\\ \\hline ",
                  "\\endfirsthead",
                  " ",
                  "\multicolumn{5}{c}",
                  "{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\  ",
                  "\\hline \multicolumn{1}{|c|}{\\textbf{Verse(s)}} & \multicolumn{1}{|c|}{\\textbf{Count}} & \multicolumn{1}{|c|}{\\textbf{Unique}} & \multicolumn{1}{|c|}{\\textbf{Italics}} & \multicolumn{1}{|c|}{\\textbf{Uniq Italic}}  \\\ \\hline ",
                  "\\endhead",
                  " ",
                  "\\hline \\multicolumn{5}{|r|}{{Continued if needed}} \\\ \\hline",
                  "\\endfoot ",]

    '''
    This section prints JUST the Bible text in a LaTeX format with a textcolor encoding added
    Original Code
    '''
    chapterDict = {}
    chapwordWords = 0
    chapterUniqueWords = 0
    numberChapterUniqueItallics = 0
    numberChapterItallics = 0
    verseNo = 0
    for line in verses:
        verseNo += 1
        verseDict = {}
#        print(line)
        words = removePunc(line[5])
        wordcount = 0
        uniqueWords = 0        
        '''
        Process for verse
        '''
        for word in words.split():
            wordcount += 1
            if word not in verseDict:
                verseDict[word] = 1
                uniqueWords += 1
            else:
                verseDict[word] = verseDict[word] + 1
        numberUniqueItallics = 0
        numberItallics = 0
        for item in verseDict:
            if "\emph{" in item:
                numberUniqueItallics += 1
                numberItallics = numberItallics + verseDict[item]                
        '''
        Process for chapter
        '''                
        for word in words.split():
            chapwordWords += 1
            if word not in chapterDict:
                chapterDict[word] = 1
                chapterUniqueWords += 1
            else:
                chapterDict[word] = chapterDict[word] + 1
        sequence = str(verseNo),str(wordcount),str(uniqueWords),str(numberItallics),str(numberUniqueItallics)+'\\\ \hline'
        S = ' & '.join(sequence)
        verseStats.append(S)
    for item in chapterDict:
        if "\emph{" in item:
            numberChapterUniqueItallics +=1
            numberChapterItallics = numberChapterItallics + chapterDict[item]

    '''
    Finish Up!
    '''
#    print ("\n",wordCount,"words in passage, of which",len(wordDict),"are unique")
#    print (emphasizedwordCount,'words in ital, of which',len(emphasizedwordDict),'are unique')
    print ("\n\n")
    verseStats.append("\hline \hline")
    sequence = 'Total',str(chapwordWords),str(chapterUniqueWords),str(numberChapterItallics),str(numberChapterUniqueItallics)
    S = ' & '.join(sequence)
    verseStats.append(S)
    for item in verseStats:
        print (item)
    print ("\n\n")
    print ("\\end{longtable}")
    print ("\\end{center}\n")
#    print (chapwordWords,"words in passage, of which",chapterUniqueWords,"are unique")
#    print (numberChapterItallics,'words in ital, of which',numberChapterUniqueItallics,'are unique')
#    print ("\n\n")
#
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
    
    return chapterDict

##########
def getWordLengthsInChapter(dictionary,arg):
    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)

    wordLengthDictionary = {}
    for item in dictionary:
        if "\emph{" in item:
            tempitem = item.replace("\emph{",'')
            tempitem = tempitem.replace('}','')
            if len(tempitem) in wordLengthDictionary:
                wordLengthDictionary[len(tempitem)] = wordLengthDictionary[len(tempitem)] + ', ' + item
            else:
                    wordLengthDictionary[len(tempitem)] = item
        else:
            if len(item) in wordLengthDictionary:
                wordLengthDictionary[len(item)] = wordLengthDictionary[len(item)] + ', ' + item
            else:
                wordLengthDictionary[len(item)] = item
    wordLengthDictionary = sorted(wordLengthDictionary.items(), key=lambda x:x[0], reverse=False)

    print('\\subsection{Word Lengths in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Words by Length in "+m.group(1)+" "+str(int(m.group(2)))+"]{Words by Length in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:WordsIn-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Length}} & \multicolumn{1}{c|}{\\textbf{Words}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Length}} & \multicolumn{1}{c|}{\\textbf{Words}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in wordLengthDictionary:
        print (item[0], "&", item[1], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")

##########
def makeVersesWith13WordsTable(list,arg):
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")

    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)


    print('\\subsection{Verses with 13 Words in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Verses with 13 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"]{Verses with 13 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:Verses with 13 Words in-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in list:
        print (item[0]+' '+item[2]+":"+item[4], "&", item[5], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
        
##########
          
##########
def makeVersesWith18WordsTable(list,arg):
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")

    r = re.compile("([a-zA-Z]+)([0-9]+)")
    m = r.match(arg)


    print('\\subsection{Verses with 18 Words in Chapter}')
    print ("\\normalsize")
#    print ("\\begin{center}")
    print ("\\begin{longtable}{l|p{3.75in}}")
    print ("\caption[Verses with 18 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"]{Verses with 18 Words  in "+m.group(1)+" "+str(int(m.group(2)))+"} \label{table:Verses with 18 Words in-"+m.group(1)+"-"+str(int(m.group(2)))+"} \\\ ")    
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endfirsthead")
    print (" ")
    print ("\\multicolumn{2}{c}")
    print ("{{\\bfseries \\tablename\ \\thetable{} -- continued from previous page}} \\\ ")
    print ("\hline \multicolumn{1}{|c|}{\\textbf{Reference}} & \multicolumn{1}{c|}{\\textbf{Verse}} \\\ \\hline ")
    print ("\\endhead")
    print (" ")
    print ("\hline \multicolumn{2}{|r|}{{Continued if needed}} \\\ \\hline")
    print ("\\endfoot")
    print (" ")
    print ("\hline \hline")
    print ("\\endlastfoot")
    for item in list:
        print (item[0]+' '+item[2]+":"+item[4], "&", item[5], "\\\ \hline")
    print ("\\end{longtable}")
#    print ("\\end{center}")
    print ("\n\n")
    print ("\n\n") 
    print ("%%%%%%%%%%\n%%%%%%%%%%")
    print (" ")
        
##########

def countLettersInVerses(arg1,arg2):
    for item in arg2:
        letterDict = {}
        for item2 in item[5]:
            if "\emph" in item2:
                item2 = item2.replace('\emph','')
            if item2 != ' ':
                if item2 in letterDict:
                    letterDict[item2] = letterDict[item2] + 1
                else:
                    letterDict[item2] =  1
            print ("for verse",item[2],":",letterDict)

def ProcessVerses(arg1,arg2):
    ''' 
    arg1 is dictionary name and arg2 is the dictionary
    '''
#    saveToPickleFile(arg1,arg2)  
    linesWith18words, linesWith13words = generatedLatexFormattedText(arg2)         # format chapter in Latex
    #linesWith13words is for use later ... 
    chapterDict = generateWordStatisticsForChapter(arg1,arg2)   # do chapter word statistics
    showWordsByFrequency(chapterDict,arg1)                      # show words by frequency
    showWordsAlphabetically(chapterDict,arg1)                   # show words alphabetically
    getWordLengthsInChapter(chapterDict,arg1)                   # show word lengths in chapter
    
    if len(linesWith13words) > 0:
        #print(linesWith13words)
        makeVersesWith13WordsTable(linesWith13words,arg1)
        
    if len(linesWith18words) > 0:
        #print(linesWith13words)
        makeVersesWith18WordsTable(linesWith18words,arg1)
        
    ''' In Development
    countLettersInVerses(arg1,arg2)
    '''



In [3]:
path = 'F:\python'
os.chdir(path)
print (os.getcwd())

F:\python


In [4]:
# %load NEW01OT-MakeConcordance-Genesis.py
"""
Created on Sun Jan 01 07:52:18 2017

@author: keith
"""
import string

from BibleWordList2 import *

Genesis01 = [
    ['Genesis','Gen','01','001','1',"In the beginning God created the heaven and the earth."],
['Genesis','Gen','01','002','2',"And the earth was without form, and void; and darkness \emph{was} upon the face of the deep. And the Spirit of God moved upon the face of the waters."],
['Genesis','Gen','01','003','3',"And God said, Let there be light: and there was light."],
['Genesis','Gen','01','004','4',"And God saw the light, that \emph{it} \emph{was} good: and God divided the light from the darkness."],
['Genesis','Gen','01','005','5',"And God called the light Day, and the darkness he called Night. And the evening and the morning were the first day."],
['Genesis','Gen','01','006','6',"And God said, Let there be a firmament in the midst of the waters, and let it divide the waters from the waters."],
['Genesis','Gen','01','007','7',"And God made the firmament, and divided the waters which \emph{were} under the firmament from the waters which \emph{were} above the firmament: and it was so."],
['Genesis','Gen','01','008','8',"And God called the firmament Heaven. And the evening and the morning were the second day."],
['Genesis','Gen','01','009','9',"And God said, Let the waters under the heaven be gathered together unto one place, and let the dry \emph{land} appear: and it was so."],
['Genesis','Gen','01','010','10',"And God called the dry \emph{land} Earth; and the gathering together of the waters called he Seas: and God saw that \emph{it} \emph{was} good."],
['Genesis','Gen','01','011','11',"And God said, Let the earth bring forth grass, the herb yielding seed, \emph{and} the fruit tree yielding fruit after his kind, whose seed \emph{is} in itself, upon the earth: and it was so."],
['Genesis','Gen','01','012','12',"And the earth brought forth grass, \emph{and} herb yielding seed after his kind, and the tree yielding fruit, whose seed \emph{was} in itself, after his kind: and God saw that \emph{it} \emph{was} good."],
['Genesis','Gen','01','013','13',"And the evening and the morning were the third day."],
['Genesis','Gen','01','014','14',"And God said, Let there be lights in the firmament of the heaven to divide the day from the night; and let them be for signs, and for seasons, and for days, and years:"],
['Genesis','Gen','01','015','15',"And let them be for lights in the firmament of the heaven to give light upon the earth: and it was so."],
['Genesis','Gen','01','016','16',"And God made two great lights; the greater light to rule the day, and the lesser light to rule the night: \emph{he} \emph{made} the stars also."],
['Genesis','Gen','01','017','17',"And God set them in the firmament of the heaven to give light upon the earth,"],
['Genesis','Gen','01','018','18',"And to rule over the day and over the night, and to divide the light from the darkness: and God saw that \emph{it} \emph{was} good."],
['Genesis','Gen','01','019','19',"And the evening and the morning were the fourth day."],
['Genesis','Gen','01','020','20',"And God said, Let the waters bring forth abundantly the moving creature that hath life, and fowl \emph{that} may fly above the earth in the open firmament of heaven."],
['Genesis','Gen','01','021','21',"And God created great whales, and every living creature that moveth, which the waters brought forth abundantly, after their kind, and every winged fowl after his kind: and God saw that \emph{it} \emph{was} good."],
['Genesis','Gen','01','022','22',"And God blessed them, saying, Be fruitful, and multiply, and fill the waters in the seas, and let fowl multiply in the earth."],
['Genesis','Gen','01','023','23',"And the evening and the morning were the fifth day."],
['Genesis','Gen','01','024','24',"And God said, Let the earth bring forth the living creature after his kind, cattle, and creeping thing, and beast of the earth after his kind: and it was so."],
['Genesis','Gen','01','025','25',"And God made the beast of the earth after his kind, and cattle after their kind, and every thing that creepeth upon the earth after his kind: and God saw that \emph{it} \emph{was} good."],
['Genesis','Gen','01','026','26',"And God said, Let us make man in our image, after our likeness: and let them have dominion over the fish of the sea, and over the fowl of the air, and over the cattle, and over all the earth, and over every creeping thing that creepeth upon the earth."],
['Genesis','Gen','01','027','27',"So God created man in his \emph{own} image, in the image of God created he him; male and female created he them."],
['Genesis','Gen','01','028','28',"And God blessed them, and God said unto them, Be fruitful, and multiply, and replenish the earth, and subdue it: and have dominion over the fish of the sea, and over the fowl of the air, and over every living thing that moveth upon the earth."],
['Genesis','Gen','01','029','29',"And God said, Behold, I have given you every herb bearing seed, which \emph{is} upon the face of all the earth, and every tree, in the which \emph{is} the fruit of a tree yielding seed; to you it shall be for meat."],
['Genesis','Gen','01','030','30',"And to every beast of the earth, and to every fowl of the air, and to every thing that creepeth upon the earth, wherein \emph{there} \emph{is} life, \emph{I} \emph{have} \emph{given} every green herb for meat: and it was so."],
['Genesis','Gen','01','031','31',"And God saw every thing that he had made, and, behold, \emph{it} \emph{was} very good. And the evening and the morning were the sixth day."]
]  
    
Genesis02 = [
    ['Genesis','Gen','02','001','1',"Thus the heavens and the earth were finished, and all the host of them."],
    ['Genesis','Gen','02','002','2',"And on the seventh day God ended his work which he had made; and he rested on the seventh day from all his work which he had made."],
    ['Genesis','Gen','02','003','3',"And God blessed the seventh day, and sanctified it: because that in it he had rested from all his work which God created and made."],
    ['Genesis','Gen','02','004','4',"These \emph{are} the generations of the heavens and of the earth when they were created, in the day that the LORD God made the earth and the heavens,"],
    ['Genesis','Gen','02','005','5',"And every plant of the field before it was in the earth, and every herb of the field before it grew: for the LORD God had not caused it to rain upon the earth, and \emph{there} \emph{was} not a man to till the ground."],
    ['Genesis','Gen','02','006','6',"But there went up a mist from the earth, and watered the whole face of the ground."],
    ['Genesis','Gen','02','007','7',"And the LORD God formed man \emph{of} the dust of the ground, and breathed into his nostrils the breath of life; and man became a living soul."],
    ['Genesis','Gen','02','008','8',"And the LORD God planted a garden eastward in Eden; and there he put the man whom he had formed."],
    ['Genesis','Gen','02','009','9',"And out of the ground made the LORD God to grow every tree that is pleasant to the sight, and good for food; the tree of life also in the midst of the garden, and the tree of knowledge of good and evil."],
    ['Genesis','Gen','02','010','10',"And a river went out of Eden to water the garden; and from thence it was parted, and became into four heads."],
    ['Genesis','Gen','02','011','11',"The name of the first \emph{is} Pison: that \emph{is} it which compasseth the whole land of Havilah, where \emph{there} \emph{is} gold;"],
    ['Genesis','Gen','02','012','12',"And the gold of that land \emph{is} good: there \emph{is} bdellium and the onyx stone."],
    ['Genesis','Gen','02','013','13',"And the name of the second river \emph{is} Gihon: the same \emph{is} it that compasseth the whole land of Ethiopia."],
    ['Genesis','Gen','02','014','14',"And the name of the third river \emph{is} Hiddekel: that \emph{is} it which goeth toward the east of Assyria. And the fourth river \emph{is} Euphrates."],
    ['Genesis','Gen','02','015','15',"And the LORD God took the man, and put him into the garden of Eden to dress it and to keep it."],
    ['Genesis','Gen','02','016','16',"And the LORD God commanded the man, saying, Of every tree of the garden thou mayest freely eat:"],
    ['Genesis','Gen','02','017','17',"But of the tree of the knowledge of good and evil, thou shalt not eat of it: for in the day that thou eatest thereof thou shalt surely die."],
    ['Genesis','Gen','02','018','18',"And the LORD God said, \emph{It} \emph{is} not good that the man should be alone; I will make him an help meet for him."],
    ['Genesis','Gen','02','019','19',"And out of the ground the LORD God formed every beast of the field, and every fowl of the air; and brought \emph{them} unto Adam to see what he would call them: and whatsoever Adam called every living creature, that \emph{was} the name thereof."],
    ['Genesis','Gen','02','020','20',"And Adam gave names to all cattle, and to the fowl of the air, and to every beast of the field; but for Adam there was not found an help meet for him."],
    ['Genesis','Gen','02','021','21',"And the LORD God caused a deep sleep to fall upon Adam, and he slept: and he took one of his ribs, and closed up the flesh instead thereof;"],
    ['Genesis','Gen','02','022','22',"And the rib, which the LORD God had taken from man, made he a woman, and brought her unto the man."],
    ['Genesis','Gen','02','023','23',"And Adam said, This \emph{is} now bone of my bones, and flesh of my flesh: she shall be called Woman, because she was taken out of Man."],
    ['Genesis','Gen','02','024','24',"Therefore shall a man leave his father and his mother, and shall cleave unto his wife: and they shall be one flesh."],
    ['Genesis','Gen','02','025','25',"And they were both naked, the man and his wife, and were not ashamed."]
]
    
Genesis03 = [
    ['Genesis','Gen','03','001','1',"Now the serpent was more subtil than any beast of the field which the LORD God had made. And he said unto the woman, Yea, hath God said, Ye shall not eat of every tree of the garden?"],
['Genesis','Gen','03','002','2',"And the woman said unto the serpent, We may eat of the fruit of the trees of the garden:"],
['Genesis','Gen','03','003','3',"But of the fruit of the tree which \emph{is} in the midst of the garden, God hath said, Ye shall not eat of it, neither shall ye touch it, lest ye die."],
['Genesis','Gen','03','004','4',"And the serpent said unto the woman, Ye shall not surely die:"],
['Genesis','Gen','03','005','5',"For God doth know that in the day ye eat thereof, then your eyes shall be opened, and ye shall be as gods, knowing good and evil."],
['Genesis','Gen','03','006','6',"And when the woman saw that the tree \emph{was} good for food, and that it \emph{was} pleasant to the eyes, and a tree to be desired to make \emph{one} wise, she took of the fruit thereof, and did eat, and gave also unto her husband with her; and he did eat."],
['Genesis','Gen','03','007','7',"And the eyes of them both were opened, and they knew that they \emph{were} naked; and they sewed fig leaves together, and made themselves aprons."],
['Genesis','Gen','03','008','8',"And they heard the voice of the LORD God walking in the garden in the cool of the day: and Adam and his wife hid themselves from the presence of the LORD God amongst the trees of the garden."],
['Genesis','Gen','03','009','9',"And the LORD God called unto Adam, and said unto him, Where \emph{art} thou?"],
['Genesis','Gen','03','010','10',"And he said, I heard thy voice in the garden, and I was afraid, because I \emph{was} naked; and I hid myself."],
['Genesis','Gen','03','011','11',"And he said, Who told thee that thou \emph{wast} naked? Hast thou eaten of the tree, whereof I commanded thee that thou shouldest not eat?"],
['Genesis','Gen','03','012','12',"And the man said, The woman whom thou gavest \emph{to} \emph{be} with me, she gave me of the tree, and I did eat."],
['Genesis','Gen','03','013','13',"And the LORD God said unto the woman, What \emph{is} this \emph{that} thou hast done? And the woman said, The serpent beguiled me, and I did eat."],
['Genesis','Gen','03','014','14',"And the LORD God said unto the serpent, Because thou hast done this, thou \emph{art} cursed above all cattle, and above every beast of the field; upon thy belly shalt thou go, and dust shalt thou eat all the days of thy life:"],
['Genesis','Gen','03','015','15',"And I will put enmity between thee and the woman, and between thy seed and her seed; it shall bruise thy head, and thou shalt bruise his heel."],
['Genesis','Gen','03','016','16',"Unto the woman he said, I will greatly multiply thy sorrow and thy conception; in sorrow thou shalt bring forth children; and thy desire \emph{shall} \emph{be} to thy husband, and he shall rule over thee."],
['Genesis','Gen','03','017','17',"And unto Adam he said, Because thou hast hearkened unto the voice of thy wife, and hast eaten of the tree, of which I commanded thee, saying, Thou shalt not eat of it: cursed \emph{is} the ground for thy sake; in sorrow shalt thou eat \emph{of} it all the days of thy life;"],
['Genesis','Gen','03','018','18',"Thorns also and thistles shall it bring forth to thee; and thou shalt eat the herb of the field;"],
['Genesis','Gen','03','019','19',"In the sweat of thy face shalt thou eat bread, till thou return unto the ground; for out of it wast thou taken: for dust thou \emph{art}, and unto dust shalt thou return."],
['Genesis','Gen','03','020','20',"And Adam called his wife's name Eve; because she was the mother of all living."],
['Genesis','Gen','03','021','21',"Unto Adam also and to his wife did the LORD God make coats of skins, and clothed them."],
['Genesis','Gen','03','022','22',"And the LORD God said, Behold, the man is become as one of us, to know good and evil: and now, lest he put forth his hand, and take also of the tree of life, and eat, and live for ever:"],
['Genesis','Gen','03','023','23',"Therefore the LORD God sent him forth from the garden of Eden, to till the ground from whence he was taken."],
['Genesis','Gen','03','024','24',"So he drove out the man; and he placed at the east of the garden of Eden Cherubims, and a flaming sword which turned every way, to keep the way of the tree of life."]
]
    
Genesis04 = [
    ['Genesis','Gen','04','001','1',"And Adam knew Eve his wife; and she conceived, and bare Cain, and said, I have gotten a man from the LORD."],
['Genesis','Gen','04','002','2',"And she again bare his brother Abel. And Abel was a keeper of sheep, but Cain was a tiller of the ground."],
['Genesis','Gen','04','003','3',"And in process of time it came to pass, that Cain brought of the fruit of the ground an offering unto the LORD."],
['Genesis','Gen','04','004','4',"And Abel, he also brought of the firstlings of his flock and of the fat thereof. And the LORD had respect unto Abel and to his offering:"],
['Genesis','Gen','04','005','5',"But unto Cain and to his offering he had not respect. And Cain was very wroth, and his countenance fell."],
['Genesis','Gen','04','006','6',"And the LORD said unto Cain, Why art thou wroth? and why is thy countenance fallen?"],
['Genesis','Gen','04','007','7',"If thou doest well, shalt thou not be accepted? and if thou doest not well, sin lieth at the door. And unto thee \emph{shall} \emph{be} his desire, and thou shalt rule over him."],
['Genesis','Gen','04','008','8',"And Cain talked with Abel his brother: and it came to pass, when they were in the field, that Cain rose up against Abel his brother, and slew him."],
['Genesis','Gen','04','009','9',"And the LORD said unto Cain, Where \emph{is} Abel thy brother? And he said, I know not: \emph{Am} I my brother's keeper?"],
['Genesis','Gen','04','010','10',"And he said, What hast thou done? the voice of thy brother's blood crieth unto me from the ground."],
['Genesis','Gen','04','011','11',"And now \emph{art} thou cursed from the earth, which hath opened her mouth to receive thy brother's blood from thy hand;"],
['Genesis','Gen','04','012','12',"When thou tillest the ground, it shall not henceforth yield unto thee her strength; a fugitive and a vagabond shalt thou be in the earth."],
['Genesis','Gen','04','013','13',"And Cain said unto the LORD, My punishment \emph{is} greater than I can bear."],
['Genesis','Gen','04','014','14',"Behold, thou hast driven me out this day from the face of the earth; and from thy face shall I be hid; and I shall be a fugitive and a vagabond in the earth; and it shall come to pass, \emph{that} every one that findeth me shall slay me."],
['Genesis','Gen','04','015','15',"And the LORD said unto him, Therefore whosoever slayeth Cain, vengeance shall be taken on him sevenfold. And the LORD set a mark upon Cain, lest any finding him should kill him."],
['Genesis','Gen','04','016','16',"And Cain went out from the presence of the LORD, and dwelt in the land of Nod, on the east of Eden."],
['Genesis','Gen','04','017','17',"And Cain knew his wife; and she conceived, and bare Enoch: and he builded a city, and called the name of the city, after the name of his son, Enoch."],
['Genesis','Gen','04','018','18',"And unto Enoch was born Irad: and Irad begat Mehujael: and Mehujael begat Methusael: and Methusael begat Lamech."],
['Genesis','Gen','04','019','19',"And Lamech took unto him two wives: the name of the one \emph{was} Adah, and the name of the other Zillah."],
['Genesis','Gen','04','020','20',"And Adah bare Jabal: he was the father of such as dwell in tents, and \emph{of} \emph{such} \emph{as} \emph{have} cattle."],
['Genesis','Gen','04','021','21',"And his brother's name \emph{was} Jubal: he was the father of all such as handle the harp and organ."],
['Genesis','Gen','04','022','22',"And Zillah, she also bare Tubal-cain, an instructer of every artificer in brass and iron: and the sister of Tubal-cain \emph{was} Naamah."],
['Genesis','Gen','04','023','23',"And Lamech said unto his wives, Adah and Zillah, Hear my voice; ye wives of Lamech, hearken unto my speech: for I have slain a man to my wounding, and a young man to my hurt."],
['Genesis','Gen','04','024','24',"If Cain shall be avenged sevenfold, truly Lamech seventy and sevenfold."],
['Genesis','Gen','04','025','25',"And Adam knew his wife again; and she bare a son, and called his name Seth: For God, \emph{said} \emph{she}, hath appointed me another seed instead of Abel, whom Cain slew."],
['Genesis','Gen','04','026','26',"And to Seth, to him also there was born a son; and he called his name Enos: then began men to call upon the name of the LORD."]
]
    
Genesis05 = [
    ['Genesis','Gen','05','001','1',"This \emph{is} the book of the generations of Adam. In the day that God created man, in the likeness of God made he him;"],
['Genesis','Gen','05','002','2',"Male and female created he them; and blessed them, and called their name Adam, in the day when they were created."],
['Genesis','Gen','05','003','3',"And Adam lived an hundred and thirty years, and begat \emph{a} \emph{son} in his own likeness, after his image; and called his name Seth:"],
['Genesis','Gen','05','004','4',"And the days of Adam after he had begotten Seth were eight hundred years: and he begat sons and daughters:"],
['Genesis','Gen','05','005','5',"And all the days that Adam lived were nine hundred and thirty years: and he died."],
['Genesis','Gen','05','006','6',"And Seth lived an hundred and five years, and begat Enos:"],
['Genesis','Gen','05','007','7',"And Seth lived after he begat Enos eight hundred and seven years, and begat sons and daughters:"],
['Genesis','Gen','05','008','8',"And all the days of Seth were nine hundred and twelve years: and he died."],
['Genesis','Gen','05','009','9',"And Enos lived ninety years, and begat Cainan:"],
['Genesis','Gen','05','010','10',"And Enos lived after he begat Cainan eight hundred and fifteen years, and begat sons and daughters:"],
['Genesis','Gen','05','011','11',"And all the days of Enos were nine hundred and five years: and he died."],
['Genesis','Gen','05','012','12',"And Cainan lived seventy years, and begat Mahalaleel:"],
['Genesis','Gen','05','013','13',"And Cainan lived after he begat Mahalaleel eight hundred and forty years, and begat sons and daughters:"],
['Genesis','Gen','05','014','14',"And all the days of Cainan were nine hundred and ten years: and he died."],
['Genesis','Gen','05','015','15',"And Mahalaleel lived sixty and five years, and begat Jared:"],
['Genesis','Gen','05','016','16',"And Mahalaleel lived after he begat Jared eight hundred and thirty years, and begat sons and daughters:"],
['Genesis','Gen','05','017','17',"And all the days of Mahalaleel were eight hundred ninety and five years: and he died."],
['Genesis','Gen','05','018','18',"And Jared lived an hundred sixty and two years, and he begat Enoch:"],
['Genesis','Gen','05','019','19',"And Jared lived after he begat Enoch eight hundred years, and begat sons and daughters:"],
['Genesis','Gen','05','020','20',"And all the days of Jared were nine hundred sixty and two years: and he died."],
['Genesis','Gen','05','021','21',"And Enoch lived sixty and five years, and begat Methuselah:"],
['Genesis','Gen','05','022','22',"And Enoch walked with God after he begat Methuselah three hundred years, and begat sons and daughters:"],
['Genesis','Gen','05','023','23',"And all the days of Enoch were three hundred sixty and five years:"],
['Genesis','Gen','05','024','24',"And Enoch walked with God: and he \emph{was} not; for God took him."],
['Genesis','Gen','05','025','25',"And Methuselah lived an hundred eighty and seven years, and begat Lamech:"],
['Genesis','Gen','05','026','26',"And Methuselah lived after he begat Lamech seven hundred eighty and two years, and begat sons and daughters:"],
['Genesis','Gen','05','027','27',"And all the days of Methuselah were nine hundred sixty and nine years: and he died."],
['Genesis','Gen','05','028','28',"And Lamech lived an hundred eighty and two years, and begat a son:"],
['Genesis','Gen','05','029','29',"And he called his name Noah, saying, This \emph{same} shall comfort us concerning our work and toil of our hands, because of the ground which the LORD hath cursed."],
['Genesis','Gen','05','030','30',"And Lamech lived after he begat Noah five hundred ninety and five years, and begat sons and daughters:"],
['Genesis','Gen','05','031','31',"And all the days of Lamech were seven hundred seventy and seven years: and he died."],
['Genesis','Gen','05','032','32',"And Noah was five hundred years old: and Noah begat Shem, Ham, and Japheth."]
]
    
Genesis06 = [
    ['Genesis','Gen','06','001','1',"And it came to pass, when men began to multiply on the face of the earth, and daughters were born unto them,"],
    ['Genesis','Gen','06','002','2',"That the sons of God saw the daughters of men that they \emph{were} fair; and they took them wives of all which they chose."],
    ['Genesis','Gen','06','003','3',"And the LORD said, My spirit shall not always strive with man, for that he also \emph{is} flesh: yet his days shall be an hundred and twenty years."],
['Genesis','Gen','06','004','4',"There were giants in the earth in those days; and also after that, when the sons of God came in unto the daughters of men, and they bare \emph{children} to them, the same \emph{became} mighty men which \emph{were} of old, men of renown."],
['Genesis','Gen','06','005','5',"And GOD saw that the wickedness of man \emph{was} great in the earth, and \emph{that} every imagination of the thoughts of his heart \emph{was} only evil continually."],
['Genesis','Gen','06','006','6',"And it repented the LORD that he had made man on the earth, and it grieved him at his heart."],
['Genesis','Gen','06','007','7',"And the LORD said, I will destroy man whom I have created from the face of the earth; both man, and beast, and the creeping thing, and the fowls of the air; for it repenteth me that I have made them."],
['Genesis','Gen','06','008','8',"But Noah found grace in the eyes of the LORD."],
['Genesis','Gen','06','009','9',"These \emph{are} the generations of Noah: Noah was a just man \emph{and} perfect in his generations, \emph{and} Noah walked with God."],
['Genesis','Gen','06','010','10',"And Noah begat three sons, Shem, Ham, and Japheth."],
['Genesis','Gen','06','011','11',"The earth also was corrupt before God, and the earth was filled with violence."],
['Genesis','Gen','06','012','12',"And God looked upon the earth, and, behold, it was corrupt; for all flesh had corrupted his way upon the earth."],
['Genesis','Gen','06','013','13',"And God said unto Noah, The end of all flesh is come before me; for the earth is filled with violence through them; and, behold, I will destroy them with the earth."],
['Genesis','Gen','06','014','14',"Make thee an ark of gopher wood; rooms shalt thou make in the ark, and shalt pitch it within and without with pitch."],
['Genesis','Gen','06','015','15',"And this \emph{is} \emph{the} \emph{fashion} which thou shalt make it \emph{of}: The length of the ark \emph{shall} \emph{be} three hundred cubits, the breadth of it fifty cubits, and the height of it thirty cubits."],
['Genesis','Gen','06','016','16',"A window shalt thou make to the ark, and in a cubit shalt thou finish it above; and the door of the ark shalt thou set in the side thereof; \emph{with} lower, second, and third \emph{stories} shalt thou make it."],
['Genesis','Gen','06','017','17',"And, behold, I, even I, do bring a flood of waters upon the earth, to destroy all flesh, wherein \emph{is} the breath of life, from under heaven; \emph{and} every thing that \emph{is} in the earth shall die."],
['Genesis','Gen','06','018','18',"But with thee will I establish my covenant; and thou shalt come into the ark, thou, and thy sons, and thy wife, and thy sons' wives with thee."],
    ['Genesis','Gen','06','019','19',"And of every living thing of all flesh, two of every \emph{sort} shalt thou bring into the ark, to keep \emph{them} alive with thee; they shall be male and female."],
    ['Genesis','Gen','06','020','20',"Of fowls after their kind, and of cattle after their kind, of every creeping thing of the earth after his kind, two of every \emph{sort} shall come unto thee, to keep \emph{them} alive."],
    ['Genesis','Gen','06','021','21',"And take thou unto thee of all food that is eaten, and thou shalt gather \emph{it} to thee; and it shall be for food for thee, and for them."],
    ['Genesis','Gen','06','022','22',"Thus did Noah; according to all that God commanded him, so did he."]
]
    
Genesis07 = [
    ['Genesis','Gen','07','001','1',"And the LORD said unto Noah, Come thou and all thy house into the ark; for thee have I seen righteous before me in this generation."],
['Genesis','Gen','07','002','2',"Of every clean beast thou shalt take to thee by sevens, the male and his female: and of beasts that \emph{are} not clean by two, the male and his female."],
['Genesis','Gen','07','003','3',"Of fowls also of the air by sevens, the male and the female; to keep seed alive upon the face of all the earth."],
['Genesis','Gen','07','004','4',"For yet seven days, and I will cause it to rain upon the earth forty days and forty nights; and every living substance that I have made will I destroy from off the face of the earth."],
['Genesis','Gen','07','005','5',"And Noah did according unto all that the LORD commanded him."],
['Genesis','Gen','07','006','6',"And Noah \emph{was} six hundred years old when the flood of waters was upon the earth."],
['Genesis','Gen','07','007','7',"And Noah went in, and his sons, and his wife, and his sons' wives with him, into the ark, because of the waters of the flood."],
['Genesis','Gen','07','008','8',"Of clean beasts, and of beasts that \emph{are} not clean, and of fowls, and of every thing that creepeth upon the earth,"],
['Genesis','Gen','07','009','9',"There went in two and two unto Noah into the ark, the male and the female, as God had commanded Noah."],
['Genesis','Gen','07','010','10',"And it came to pass after seven days, that the waters of the flood were upon the earth."],
['Genesis','Gen','07','011','11',"In the six hundredth year of Noah's life, in the second month, the seventeenth day of the month, the same day were all the fountains of the great deep broken up, and the windows of heaven were opened."],
['Genesis','Gen','07','012','12',"And the rain was upon the earth forty days and forty nights."],
['Genesis','Gen','07','013','13',"In the selfsame day entered Noah, and Shem, and Ham, and Japheth, the sons of Noah, and Noah's wife, and the three wives of his sons with them, into the ark;"],
['Genesis','Gen','07','014','14',"They, and every beast after his kind, and all the cattle after their kind, and every creeping thing that creepeth upon the earth after his kind, and every fowl after his kind, every bird of every sort."],
['Genesis','Gen','07','015','15',"And they went in unto Noah into the ark, two and two of all flesh, wherein \emph{is} the breath of life."],
['Genesis','Gen','07','016','16',"And they that went in, went in male and female of all flesh, as God had commanded him: and the LORD shut him in."],
['Genesis','Gen','07','017','17',"And the flood was forty days upon the earth; and the waters increased, and bare up the ark, and it was lift up above the earth."],
['Genesis','Gen','07','018','18',"And the waters prevailed, and were increased greatly upon the earth; and the ark went upon the face of the waters."],
['Genesis','Gen','07','019','19',"And the waters prevailed exceedingly upon the earth; and all the high hills, that \emph{were} under the whole heaven, were covered."],
['Genesis','Gen','07','020','20',"Fifteen cubits upward did the waters prevail; and the mountains were covered."],
['Genesis','Gen','07','021','21',"And all flesh died that moved upon the earth, both of fowl, and of cattle, and of beast, and of every creeping thing that creepeth upon the earth, and every man:"],
['Genesis','Gen','07','022','22',"All in whose nostrils \emph{was} the breath of life, of all that \emph{was} in the dry \emph{land}, died."],
['Genesis','Gen','07','023','23',"And every living substance was destroyed which was upon the face of the ground, both man, and cattle, and the creeping things, and the fowl of the heaven; and they were destroyed from the earth: and Noah only remained \emph{alive}, and they that \emph{were} with him in the ark."],
['Genesis','Gen','07','024','24',"And the waters prevailed upon the earth an hundred and fifty days."]
]   
    
Genesis08 = [
    ['Genesis','Gen','08','001','1',"And God remembered Noah, and every living thing, and all the cattle that \emph{was} with him in the ark: and God made a wind to pass over the earth, and the waters asswaged;"],
['Genesis','Gen','08','002','2',"The fountains also of the deep and the windows of heaven were stopped, and the rain from heaven was restrained;"],
['Genesis','Gen','08','003','3',"And the waters returned from off the earth continually: and after the end of the hundred and fifty days the waters were abated."],
['Genesis','Gen','08','004','4',"And the ark rested in the seventh month, on the seventeenth day of the month, upon the mountains of Ararat."],
['Genesis','Gen','08','005','5',"And the waters decreased continually until the tenth month: in the tenth \emph{month}, on the first \emph{day} of the month, were the tops of the mountains seen."],
['Genesis','Gen','08','006','6',"And it came to pass at the end of forty days, that Noah opened the window of the ark which he had made:"],
['Genesis','Gen','08','007','7',"And he sent forth a raven, which went forth to and fro, until the waters were dried up from off the earth."],
['Genesis','Gen','08','008','8',"Also he sent forth a dove from him, to see if the waters were abated from off the face of the ground;"],
['Genesis','Gen','08','009','9',"But the dove found no rest for the sole of her foot, and she returned unto him into the ark, for the waters \emph{were} on the face of the whole earth: then he put forth his hand, and took her, and pulled her in unto him into the ark."],
['Genesis','Gen','08','010','10',"And he stayed yet other seven days; and again he sent forth the dove out of the ark;"],
['Genesis','Gen','08','011','11',"And the dove came in to him in the evening; and, lo, in her mouth \emph{was} an olive leaf pluckt off: so Noah knew that the waters were abated from off the earth."],
['Genesis','Gen','08','012','12',"And he stayed yet other seven days; and sent forth the dove; which returned not again unto him any more."],
['Genesis','Gen','08','013','13',"And it came to pass in the six hundredth and first year, in the first \emph{month}, the first \emph{day} of the month, the waters were dried up from off the earth: and Noah removed the covering of the ark, and looked, and, behold, the face of the ground was dry."],
['Genesis','Gen','08','014','14',"And in the second month, on the seven and twentieth day of the month, was the earth dried."],
['Genesis','Gen','08','015','15',"And God spake unto Noah, saying,"],
['Genesis','Gen','08','016','16',"Go forth of the ark, thou, and thy wife, and thy sons, and thy sons' wives with thee."],
['Genesis','Gen','08','017','17',"Bring forth with thee every living thing that \emph{is} with thee, of all flesh, \emph{both} of fowl, and of cattle, and of every creeping thing that creepeth upon the earth; that they may breed abundantly in the earth, and be fruitful, and multiply upon the earth."],
['Genesis','Gen','08','018','18',"And Noah went forth, and his sons, and his wife, and his sons' wives with him:"],
['Genesis','Gen','08','019','19',"Every beast, every creeping thing, and every fowl, \emph{and} whatsoever creepeth upon the earth, after their kinds, went forth out of the ark."],
['Genesis','Gen','08','020','20',"And Noah builded an altar unto the LORD; and took of every clean beast, and of every clean fowl, and offered burnt offerings on the altar."],
['Genesis','Gen','08','021','21',"And the LORD smelled a sweet savour; and the LORD said in his heart, I will not again curse the ground any more for man's sake; for the imagination of man's heart \emph{is} evil from his youth; neither will I again smite any more every thing living, as I have done."],
['Genesis','Gen','08','022','22',"While the earth remaineth, seedtime and harvest, and cold and heat, and summer and winter, and day and night shall not cease."]
]
    
Genesis09 = [
    ['Genesis','Gen','09','001','1',"And God blessed Noah and his sons, and said unto them, Be fruitful, and multiply, and replenish the earth."],
['Genesis','Gen','09','002','2',"And the fear of you and the dread of you shall be upon every beast of the earth, and upon every fowl of the air, upon all that moveth \emph{upon} the earth, and upon all the fishes of the sea; into your hand are they delivered."],
['Genesis','Gen','09','003','3',"Every moving thing that liveth shall be meat for you; even as the green herb have I given you all things."],
['Genesis','Gen','09','004','4',"But flesh with the life thereof, \emph{which} \emph{is} the blood thereof, shall ye not eat."],
['Genesis','Gen','09','005','5',"And surely your blood of your lives will I require; at the hand of every beast will I require it, and at the hand of man; at the hand of every man's brother will I require the life of man."],
['Genesis','Gen','09','006','6',"Whoso sheddeth man's blood, by man shall his blood be shed: for in the image of God made he man."],
['Genesis','Gen','09','007','7',"And you, be ye fruitful, and multiply; bring forth abundantly in the earth, and multiply therein."],
['Genesis','Gen','09','008','8',"And God spake unto Noah, and to his sons with him, saying,"],
['Genesis','Gen','09','009','9',"And I, behold, I establish my covenant with you, and with your seed after you;"],
['Genesis','Gen','09','010','10',"And with every living creature that \emph{is} with you, of the fowl, of the cattle, and of every beast of the earth with you; from all that go out of the ark, to every beast of the earth."],
['Genesis','Gen','09','011','11',"And I will establish my covenant with you; neither shall all flesh be cut off any more by the waters of a flood; neither shall there any more be a flood to destroy the earth."],
['Genesis','Gen','09','012','12',"And God said, This \emph{is} the token of the covenant which I make between me and you and every living creature that \emph{is} with you, for perpetual generations:"],
['Genesis','Gen','09','013','13',"I do set my bow in the cloud, and it shall be for a token of a covenant between me and the earth."],
['Genesis','Gen','09','014','14',"And it shall come to pass, when I bring a cloud over the earth, that the bow shall be seen in the cloud:"],
['Genesis','Gen','09','015','15',"And I will remember my covenant, which \emph{is} between me and you and every living creature of all flesh; and the waters shall no more become a flood to destroy all flesh."],
['Genesis','Gen','09','016','16',"And the bow shall be in the cloud; and I will look upon it, that I may remember the everlasting covenant between God and every living creature of all flesh that \emph{is} upon the earth."],
['Genesis','Gen','09','017','17',"And God said unto Noah, This \emph{is} the token of the covenant, which I have established between me and all flesh that \emph{is} upon the earth."],
['Genesis','Gen','09','018','18',"And the sons of Noah, that went forth of the ark, were Shem, and Ham, and Japheth: and Ham \emph{is} the father of Canaan."],
['Genesis','Gen','09','019','19',"These \emph{are} the three sons of Noah: and of them was the whole earth overspread."],
['Genesis','Gen','09','020','20',"And Noah began \emph{to} \emph{be} an husbandman, and he planted a vineyard:"],
['Genesis','Gen','09','021','21',"And he drank of the wine, and was drunken; and he was uncovered within his tent."],
['Genesis','Gen','09','022','22',"And Ham, the father of Canaan, saw the nakedness of his father, and told his two brethren without."],
['Genesis','Gen','09','023','23',"And Shem and Japheth took a garment, and laid \emph{it} upon both their shoulders, and went backward, and covered the nakedness of their father; and their faces \emph{were} backward, and they saw not their father's nakedness."],
['Genesis','Gen','09','024','24',"And Noah awoke from his wine, and knew what his younger son had done unto him."],
['Genesis','Gen','09','025','25',"And he said, Cursed \emph{be} Canaan; a servant of servants shall he be unto his brethren."],
['Genesis','Gen','09','026','26',"And he said, Blessed \emph{be} the LORD God of Shem; and Canaan shall be his servant."],
['Genesis','Gen','09','027','27',"God shall enlarge Japheth, and he shall dwell in the tents of Shem; and Canaan shall be his servant."],
['Genesis','Gen','09','028','28',"And Noah lived after the flood three hundred and fifty years."],
['Genesis','Gen','09','029','29',"And all the days of Noah were nine hundred and fifty years: and he died."]
]

Genesis10 = [
    ['Genesis','Gen','10','001','1',"Now these \emph{are} the generations of the sons of Noah, Shem, Ham, and Japheth: and unto them were sons born after the flood."],
['Genesis','Gen','10','002','2',"The sons of Japheth; Gomer, and Magog, and Madai, and Javan, and Tubal, and Meshech, and Tiras."],
['Genesis','Gen','10','003','3',"And the sons of Gomer; Ashkenaz, and Riphath, and Togarmah."],
['Genesis','Gen','10','004','4',"And the sons of Javan; Elishah, and Tarshish, Kittim, and Dodanim."],
['Genesis','Gen','10','005','5',"By these were the isles of the Gentiles divided in their lands; every one after his tongue, after their families, in their nations."],
['Genesis','Gen','10','006','6',"And the sons of Ham; Cush, and Mizraim, and Phut, and Canaan."],
['Genesis','Gen','10','007','7',"And the sons of Cush; Seba, and Havilah, and Sabtah, and Raamah, and Sabtecha: and the sons of Raamah; Sheba, and Dedan."],
['Genesis','Gen','10','008','8',"And Cush begat Nimrod: he began to be a mighty one in the earth."],
['Genesis','Gen','10','009','9',"He was a mighty hunter before the LORD: wherefore it is said, Even as Nimrod the mighty hunter before the LORD."],
['Genesis','Gen','10','010','10',"And the beginning of his kingdom was Babel, and Erech, and Accad, and Calneh, in the land of Shinar."],
['Genesis','Gen','10','011','11',"Out of that land went forth Asshur, and builded Nineveh, and the city Rehoboth, and Calah,"],
['Genesis','Gen','10','012','12',"And Resen between Nineveh and Calah: the same \emph{is} a great city."],
['Genesis','Gen','10','013','13',"And Mizraim begat Ludim, and Anamim, and Lehabim, and Naphtuhim,"],
['Genesis','Gen','10','014','14',"And Pathrusim, and Casluhim, (out of whom came Philistim,) and Caphtorim."],
['Genesis','Gen','10','015','15',"And Canaan begat Sidon his firstborn, and Heth,"],
['Genesis','Gen','10','016','16',"And the Jebusite, and the Amorite, and the Girgasite,"],
['Genesis','Gen','10','017','17',"And the Hivite, and the Arkite, and the Sinite,"],
['Genesis','Gen','10','018','18',"And the Arvadite, and the Zemarite, and the Hamathite: and afterward were the families of the Canaanites spread abroad."],
['Genesis','Gen','10','019','19',"And the border of the Canaanites was from Sidon, as thou comest to Gerar, unto Gaza; as thou goest, unto Sodom, and Gomorrah, and Admah, and Zeboim, even unto Lasha."],
['Genesis','Gen','10','020','20',"These \emph{are} the sons of Ham, after their families, after their tongues, in their countries, \emph{and} in their nations."],
['Genesis','Gen','10','021','21',"Unto Shem also, the father of all the children of Eber, the brother of Japheth the elder, even to him were \emph{children} born."],
['Genesis','Gen','10','022','22',"The children of Shem; Elam, and Asshur, and Arphaxad, and Lud, and Aram."],
['Genesis','Gen','10','023','23',"And the children of Aram; Uz, and Hul, and Gether, and Mash."],
['Genesis','Gen','10','024','24',"And Arphaxad begat Salah; and Salah begat Eber."],
['Genesis','Gen','10','025','25',"And unto Eber were born two sons: the name of one \emph{was} Peleg; for in his days was the earth divided; and his brother's name \emph{was} Joktan."],
['Genesis','Gen','10','026','26',"And Joktan begat Almodad, and Sheleph, and Hazarmaveth, and Jerah,"],
['Genesis','Gen','10','027','27',"And Hadoram, and Uzal, and Diklah,"],
['Genesis','Gen','10','028','28',"And Obal, and Abimael, and Sheba,"],
['Genesis','Gen','10','029','29',"And Ophir, and Havilah, and Jobab: all these \emph{were} the sons of Joktan."],
['Genesis','Gen','10','030','30',"And their dwelling was from Mesha, as thou goest unto Sephar a mount of the east."],
['Genesis','Gen','10','031','31',"These \emph{are} the sons of Shem, after their families, after their tongues, in their lands, after their nations."],
['Genesis','Gen','10','032','32',"These \emph{are} the families of the sons of Noah, after their generations, in their nations: and by these were the nations divided in the earth after the flood."]
]

Genesis11 = [
    ['Genesis','Gen','11','001','1',"And the whole earth was of one language, and of one speech."],
['Genesis','Gen','11','002','2',"And it came to pass, as they journeyed from the east, that they found a plain in the land of Shinar; and they dwelt there."],
['Genesis','Gen','11','003','3',"And they said one to another, Go to, let us make brick, and burn them throughly. And they had brick for stone, and slime had they for morter."],
['Genesis','Gen','11','004','4',"And they said, Go to, let us build us a city and a tower, whose top \emph{may} \emph{reach} unto heaven; and let us make us a name, lest we be scattered abroad upon the face of the whole earth."],
['Genesis','Gen','11','005','5',"And the LORD came down to see the city and the tower, which the children of men builded."],
['Genesis','Gen','11','006','6',"And the LORD said, Behold, the people \emph{is} one, and they have all one language; and this they begin to do: and now nothing will be restrained from them, which they have imagined to do."],
['Genesis','Gen','11','007','7',"Go to, let us go down, and there confound their language, that they may not understand one another's speech."],
['Genesis','Gen','11','008','8',"So the LORD scattered them abroad from thence upon the face of all the earth: and they left off to build the city."],
['Genesis','Gen','11','009','9',"Therefore is the name of it called Babel; because the LORD did there confound the language of all the earth: and from thence did the LORD scatter them abroad upon the face of all the earth."],
['Genesis','Gen','11','010','10',"These \emph{are} the generations of Shem: Shem \emph{was} an hundred years old, and begat Arphaxad two years after the flood:"],
['Genesis','Gen','11','011','11',"And Shem lived after he begat Arphaxad five hundred years, and begat sons and daughters."],
['Genesis','Gen','11','012','12',"And Arphaxad lived five and thirty years, and begat Salah:"],
['Genesis','Gen','11','013','13',"And Arphaxad lived after he begat Salah four hundred and three years, and begat sons and daughters."],
['Genesis','Gen','11','014','14',"And Salah lived thirty years, and begat Eber:"],
['Genesis','Gen','11','015','15',"And Salah lived after he begat Eber four hundred and three years, and begat sons and daughters."],
['Genesis','Gen','11','016','16',"And Eber lived four and thirty years, and begat Peleg:"],
['Genesis','Gen','11','017','17',"And Eber lived after he begat Peleg four hundred and thirty years, and begat sons and daughters."],
['Genesis','Gen','11','018','18',"And Peleg lived thirty years, and begat Reu:"],
['Genesis','Gen','11','019','19',"And Peleg lived after he begat Reu two hundred and nine years, and begat sons and daughters."],
['Genesis','Gen','11','020','20',"And Reu lived two and thirty years, and begat Serug:"],
['Genesis','Gen','11','021','21',"And Reu lived after he begat Serug two hundred and seven years, and begat sons and daughters."],
['Genesis','Gen','11','022','22',"And Serug lived thirty years, and begat Nahor:"],
['Genesis','Gen','11','023','23',"And Serug lived after he begat Nahor two hundred years, and begat sons and daughters."],
['Genesis','Gen','11','024','24',"And Nahor lived nine and twenty years, and begat Terah:"],
['Genesis','Gen','11','025','25',"And Nahor lived after he begat Terah an hundred and nineteen years, and begat sons and daughters."],
['Genesis','Gen','11','026','26',"And Terah lived seventy years, and begat Abram, Nahor, and Haran."],
['Genesis','Gen','11','027','27',"Now these \emph{are} the generations of Terah: Terah begat Abram, Nahor, and Haran; and Haran begat Lot."],
['Genesis','Gen','11','028','28',"And Haran died before his father Terah in the land of his nativity, in Ur of the Chaldees."],
['Genesis','Gen','11','029','29',"And Abram and Nahor took them wives: the name of Abram's wife \emph{was} Sarai; and the name of Nahor's wife, Milcah, the daughter of Haran, the father of Milcah, and the father of Iscah."],
['Genesis','Gen','11','030','30',"But Sarai was barren; she \emph{had} no child."],
['Genesis','Gen','11','031','31',"And Terah took Abram his son, and Lot the son of Haran his son's son, and Sarai his daughter in law, his son Abram's wife; and they went forth with them from Ur of the Chaldees, to go into the land of Canaan; and they came unto Haran, and dwelt there."],
['Genesis','Gen','11','032','32',"And the days of Terah were two hundred and five years: and Terah died in Haran."]
]

Genesis12 = [
    ['Genesis','Gen','12','001','1',"Now the LORD had said unto Abram, Get thee out of thy country, and from thy kindred, and from thy father's house, unto a land that I will shew thee:"],
['Genesis','Gen','12','002','2',"And I will make of thee a great nation, and I will bless thee, and make thy name great; and thou shalt be a blessing:"],
['Genesis','Gen','12','003','3',"And I will bless them that bless thee, and curse him that curseth thee: and in thee shall all families of the earth be blessed."],
['Genesis','Gen','12','004','4',"So Abram departed, as the LORD had spoken unto him; and Lot went with him: and Abram \emph{was} seventy and five years old when he departed out of Haran."],
['Genesis','Gen','12','005','5',"And Abram took Sarai his wife, and Lot his brother's son, and all their substance that they had gathered, and the souls that they had gotten in Haran; and they went forth to go into the land of Canaan; and into the land of Canaan they came."],
['Genesis','Gen','12','006','6',"And Abram passed through the land unto the place of Sichem, unto the plain of Moreh. And the Canaanite \emph{was} then in the land."],
['Genesis','Gen','12','007','7',"And the LORD appeared unto Abram, and said, Unto thy seed will I give this land: and there builded he an altar unto the LORD, who appeared unto him."],
['Genesis','Gen','12','008','8',"And he removed from thence unto a mountain on the east of Beth-el, and pitched his tent, \emph{having} Beth-el on the west, and Hai on the east: and there he builded an altar unto the LORD, and called upon the name of the LORD."],
['Genesis','Gen','12','009','9',"And Abram journeyed, going on still toward the south."],
['Genesis','Gen','12','010','10',"And there was a famine in the land: and Abram went down into Egypt to sojourn there; for the famine \emph{was} grievous in the land."],
['Genesis','Gen','12','011','11',"And it came to pass, when he was come near to enter into Egypt, that he said unto Sarai his wife, Behold now, I know that thou \emph{art} a fair woman to look upon:"],
['Genesis','Gen','12','012','12',"Therefore it shall come to pass, when the Egyptians shall see thee, that they shall say, This \emph{is} his wife: and they will kill me, but they will save thee alive."],
['Genesis','Gen','12','013','13',"Say, I pray thee, thou \emph{art} my sister: that it may be well with me for thy sake; and my soul shall live because of thee."],
['Genesis','Gen','12','014','14',"And it came to pass, that, when Abram was come into Egypt, the Egyptians beheld the woman that she \emph{was} very fair."],
['Genesis','Gen','12','015','15',"The princes also of Pharaoh saw her, and commended her before Pharaoh: and the woman was taken into Pharaoh's house."],
['Genesis','Gen','12','016','16',"And he entreated Abram well for her sake: and he had sheep, and oxen, and he asses, and menservants, and maidservants, and she asses, and camels."],
['Genesis','Gen','12','017','17',"And the LORD plagued Pharaoh and his house with great plagues because of Sarai Abram's wife."],
['Genesis','Gen','12','018','18',"And Pharaoh called Abram, and said, What \emph{is} this \emph{that} thou hast done unto me? why didst thou not tell me that she \emph{was} thy wife?"],
['Genesis','Gen','12','019','19',"Why saidst thou, She \emph{is} my sister? so I might have taken her to me to wife: now therefore behold thy wife, take \emph{her}, and go thy way."],
['Genesis','Gen','12','020','20',"And Pharaoh commanded \emph{his} men concerning him: and they sent him away, and his wife, and all that he had."]
]

Genesis13 = [
    ['Genesis','Gen','13','001','1',"And Abram went up out of Egypt, he, and his wife, and all that he had, and Lot with him, into the south."],
['Genesis','Gen','13','002','2',"And Abram \emph{was} very rich in cattle, in silver, and in gold."],
['Genesis','Gen','13','003','3',"And he went on his journeys from the south even to Beth-el, unto the place where his tent had been at the beginning, between Beth-el and Hai;"],
['Genesis','Gen','13','004','4',"Unto the place of the altar, which he had made there at the first: and there Abram called on the name of the LORD."],
['Genesis','Gen','13','005','5',"And Lot also, which went with Abram, had flocks, and herds, and tents."],
['Genesis','Gen','13','006','6',"And the land was not able to bear them, that they might dwell together: for their substance was great, so that they could not dwell together."],
['Genesis','Gen','13','007','7',"And there was a strife between the herdmen of Abram's cattle and the herdmen of Lot's cattle: and the Canaanite and the Perizzite dwelled then in the land."],
['Genesis','Gen','13','008','8',"And Abram said unto Lot, Let there be no strife, I pray thee, between me and thee, and between my herdmen and thy herdmen; for we \emph{be} brethren."],
['Genesis','Gen','13','009','9',"\emph{Is} not the whole land before thee? separate thyself, I pray thee, from me: if \emph{thou} \emph{wilt} \emph{take} the left hand, then I will go to the right; or if \emph{thou} \emph{depart} to the right hand, then I will go to the left."],
['Genesis','Gen','13','010','10',"And Lot lifted up his eyes, and beheld all the plain of Jordan, that it \emph{was} well watered every where, before the LORD destroyed Sodom and Gomorrah, \emph{even} as the garden of the LORD, like the land of Egypt, as thou comest unto Zoar."],
['Genesis','Gen','13','011','11',"Then Lot chose him all the plain of Jordan; and Lot journeyed east: and they separated themselves the one from the other."],
['Genesis','Gen','13','012','12',"Abram dwelled in the land of Canaan, and Lot dwelled in the cities of the plain, and pitched \emph{his} tent toward Sodom."],
['Genesis','Gen','13','013','13',"But the men of Sodom \emph{were} wicked and sinners before the LORD exceedingly."],
['Genesis','Gen','13','014','14',"And the LORD said unto Abram, after that Lot was separated from him, Lift up now thine eyes, and look from the place where thou art northward, and southward, and eastward, and westward:"],
['Genesis','Gen','13','015','15',"For all the land which thou seest, to thee will I give it, and to thy seed for ever."],
['Genesis','Gen','13','016','16',"And I will make thy seed as the dust of the earth: so that if a man can number the dust of the earth, \emph{then} shall thy seed also be numbered."],
['Genesis','Gen','13','017','17',"Arise, walk through the land in the length of it and in the breadth of it; for I will give it unto thee."],
['Genesis','Gen','13','018','18',"Then Abram removed \emph{his} tent, and came and dwelt in the plain of Mamre, which \emph{is} in Hebron, and built there an altar unto the LORD."]
]

Genesis14 = [
    ['Genesis','Gen','14','001','1',"And it came to pass in the days of Amraphel king of Shinar, Arioch king of Ellasar, Chedorlaomer king of Elam, and Tidal king of nations;"],
['Genesis','Gen','14','002','2',"\emph{That} \emph{these} made war with Bera king of Sodom, and with Birsha king of Gomorrah, Shinab king of Admah, and Shemeber king of Zeboiim, and the king of Bela, which is Zoar."],
['Genesis','Gen','14','003','3',"All these were joined together in the vale of Siddim, which is the salt sea."],
['Genesis','Gen','14','004','4',"Twelve years they served Chedorlaomer, and in the thirteenth year they rebelled."],
['Genesis','Gen','14','005','5',"And in the fourteenth year came Chedorlaomer, and the kings that \emph{were} with him, and smote the Rephaims in Ashteroth Karnaim, and the Zuzims in Ham, and the Emims in Shaveh Kiriathaim,"],
['Genesis','Gen','14','006','6',"And the Horites in their mount Seir, unto El-paran, which \emph{is} by the wilderness."],
['Genesis','Gen','14','007','7',"And they returned, and came to En-mishpat, which \emph{is} Kadesh, and smote all the country of the Amalekites, and also the Amorites, that dwelt in Hazezon-tamar."],
['Genesis','Gen','14','008','8',"And there went out the king of Sodom, and the king of Gomorrah, and the king of Admah, and the king of Zeboiim, and the king of Bela (the same \emph{is} Zoar;) and they joined battle with them in the vale of Siddim;"],
['Genesis','Gen','14','009','9',"With Chedorlaomer the king of Elam, and with Tidal king of nations, and Amraphel king of Shinar, and Arioch king of Ellasar; four kings with five."],
['Genesis','Gen','14','010','10',"And the vale of Siddim \emph{was} \emph{full} \emph{of} slimepits; and the kings of Sodom and Gomorrah fled, and fell there; and they that remained fled to the mountain."],
['Genesis','Gen','14','011','11',"And they took all the goods of Sodom and Gomorrah, and all their victuals, and went their way."],
['Genesis','Gen','14','012','12',"And they took Lot, Abram's brother's son, who dwelt in Sodom, and his goods, and departed."],
['Genesis','Gen','14','013','13',"And there came one that had escaped, and told Abram the Hebrew; for he dwelt in the plain of Mamre the Amorite, brother of Eshcol, and brother of Aner: and these \emph{were} confederate with Abram."],
['Genesis','Gen','14','014','14',"And when Abram heard that his brother was taken captive, he armed his trained \emph{servants}, born in his own house, three hundred and eighteen, and pursued \emph{them} unto Dan."],
['Genesis','Gen','14','015','15',"And he divided himself against them, he and his servants, by night, and smote them, and pursued them unto Hobah, which \emph{is} on the left hand of Damascus."],
['Genesis','Gen','14','016','16',"And he brought back all the goods, and also brought again his brother Lot, and his goods, and the women also, and the people."],
['Genesis','Gen','14','017','17',"And the king of Sodom went out to meet him after his return from the slaughter of Chedorlaomer, and of the kings that \emph{were} with him, at the valley of Shaveh, which \emph{is} the king's dale."],
['Genesis','Gen','14','018','18',"And Melchizedek king of Salem brought forth bread and wine: and he \emph{was} the priest of the most high God."],
['Genesis','Gen','14','019','19',"And he blessed him, and said, Blessed \emph{be} Abram of the most high God, possessor of heaven and earth:"],
['Genesis','Gen','14','020','20',"And blessed be the most high God, which hath delivered thine enemies into thy hand. And he gave him tithes of all."],
['Genesis','Gen','14','021','21',"And the king of Sodom said unto Abram, Give me the persons, and take the goods to thyself."],
['Genesis','Gen','14','022','22',"And Abram said to the king of Sodom, I have lift up mine hand unto the LORD, the most high God, the possessor of heaven and earth,"],
['Genesis','Gen','14','023','23',"That I will not \emph{take} from a thread even to a shoelatchet, and that I will not take any thing that \emph{is} thine, lest thou shouldest say, I have made Abram rich:"],
['Genesis','Gen','14','024','24',"Save only that which the young men have eaten, and the portion of the men which went with me, Aner, Eshcol, and Mamre; let them take their portion."]
]

Genesis15 = [
    ['Genesis','Gen','15','001','1',"After these things the word of the LORD came unto Abram in a vision, saying, Fear not, Abram: I \emph{am} thy shield, \emph{and} thy exceeding great reward."],
['Genesis','Gen','15','002','2',"And Abram said, Lord GOD, what wilt thou give me, seeing I go childless, and the steward of my house \emph{is} this Eliezer of Damascus?"],
['Genesis','Gen','15','003','3',"And Abram said, Behold, to me thou hast given no seed: and, lo, one born in my house is mine heir."],
['Genesis','Gen','15','004','4',"And, behold, the word of the LORD \emph{came} unto him, saying, This shall not be thine heir; but he that shall come forth out of thine own bowels shall be thine heir."],
['Genesis','Gen','15','005','5',"And he brought him forth abroad, and said, Look now toward heaven, and tell the stars, if thou be able to number them: and he said unto him, So shall thy seed be."],
['Genesis','Gen','15','006','6',"And he believed in the LORD; and he counted it to him for righteousness."],
['Genesis','Gen','15','007','7',"And he said unto him, I \emph{am} the LORD that brought thee out of Ur of the Chaldees, to give thee this land to inherit it."],
['Genesis','Gen','15','008','8',"And he said, Lord GOD, whereby shall I know that I shall inherit it?"],
['Genesis','Gen','15','009','9',"And he said unto him, Take me an heifer of three years old, and a she goat of three years old, and a ram of three years old, and a turtledove, and a young pigeon."],
['Genesis','Gen','15','010','10',"And he took unto him all these, and divided them in the midst, and laid each piece one against another: but the birds divided he not."],
['Genesis','Gen','15','011','11',"And when the fowls came down upon the carcases, Abram drove them away."],
['Genesis','Gen','15','012','12',"And when the sun was going down, a deep sleep fell upon Abram; and, lo, an horror of great darkness fell upon him."],
['Genesis','Gen','15','013','13',"And he said unto Abram, Know of a surety that thy seed shall be a stranger in a land \emph{that} \emph{is} not their's, and shall serve them; and they shall afflict them four hundred years;"],
['Genesis','Gen','15','014','14',"And also that nation, whom they shall serve, will I judge: and afterward shall they come out with great substance."],
['Genesis','Gen','15','015','15',"And thou shalt go to thy fathers in peace; thou shalt be buried in a good old age."],
['Genesis','Gen','15','016','16',"But in the fourth generation they shall come hither again: for the iniquity of the Amorites \emph{is} not yet full."],
['Genesis','Gen','15','017','17',"And it came to pass, that, when the sun went down, and it was dark, behold a smoking furnace, and a burning lamp that passed between those pieces."],
['Genesis','Gen','15','018','18',"In the same day the LORD made a covenant with Abram, saying, Unto thy seed have I given this land, from the river of Egypt unto the great river, the river Euphrates:"],
['Genesis','Gen','15','019','19',"The Kenites, and the Kenizzites, and the Kadmonites,"],
['Genesis','Gen','15','020','20',"And the Hittites, and the Perizzites, and the Rephaims,"],
['Genesis','Gen','15','021','21',"And the Amorites, and the Canaanites, and the Girgashites, and the Jebusites."]
]

    
Genesis16 = [
    ['Genesis','Gen','16','001','1',"Now Sarai Abram's wife bare him no children: and she had an handmaid, an Egyptian, whose name \emph{was} Hagar."],
    ['Genesis','Gen','16','002','2',"And Sarai said unto Abram, Behold now, the LORD hath restrained me from bearing: I pray thee, go in unto my maid; it may be that I may obtain children by her. And Abram hearkened to the voice of Sarai."],
    ['Genesis','Gen','16','003','3',"And Sarai Abram's wife took Hagar her maid the Egyptian, after Abram had dwelt ten years in the land of Canaan, and gave her to her husband Abram to be his wife."],
    ['Genesis','Gen','16','004','4',"And he went in unto Hagar, and she conceived: and when she saw that she had conceived, her mistress was despised in her eyes."],
    ['Genesis','Gen','16','005','5',"And Sarai said unto Abram, My wrong \emph{be} upon thee: I have given my maid into thy bosom; and when she saw that she had conceived, I was despised in her eyes: the LORD judge between me and thee."],
    ['Genesis','Gen','16','006','6',"But Abram said unto Sarai, Behold, thy maid \emph{is} in thy hand; do to her as it pleaseth thee. And when Sarai dealt hardly with her, she fled from her face."],
    ['Genesis','Gen','16','007','7',"And the angel of the LORD found her by a fountain of water in the wilderness, by the fountain in the way to Shur."],
    ['Genesis','Gen','16','008','8',"And he said, Hagar, Sarai's maid, whence camest thou? and whither wilt thou go? And she said, I flee from the face of my mistress Sarai."],
    ['Genesis','Gen','16','009','9',"And the angel of the LORD said unto her, Return to thy mistress, and submit thyself under her hands."],
    ['Genesis','Gen','16','010','10',"And the angel of the LORD said unto her, I will multiply thy seed exceedingly, that it shall not be numbered for multitude."],
    ['Genesis','Gen','16','011','11',"And the angel of the LORD said unto her, Behold, thou \emph{art} with child, and shalt bear a son, and shalt call his name Ishmael; because the LORD hath heard thy affliction."],
    ['Genesis','Gen','16','012','12',"And he will be a wild man; his hand \emph{will} \emph{be} against every man, and every man's hand against him; and he shall dwell in the presence of all his brethren."],
    ['Genesis','Gen','16','013','13',"And she called the name of the LORD that spake unto her, Thou God seest me: for she said, Have I also here looked after him that seeth me?"],
    ['Genesis','Gen','16','014','14',"Wherefore the well was called Beer-lahai-roi; behold, \emph{it} \emph{is} between Kadesh and Bered."],
    ['Genesis','Gen','16','015','15',"And Hagar bare Abram a son: and Abram called his son's name, which Hagar bare, Ishmael."],
    ['Genesis','Gen','16','016','16',"And Abram \emph{was} fourscore and six years old, when Hagar bare Ishmael to Abram."]
]
    
Genesis17 = [
    ['Genesis','Gen','17','001','1',"And when Abram was ninety years old and nine, the LORD appeared to Abram, and said unto him, I \emph{am} the Almighty God; walk before me, and be thou perfect."],
    ['Genesis','Gen','17','002','2',"And I will make my covenant between me and thee, and will multiply thee exceedingly."],
    ['Genesis','Gen','17','003','3',"And Abram fell on his face: and God talked with him, saying,"],
    ['Genesis','Gen','17','004','4',"As for me, behold, my covenant \emph{is} with thee, and thou shalt be a father of many nations."],
    ['Genesis','Gen','17','005','5',"Neither shall thy name any more be called Abram, but thy name shall be Abraham; for a father of many nations have I made thee."],
    ['Genesis','Gen','17','006','6',"And I will make thee exceeding fruitful, and I will make nations of thee, and kings shall come out of thee."],
    ['Genesis','Gen','17','007','7',"And I will establish my covenant between me and thee and thy seed after thee in their generations for an everlasting covenant, to be a God unto thee, and to thy seed after thee."],
    ['Genesis','Gen','17','008','8',"And I will give unto thee, and to thy seed after thee, the land wherein thou art a stranger, all the land of Canaan, for an everlasting possession; and I will be their God."],
    ['Genesis','Gen','17','009','9',"And God said unto Abraham, Thou shalt keep my covenant therefore, thou, and thy seed after thee in their generations."],
    ['Genesis','Gen','17','010','10',"This \emph{is} my covenant, which ye shall keep, between me and you and thy seed after thee; Every man child among you shall be circumcised."],
    ['Genesis','Gen','17','011','11',"And ye shall circumcise the flesh of your foreskin; and it shall be a token of the covenant betwixt me and you."],
    ['Genesis','Gen','17','012','12',"And he that is eight days old shall be circumcised among you, every man child in your generations, he that is born in the house, or bought with money of any stranger, which \emph{is} not of thy seed."],
    ['Genesis','Gen','17','013','13',"He that is born in thy house, and he that is bought with thy money, must needs be circumcised: and my covenant shall be in your flesh for an everlasting covenant."],
    ['Genesis','Gen','17','014','14',"And the uncircumcised man child whose flesh of his foreskin is not circumcised, that soul shall be cut off from his people; he hath broken my covenant."],
    ['Genesis','Gen','17','015','15',"And God said unto Abraham, As for Sarai thy wife, thou shalt not call her name Sarai, but Sarah \emph{shall} her name \emph{be}."],
    ['Genesis','Gen','17','016','16',"And I will bless her, and give thee a son also of her: yea, I will bless her, and she shall be \emph{a} \emph{mother} of nations; kings of people shall be of her."],
    ['Genesis','Gen','17','017','17',"Then Abraham fell upon his face, and laughed, and said in his heart, Shall \emph{a} \emph{child} be born unto him that is an hundred years old? and shall Sarah, that is ninety years old, bear?"],
    ['Genesis','Gen','17','018','18',"And Abraham said unto God, O that Ishmael might live before thee!"],
    ['Genesis','Gen','17','019','19',"And God said, Sarah thy wife shall bear thee a son indeed; and thou shalt call his name Isaac: and I will establish my covenant with him for an everlasting covenant, \emph{and} with his seed after him."],
    ['Genesis','Gen','17','020','20',"And as for Ishmael, I have heard thee: Behold, I have blessed him, and will make him fruitful, and will multiply him exceedingly; twelve princes shall he beget, and I will make him a great nation."],
    ['Genesis','Gen','17','021','21',"But my covenant will I establish with Isaac, which Sarah shall bear unto thee at this set time in the next year."],
    ['Genesis','Gen','17','022','22',"And he left off talking with him, and God went up from Abraham."],
    ['Genesis','Gen','17','023','23',"And Abraham took Ishmael his son, and all that were born in his house, and all that were bought with his money, every male among the men of Abraham's house; and circumcised the flesh of their foreskin in the selfsame day, as God had said unto him."],
    ['Genesis','Gen','17','024','24',"And Abraham \emph{was} ninety years old and nine, when he was circumcised in the flesh of his foreskin."],
    ['Genesis','Gen','17','025','25',"And Ishmael his son \emph{was} thirteen years old, when he was circumcised in the flesh of his foreskin."],
    ['Genesis','Gen','17','026','26',"In the selfsame day was Abraham circumcised, and Ishmael his son."],
    ['Genesis','Gen','17','027','27',"And all the men of his house, born in the house, and bought with money of the stranger, were circumcised with him."]
]
    
Genesis18 = [
['Genesis','Gen','18','001','1',"And the LORD appeared unto him in the plains of Mamre: and he sat in the tent door in the heat of the day;"],
['Genesis','Gen','18','002','2',"And he lift up his eyes and looked, and, lo, three men stood by him: and when he saw \emph{them}, he ran to meet them from the tent door, and bowed himself toward the ground,"],
['Genesis','Gen','18','003','3',"And said, My Lord, if now I have found favour in thy sight, pass not away, I pray thee, from thy servant:"],
['Genesis','Gen','18','004','4',"Let a little water, I pray you, be fetched, and wash your feet, and rest yourselves under the tree:"],
['Genesis','Gen','18','005','5',"And I will fetch a morsel of bread, and comfort ye your hearts; after that ye shall pass on: for therefore are ye come to your servant. And they said, So do, as thou hast said."],
['Genesis','Gen','18','006','6',"And Abraham hastened into the tent unto Sarah, and said, Make ready quickly three measures of fine meal, knead \emph{it}, and make cakes upon the hearth."],
['Genesis','Gen','18','007','7',"And Abraham ran unto the herd, and fetcht a calf tender and good, and gave \emph{it} unto a young man; and he hasted to dress it."],
['Genesis','Gen','18','008','8',"And he took butter, and milk, and the calf which he had dressed, and set \emph{it} before them; and he stood by them under the tree, and they did eat."],
['Genesis','Gen','18','009','9',"And they said unto him, Where \emph{is} Sarah thy wife? And he said, Behold, in the tent."],
['Genesis','Gen','18','010','10',"And he said, I will certainly return unto thee according to the time of life; and, lo, Sarah thy wife shall have a son. And Sarah heard \emph{it} in the tent door, which \emph{was} behind him."],
['Genesis','Gen','18','011','11',"Now Abraham and Sarah \emph{were} old \emph{and} well stricken in age; \emph{and} it ceased to be with Sarah after the manner of women."],
['Genesis','Gen','18','012','12',"Therefore Sarah laughed within herself, saying, After I am waxed old shall I have pleasure, my lord being old also?"],
['Genesis','Gen','18','013','13',"And the LORD said unto Abraham, Wherefore did Sarah laugh, saying, Shall I of a surety bear a child, which am old?"],
['Genesis','Gen','18','014','14',"Is any thing too hard for the LORD? At the time appointed I will return unto thee, according to the time of life, and Sarah shall have a son."],
['Genesis','Gen','18','015','15',"Then Sarah denied, saying, I laughed not; for she was afraid. And he said, Nay; but thou didst laugh."],
['Genesis','Gen','18','016','16',"And the men rose up from thence, and looked toward Sodom: and Abraham went with them to bring them on the way."],
['Genesis','Gen','18','017','17',"And the LORD said, Shall I hide from Abraham that thing which I do;"],
['Genesis','Gen','18','018','18',"Seeing that Abraham shall surely become a great and mighty nation, and all the nations of the earth shall be blessed in him?"],
['Genesis','Gen','18','019','19',"For I know him, that he will command his children and his household after him, and they shall keep the way of the LORD, to do justice and judgment; that the LORD may bring upon Abraham that which he hath spoken of him."],
['Genesis','Gen','18','020','20',"And the LORD said, Because the cry of Sodom and Gomorrah is great, and because their sin is very grievous;"],
['Genesis','Gen','18','021','21',"I will go down now, and see whether they have done altogether according to the cry of it, which is come unto me; and if not, I will know."],
['Genesis','Gen','18','022','22',"And the men turned their faces from thence, and went toward Sodom: but Abraham stood yet before the LORD."],
['Genesis','Gen','18','023','23',"And Abraham drew near, and said, Wilt thou also destroy the righteous with the wicked?"],
['Genesis','Gen','18','024','24',"Peradventure there be fifty righteous within the city: wilt thou also destroy and not spare the place for the fifty righteous that \emph{are} therein?"],
['Genesis','Gen','18','025','25',"That be far from thee to do after this manner, to slay the righteous with the wicked: and that the righteous should be as the wicked, that be far from thee: Shall not the Judge of all the earth do right?"],
['Genesis','Gen','18','026','26',"And the LORD said, If I find in Sodom fifty righteous within the city, then I will spare all the place for their sakes."],
['Genesis','Gen','18','027','27',"And Abraham answered and said, Behold now, I have taken upon me to speak unto the Lord, which \emph{am} \emph{but} dust and ashes:"],
['Genesis','Gen','18','028','28',"Peradventure there shall lack five of the fifty righteous: wilt thou destroy all the city for \emph{lack} \emph{of} five? And he said, If I find there forty and five, I will not destroy \emph{it}."],
['Genesis','Gen','18','029','29',"And he spake unto him yet again, and said, Peradventure there shall be forty found there. And he said, I will not do \emph{it} for forty's sake."],
['Genesis','Gen','18','030','30',"And he said \emph{unto} \emph{him}, Oh let not the Lord be angry, and I will speak: Peradventure there shall thirty be found there. And he said, I will not do \emph{it}, if I find thirty there."],
['Genesis','Gen','18','031','31',"And he said, Behold now, I have taken upon me to speak unto the Lord: Peradventure there shall be twenty found there. And he said, I will not destroy \emph{it} for twenty's sake."],
['Genesis','Gen','18','032','32',"And he said, Oh let not the Lord be angry, and I will speak yet but this once: Peradventure ten shall be found there. And he said, I will not destroy \emph{it} for ten's sake."],
['Genesis','Gen','18','033','33',"And the LORD went his way, as soon as he had left communing with Abraham: and Abraham returned unto his place."]
]

Genesis19 = [
    ['Genesis','Gen','19','001','1',"And there came two angels to Sodom at even; and Lot sat in the gate of Sodom: and Lot seeing \emph{them} rose up to meet them; and he bowed himself with his face toward the ground;"],
['Genesis','Gen','19','002','2',"And he said, Behold now, my lords, turn in, I pray you, into your servant's house, and tarry all night, and wash your feet, and ye shall rise up early, and go on your ways. And they said, Nay; but we will abide in the street all night."],
['Genesis','Gen','19','003','3',"And he pressed upon them greatly; and they turned in unto him, and entered into his house; and he made them a feast, and did bake unleavened bread, and they did eat."],
['Genesis','Gen','19','004','4',"But before they lay down, the men of the city, \emph{even} the men of Sodom, compassed the house round, both old and young, all the people from every quarter:"],
['Genesis','Gen','19','005','5',"And they called unto Lot, and said unto him, Where \emph{are} the men which came in to thee this night? bring them out unto us, that we may know them."],
['Genesis','Gen','19','006','6',"And Lot went out at the door unto them, and shut the door after him,"],
['Genesis','Gen','19','007','7',"And said, I pray you, brethren, do not so wickedly."],
['Genesis','Gen','19','008','8',"Behold now, I have two daughters which have not known man; let me, I pray you, bring them out unto you, and do ye to them as \emph{is} good in your eyes: only unto these men do nothing; for therefore came they under the shadow of my roof."],
['Genesis','Gen','19','009','9',"And they said, Stand back. And they said \emph{again}, This one \emph{fellow} came in to sojourn, and he will needs be a judge: now will we deal worse with thee, than with them. And they pressed sore upon the man, \emph{even} Lot, and came near to break the door."],
['Genesis','Gen','19','010','10',"But the men put forth their hand, and pulled Lot into the house to them, and shut to the door."],
['Genesis','Gen','19','011','11',"And they smote the men that \emph{were} at the door of the house with blindness, both small and great: so that they wearied themselves to find the door."],
['Genesis','Gen','19','012','12',"And the men said unto Lot, Hast thou here any besides? son in law, and thy sons, and thy daughters, and whatsoever thou hast in the city, bring \emph{them} out of this place:"],
['Genesis','Gen','19','013','13',"For we will destroy this place, because the cry of them is waxen great before the face of the LORD; and the LORD hath sent us to destroy it."],
['Genesis','Gen','19','014','14',"And Lot went out, and spake unto his sons in law, which married his daughters, and said, Up, get you out of this place; for the LORD will destroy this city. But he seemed as one that mocked unto his sons in law."],
['Genesis','Gen','19','015','15',"And when the morning arose, then the angels hastened Lot, saying, Arise, take thy wife, and thy two daughters, which are here; lest thou be consumed in the iniquity of the city."],
['Genesis','Gen','19','016','16',"And while he lingered, the men laid hold upon his hand, and upon the hand of his wife, and upon the hand of his two daughters; the LORD being merciful unto him: and they brought him forth, and set him without the city."],
['Genesis','Gen','19','017','17',"And it came to pass, when they had brought them forth abroad, that he said, Escape for thy life; look not behind thee, neither stay thou in all the plain; escape to the mountain, lest thou be consumed."],
['Genesis','Gen','19','018','18',"And Lot said unto them, Oh, not so, my Lord:"],
['Genesis','Gen','19','019','19',"Behold now, thy servant hath found grace in thy sight, and thou hast magnified thy mercy, which thou hast shewed unto me in saving my life; and I cannot escape to the mountain, lest some evil take me, and I die:"],
['Genesis','Gen','19','020','20',"Behold now, this city \emph{is} near to flee unto, and it \emph{is} a little one: Oh, let me escape thither, \emph{(is} it not a little one?) and my soul shall live."],
['Genesis','Gen','19','021','21',"And he said unto him, See, I have accepted thee concerning this thing also, that I will not overthrow this city, for the which thou hast spoken."],
['Genesis','Gen','19','022','22',"Haste thee, escape thither; for I cannot do any thing till thou be come thither. Therefore the name of the city was called Zoar."],
['Genesis','Gen','19','023','23',"The sun was risen upon the earth when Lot entered into Zoar."],
['Genesis','Gen','19','024','24',"Then the LORD rained upon Sodom and upon Gomorrah brimstone and fire from the LORD out of heaven;"],
['Genesis','Gen','19','025','25',"And he overthrew those cities, and all the plain, and all the inhabitants of the cities, and that which grew upon the ground."],
['Genesis','Gen','19','026','26',"But his wife looked back from behind him, and she became a pillar of salt."],
['Genesis','Gen','19','027','27',"And Abraham gat up early in the morning to the place where he stood before the LORD:"],
['Genesis','Gen','19','028','28',"And he looked toward Sodom and Gomorrah, and toward all the land of the plain, and beheld, and, lo, the smoke of the country went up as the smoke of a furnace."],
['Genesis','Gen','19','029','29',"And it came to pass, when God destroyed the cities of the plain, that God remembered Abraham, and sent Lot out of the midst of the overthrow, when he overthrew the cities in the which Lot dwelt."],
['Genesis','Gen','19','030','30',"And Lot went up out of Zoar, and dwelt in the mountain, and his two daughters with him; for he feared to dwell in Zoar: and he dwelt in a cave, he and his two daughters."],
['Genesis','Gen','19','031','31',"And the firstborn said unto the younger, Our father \emph{is} old, and \emph{there} \emph{is} not a man in the earth to come in unto us after the manner of all the earth:"],
['Genesis','Gen','19','032','32',"Come, let us make our father drink wine, and we will lie with him, that we may preserve seed of our father."],
['Genesis','Gen','19','033','33',"And they made their father drink wine that night: and the firstborn went in, and lay with her father; and he perceived not when she lay down, nor when she arose."],
['Genesis','Gen','19','034','34',"And it came to pass on the morrow, that the firstborn said unto the younger, Behold, I lay yesternight with my father: let us make him drink wine this night also; and go thou in, \emph{and} lie with him, that we may preserve seed of our father."],
['Genesis','Gen','19','035','35',"And they made their father drink wine that night also: and the younger arose, and lay with him; and he perceived not when she lay down, nor when she arose."],
['Genesis','Gen','19','036','36',"Thus were both the daughters of Lot with child by their father."],
['Genesis','Gen','19','037','37',"And the firstborn bare a son, and called his name Moab: the same \emph{is} the father of the Moabites unto this day."],
['Genesis','Gen','19','038','38',"And the younger, she also bare a son, and called his name Ben-ammi: the same \emph{is} the father of the children of Ammon unto this day."]
]

Genesis20 = [
    ['Genesis','Gen','20','001','1',"And Abraham journeyed from thence toward the south country, and dwelled between Kadesh and Shur, and sojourned in Gerar."],
['Genesis','Gen','20','002','2',"And Abraham said of Sarah his wife, She \emph{is} my sister: and Abimelech king of Gerar sent, and took Sarah."],
['Genesis','Gen','20','003','3',"But God came to Abimelech in a dream by night, and said to him, Behold, thou \emph{art} \emph{but} a dead man, for the woman which thou hast taken; for she \emph{is} a man's wife."],
['Genesis','Gen','20','004','4',"But Abimelech had not come near her: and he said, Lord, wilt thou slay also a righteous nation?"],
['Genesis','Gen','20','005','5',"Said he not unto me, She \emph{is} my sister? and she, even she herself said, He \emph{is} my brother: in the integrity of my heart and innocency of my hands have I done this."],
['Genesis','Gen','20','006','6',"And God said unto him in a dream, Yea, I know that thou didst this in the integrity of thy heart; for I also withheld thee from sinning against me: therefore suffered I thee not to touch her."],
['Genesis','Gen','20','007','7',"Now therefore restore the man \emph{his} wife; for he \emph{is} a prophet, and he shall pray for thee, and thou shalt live: and if thou restore \emph{her} not, know thou that thou shalt surely die, thou, and all that \emph{are} thine."],
['Genesis','Gen','20','008','8',"Therefore Abimelech rose early in the morning, and called all his servants, and told all these things in their ears: and the men were sore afraid."],
['Genesis','Gen','20','009','9',"Then Abimelech called Abraham, and said unto him, What hast thou done unto us? and what have I offended thee, that thou hast brought on me and on my kingdom a great sin? thou hast done deeds unto me that ought not to be done."],
['Genesis','Gen','20','010','10',"And Abimelech said unto Abraham, What sawest thou, that thou hast done this thing?"],
['Genesis','Gen','20','011','11',"And Abraham said, Because I thought, Surely the fear of God \emph{is} not in this place; and they will slay me for my wife's sake."],
['Genesis','Gen','20','012','12',"And yet indeed \emph{she} \emph{is} my sister; she \emph{is} the daughter of my father, but not the daughter of my mother; and she became my wife."],
['Genesis','Gen','20','013','13',"And it came to pass, when God caused me to wander from my father's house, that I said unto her, This \emph{is} thy kindness which thou shalt shew unto me; at every place whither we shall come, say of me, He \emph{is} my brother."],
['Genesis','Gen','20','014','14',"And Abimelech took sheep, and oxen, and menservants, and womenservants, and gave \emph{them} unto Abraham, and restored him Sarah his wife."],
['Genesis','Gen','20','015','15',"And Abimelech said, Behold, my land \emph{is} before thee: dwell where it pleaseth thee."],
['Genesis','Gen','20','016','16',"And unto Sarah he said, Behold, I have given thy brother a thousand \emph{pieces} of silver: behold, he \emph{is} to thee a covering of the eyes, unto all that \emph{are} with thee, and with all \emph{other}: thus she was reproved."],
['Genesis','Gen','20','017','17',"So Abraham prayed unto God: and God healed Abimelech, and his wife, and his maidservants; and they bare \emph{children}."],
['Genesis','Gen','20','018','18',"For the LORD had fast closed up all the wombs of the house of Abimelech, because of Sarah Abraham's wife."]
]

Genesis21= [
    ['Genesis','Gen','21','001','1',"And the LORD visited Sarah as he had said, and the LORD did unto Sarah as he had spoken."],
['Genesis','Gen','21','002','2',"For Sarah conceived, and bare Abraham a son in his old age, at the set time of which God had spoken to him."],
['Genesis','Gen','21','003','3',"And Abraham called the name of his son that was born unto him, whom Sarah bare to him, Isaac."],
['Genesis','Gen','21','004','4',"And Abraham circumcised his son Isaac being eight days old, as God had commanded him."],
['Genesis','Gen','21','005','5',"And Abraham was an hundred years old, when his son Isaac was born unto him."],
['Genesis','Gen','21','006','6',"And Sarah said, God hath made me to laugh, \emph{so} \emph{that} all that hear will laugh with me."],
['Genesis','Gen','21','007','7',"And she said, Who would have said unto Abraham, that Sarah should have given children suck? for I have born \emph{him} a son in his old age."],
['Genesis','Gen','21','008','8',"And the child grew, and was weaned: and Abraham made a great feast the \emph{same} day that Isaac was weaned."],
['Genesis','Gen','21','009','9',"And Sarah saw the son of Hagar the Egyptian, which she had born unto Abraham, mocking."],
['Genesis','Gen','21','010','10',"Wherefore she said unto Abraham, Cast out this bondwoman and her son: for the son of this bondwoman shall not be heir with my son, \emph{even} with Isaac."],
['Genesis','Gen','21','011','11',"And the thing was very grievous in Abraham's sight because of his son."],
['Genesis','Gen','21','012','12',"And God said unto Abraham, Let it not be grievous in thy sight because of the lad, and because of thy bondwoman; in all that Sarah hath said unto thee, hearken unto her voice; for in Isaac shall thy seed be called."],
['Genesis','Gen','21','013','13',"And also of the son of the bondwoman will I make a nation, because he \emph{is} thy seed."],
['Genesis','Gen','21','014','14',"And Abraham rose up early in the morning, and took bread, and a bottle of water, and gave \emph{it} unto Hagar, putting \emph{it} on her shoulder, and the child, and sent her away: and she departed, and wandered in the wilderness of Beer-sheba."],
['Genesis','Gen','21','015','15',"And the water was spent in the bottle, and she cast the child under one of the shrubs."],
['Genesis','Gen','21','016','16',"And she went, and sat her down over against \emph{him} a good way off, as it were a bowshot: for she said, Let me not see the death of the child. And she sat over against \emph{him}, and lift up her voice, and wept."],
['Genesis','Gen','21','017','17',"And God heard the voice of the lad; and the angel of God called to Hagar out of heaven, and said unto her, What aileth thee, Hagar? fear not; for God hath heard the voice of the lad where he \emph{is}."],
['Genesis','Gen','21','018','18',"Arise, lift up the lad, and hold him in thine hand; for I will make him a great nation."],
['Genesis','Gen','21','019','19',"And God opened her eyes, and she saw a well of water; and she went, and filled the bottle with water, and gave the lad drink."],
['Genesis','Gen','21','020','20',"And God was with the lad; and he grew, and dwelt in the wilderness, and became an archer."],
['Genesis','Gen','21','021','21',"And he dwelt in the wilderness of Paran: and his mother took him a wife out of the land of Egypt."],
['Genesis','Gen','21','022','22',"And it came to pass at that time, that Abimelech and Phichol the chief captain of his host spake unto Abraham, saying, God \emph{is} with thee in all that thou doest:"],
['Genesis','Gen','21','023','23',"Now therefore swear unto me here by God that thou wilt not deal falsely with me, nor with my son, nor with my son's son: \emph{but} according to the kindness that I have done unto thee, thou shalt do unto me, and to the land wherein thou hast sojourned."],
['Genesis','Gen','21','024','24',"And Abraham said, I will swear."],
['Genesis','Gen','21','025','25',"And Abraham reproved Abimelech because of a well of water, which Abimelech's servants had violently taken away."],
['Genesis','Gen','21','026','26',"And Abimelech said, I wot not who hath done this thing: neither didst thou tell me, neither yet heard I \emph{of} \emph{it}, but to day."],
['Genesis','Gen','21','027','27',"And Abraham took sheep and oxen, and gave them unto Abimelech; and both of them made a covenant."],
['Genesis','Gen','21','028','28',"And Abraham set seven ewe lambs of the flock by themselves."],
['Genesis','Gen','21','029','29',"And Abimelech said unto Abraham, What \emph{mean} these seven ewe lambs which thou hast set by themselves?"],
['Genesis','Gen','21','030','30',"And he said, For \emph{these} seven ewe lambs shalt thou take of my hand, that they may be a witness unto me, that I have digged this well."],
['Genesis','Gen','21','031','31',"Wherefore he called that place Beer-sheba; because there they sware both of them."],
['Genesis','Gen','21','032','32',"Thus they made a covenant at Beer-sheba: then Abimelech rose up, and Phichol the chief captain of his host, and they returned into the land of the Philistines."],
['Genesis','Gen','21','033','33',"And \emph{Abraham} planted a grove in Beer-sheba, and called there on the name of the LORD, the everlasting God."],
['Genesis','Gen','21','034','34',"And Abraham sojourned in the Philistines' land many days."]
]

Genesis22 = [
    ['Genesis','Gen','22','001','1',"And it came to pass after these things, that God did tempt Abraham, and said unto him, Abraham: and he said, Behold, \emph{here} I \emph{am}."],
['Genesis','Gen','22','002','2',"And he said, Take now thy son, thine only \emph{son} Isaac, whom thou lovest, and get thee into the land of Moriah; and offer him there for a burnt offering upon one of the mountains which I will tell thee of."],
['Genesis','Gen','22','003','3',"And Abraham rose up early in the morning, and saddled his ass, and took two of his young men with him, and Isaac his son, and clave the wood for the burnt offering, and rose up, and went unto the place of which God had told him."],
['Genesis','Gen','22','004','4',"Then on the third day Abraham lifted up his eyes, and saw the place afar off."],
['Genesis','Gen','22','005','5',"And Abraham said unto his young men, Abide ye here with the ass; and I and the lad will go yonder and worship, and come again to you."],
['Genesis','Gen','22','006','6',"And Abraham took the wood of the burnt offering, and laid \emph{it} upon Isaac his son; and he took the fire in his hand, and a knife; and they went both of them together."],
['Genesis','Gen','22','007','7',"And Isaac spake unto Abraham his father, and said, My father: and he said, Here \emph{am} I, my son. And he said, Behold the fire and the wood: but where \emph{is} the lamb for a burnt offering?"],
['Genesis','Gen','22','008','8',"And Abraham said, My son, God will provide himself a lamb for a burnt offering: so they went both of them together."],
['Genesis','Gen','22','009','9',"And they came to the place which God had told him of; and Abraham built an altar there, and laid the wood in order, and bound Isaac his son, and laid him on the altar upon the wood."],
['Genesis','Gen','22','010','10',"And Abraham stretched forth his hand, and took the knife to slay his son."],
['Genesis','Gen','22','011','11',"And the angel of the LORD called unto him out of heaven, and said, Abraham, Abraham: and he said, Here \emph{am} I."],
['Genesis','Gen','22','012','12',"And he said, Lay not thine hand upon the lad, neither do thou any thing unto him: for now I know that thou fearest God, seeing thou hast not withheld thy son, thine only \emph{son} from me."],
['Genesis','Gen','22','013','13',"And Abraham lifted up his eyes, and looked, and behold behind \emph{him} a ram caught in a thicket by his horns: and Abraham went and took the ram, and offered him up for a burnt offering in the stead of his son."],
['Genesis','Gen','22','014','14',"And Abraham called the name of that place Jehovah-jireh: as it is said \emph{to} this day, In the mount of the LORD it shall be seen."],
['Genesis','Gen','22','015','15',"And the angel of the LORD called unto Abraham out of heaven the second time,"],
['Genesis','Gen','22','016','16',"And said, By myself have I sworn, saith the LORD, for because thou hast done this thing, and hast not withheld thy son, thine only \emph{son}:"],
['Genesis','Gen','22','017','17',"That in blessing I will bless thee, and in multiplying I will multiply thy seed as the stars of the heaven, and as the sand which \emph{is} upon the sea shore; and thy seed shall possess the gate of his enemies;"],
['Genesis','Gen','22','018','18',"And in thy seed shall all the nations of the earth be blessed; because thou hast obeyed my voice."],
['Genesis','Gen','22','019','19',"So Abraham returned unto his young men, and they rose up and went together to Beer-sheba; and Abraham dwelt at Beer-sheba."],
['Genesis','Gen','22','020','20',"And it came to pass after these things, that it was told Abraham, saying, Behold, Milcah, she hath also born children unto thy brother Nahor;"],
['Genesis','Gen','22','021','21',"Huz his firstborn, and Buz his brother, and Kemuel the father of Aram,"],
['Genesis','Gen','22','022','22',"And Chesed, and Hazo, and Pildash, and Jidlaph, and Bethuel."],
['Genesis','Gen','22','023','23',"And Bethuel begat Rebekah: these eight Milcah did bear to Nahor, Abraham's brother."],
['Genesis','Gen','22','024','24',"And his concubine, whose name \emph{was} Reumah, she bare also Tebah, and Gaham, and Thahash, and Maachah."]
]

Genesis23 = [
    ['Genesis','Gen','23','001','1',"And Sarah was an hundred and seven and twenty years old: \emph{these} \emph{were} the years of the life of Sarah."],
['Genesis','Gen','23','002','2',"And Sarah died in Kirjath-arba; the same \emph{is} Hebron in the land of Canaan: and Abraham came to mourn for Sarah, and to weep for her."],
['Genesis','Gen','23','003','3',"And Abraham stood up from before his dead, and spake unto the sons of Heth, saying,"],
['Genesis','Gen','23','004','4',"I \emph{am} a stranger and a sojourner with you: give me a possession of a buryingplace with you, that I may bury my dead out of my sight."],
['Genesis','Gen','23','005','5',"And the children of Heth answered Abraham, saying unto him,"],
['Genesis','Gen','23','006','6',"Hear us, my lord: thou \emph{art} a mighty prince among us: in the choice of our sepulchres bury thy dead; none of us shall withhold from thee his sepulchre, but that thou mayest bury thy dead."],
['Genesis','Gen','23','007','7',"And Abraham stood up, and bowed himself to the people of the land, \emph{even} to the children of Heth."],
['Genesis','Gen','23','008','8',"And he communed with them, saying, If it be your mind that I should bury my dead out of my sight; hear me, and intreat for me to Ephron the son of Zohar,"],
['Genesis','Gen','23','009','9',"That he may give me the cave of Machpelah, which he hath, which \emph{is} in the end of his field; for as much money as it is worth he shall give it me for a possession of a buryingplace amongst you."],
['Genesis','Gen','23','010','10',"And Ephron dwelt among the children of Heth: and Ephron the Hittite answered Abraham in the audience of the children of Heth, \emph{even} of all that went in at the gate of his city, saying,"],
['Genesis','Gen','23','011','11',"Nay, my lord, hear me: the field give I thee, and the cave that \emph{is} therein, I give it thee; in the presence of the sons of my people give I it thee: bury thy dead."],
['Genesis','Gen','23','012','12',"And Abraham bowed down himself before the people of the land."],
['Genesis','Gen','23','013','13',"And he spake unto Ephron in the audience of the people of the land, saying, But if thou \emph{wilt} \emph{give} \emph{it}, I pray thee, hear me: I will give thee money for the field; take \emph{it} of me, and I will bury my dead there."],
['Genesis','Gen','23','014','14',"And Ephron answered Abraham, saying unto him,"],
['Genesis','Gen','23','015','15',"My lord, hearken unto me: the land \emph{is} \emph{worth} four hundred shekels of silver; what \emph{is} that betwixt me and thee? bury therefore thy dead."],
['Genesis','Gen','23','016','16',"And Abraham hearkened unto Ephron; and Abraham weighed to Ephron the silver, which he had named in the audience of the sons of Heth, four hundred shekels of silver, current \emph{money} with the merchant."],
['Genesis','Gen','23','017','17',"And the field of Ephron, which \emph{was} in Machpelah, which \emph{was} before Mamre, the field, and the cave which \emph{was} therein, and all the trees that \emph{were} in the field, that \emph{were} in all the borders round about, were made sure"],
['Genesis','Gen','23','018','18',"Unto Abraham for a possession in the presence of the children of Heth, before all that went in at the gate of his city."],
['Genesis','Gen','23','019','19',"And after this, Abraham buried Sarah his wife in the cave of the field of Machpelah before Mamre: the same \emph{is} Hebron in the land of Canaan."],
['Genesis','Gen','23','020','20',"And the field, and the cave that \emph{is} therein, were made sure unto Abraham for a possession of a buryingplace by the sons of Heth."]
]

Genesis24 = [
    ['Genesis','Gen','24','001','1',"And Abraham was old, \emph{and} well stricken in age: and the LORD had blessed Abraham in all things."],
['Genesis','Gen','24','002','2',"And Abraham said unto his eldest servant of his house, that ruled over all that he had, Put, I pray thee, thy hand under my thigh:"],
['Genesis','Gen','24','003','3',"And I will make thee swear by the LORD, the God of heaven, and the God of the earth, that thou shalt not take a wife unto my son of the daughters of the Canaanites, among whom I dwell:"],
['Genesis','Gen','24','004','4',"But thou shalt go unto my country, and to my kindred, and take a wife unto my son Isaac."],
['Genesis','Gen','24','005','5',"And the servant said unto him, Peradventure the woman will not be willing to follow me unto this land: must I needs bring thy son again unto the land from whence thou camest?"],
['Genesis','Gen','24','006','6',"And Abraham said unto him, Beware thou that thou bring not my son thither again."],
['Genesis','Gen','24','007','7',"The LORD God of heaven, which took me from my father's house, and from the land of my kindred, and which spake unto me, and that sware unto me, saying, Unto thy seed will I give this land; he shall send his angel before thee, and thou shalt take a wife unto my son from thence."],
['Genesis','Gen','24','008','8',"And if the woman will not be willing to follow thee, then thou shalt be clear from this my oath: only bring not my son thither again."],
['Genesis','Gen','24','009','9',"And the servant put his hand under the thigh of Abraham his master, and sware to him concerning that matter."],
['Genesis','Gen','24','010','10',"And the servant took ten camels of the camels of his master, and departed; for all the goods of his master \emph{were} in his hand: and he arose, and went to Mesopotamia, unto the city of Nahor."],
['Genesis','Gen','24','011','11',"And he made his camels to kneel down without the city by a well of water at the time of the evening, \emph{even} the time that women go out to draw \emph{water}."],
['Genesis','Gen','24','012','12',"And he said, O LORD God of my master Abraham, I pray thee, send me good speed this day, and shew kindness unto my master Abraham."],
['Genesis','Gen','24','013','13',"Behold, I stand \emph{here} by the well of water; and the daughters of the men of the city come out to draw water:"],
['Genesis','Gen','24','014','14',"And let it come to pass, that the damsel to whom I shall say, Let down thy pitcher, I pray thee, that I may drink; and she shall say, Drink, and I will give thy camels drink also: \emph{let} \emph{the} \emph{same} \emph{be} she \emph{that} thou hast appointed for thy servant Isaac; and thereby shall I know that thou hast shewed kindness unto my master."],
['Genesis','Gen','24','015','15',"And it came to pass, before he had done speaking, that, behold, Rebekah came out, who was born to Bethuel, son of Milcah, the wife of Nahor, Abraham's brother, with her pitcher upon her shoulder."],
['Genesis','Gen','24','016','16',"And the damsel \emph{was} very fair to look upon, a virgin, neither had any man known her: and she went down to the well, and filled her pitcher, and came up."],
['Genesis','Gen','24','017','17',"And the servant ran to meet her, and said, Let me, I pray thee, drink a little water of thy pitcher."],
['Genesis','Gen','24','018','18',"And she said, Drink, my lord: and she hasted, and let down her pitcher upon her hand, and gave him drink."],
['Genesis','Gen','24','019','19',"And when she had done giving him drink, she said, I will draw \emph{water} for thy camels also, until they have done drinking."],
['Genesis','Gen','24','020','20',"And she hasted, and emptied her pitcher into the trough, and ran again unto the well to draw \emph{water}, and drew for all his camels."],
['Genesis','Gen','24','021','21',"And the man wondering at her held his peace, to wit whether the LORD had made his journey prosperous or not."],
['Genesis','Gen','24','022','22',"And it came to pass, as the camels had done drinking, that the man took a golden earring of half a shekel weight, and two bracelets for her hands of ten \emph{shekels} weight of gold;"],
['Genesis','Gen','24','023','23',"And said, Whose daughter \emph{art} thou? tell me, I pray thee: is there room \emph{in} thy father's house for us to lodge in?"],
['Genesis','Gen','24','024','24',"And she said unto him, I \emph{am} the daughter of Bethuel the son of Milcah, which she bare unto Nahor."],
['Genesis','Gen','24','025','25',"She said moreover unto him, We have both straw and provender enough, and room to lodge in."],
['Genesis','Gen','24','026','26',"And the man bowed down his head, and worshipped the LORD."],
['Genesis','Gen','24','027','27',"And he said, Blessed \emph{be} the LORD God of my master Abraham, who hath not left destitute my master of his mercy and his truth: I \emph{being} in the way, the LORD led me to the house of my master's brethren."],
['Genesis','Gen','24','028','28',"And the damsel ran, and told \emph{them} \emph{of} her mother's house these things."],
['Genesis','Gen','24','029','29',"And Rebekah had a brother, and his name \emph{was} Laban: and Laban ran out unto the man, unto the well."],
['Genesis','Gen','24','030','30',"And it came to pass, when he saw the earring and bracelets upon his sister's hands, and when he heard the words of Rebekah his sister, saying, Thus spake the man unto me; that he came unto the man; and, behold, he stood by the camels at the well."],
['Genesis','Gen','24','031','31',"And he said, Come in, thou blessed of the LORD; wherefore standest thou without? for I have prepared the house, and room for the camels."],
['Genesis','Gen','24','032','32',"And the man came into the house: and he ungirded his camels, and gave straw and provender for the camels, and water to wash his feet, and the men's feet that \emph{were} with him."],
['Genesis','Gen','24','033','33',"And there was set \emph{meat} before him to eat: but he said, I will not eat, until I have told mine errand. And he said, Speak on."],
['Genesis','Gen','24','034','34',"And he said, I \emph{am} Abraham's servant."],
['Genesis','Gen','24','035','35',"And the LORD hath blessed my master greatly; and he is become great: and he hath given him flocks, and herds, and silver, and gold, and menservants, and maidservants, and camels, and asses."],
['Genesis','Gen','24','036','36',"And Sarah my master's wife bare a son to my master when she was old: and unto him hath he given all that he hath."],
['Genesis','Gen','24','037','37',"And my master made me swear, saying, Thou shalt not take a wife to my son of the daughters of the Canaanites, in whose land I dwell:"],
['Genesis','Gen','24','038','38',"But thou shalt go unto my father's house, and to my kindred, and take a wife unto my son."],
['Genesis','Gen','24','039','39',"And I said unto my master, Peradventure the woman will not follow me."],
['Genesis','Gen','24','040','40',"And he said unto me, The LORD, before whom I walk, will send his angel with thee, and prosper thy way; and thou shalt take a wife for my son of my kindred, and of my father's house:"],
['Genesis','Gen','24','041','41',"Then shalt thou be clear from \emph{this} my oath, when thou comest to my kindred; and if they give not thee \emph{one}, thou shalt be clear from my oath."],
['Genesis','Gen','24','042','42',"And I came this day unto the well, and said, O LORD God of my master Abraham, if now thou do prosper my way which I go:"],
['Genesis','Gen','24','043','43',"Behold, I stand by the well of water; and it shall come to pass, that when the virgin cometh forth to draw \emph{water}, and I say to her, Give me, I pray thee, a little water of thy pitcher to drink;"],
['Genesis','Gen','24','044','44',"And she say to me, Both drink thou, and I will also draw for thy camels: \emph{let} the same \emph{be} the woman whom the LORD hath appointed out for my master's son."],
['Genesis','Gen','24','045','45',"And before I had done speaking in mine heart, behold, Rebekah came forth with her pitcher on her shoulder; and she went down unto the well, and drew \emph{water}: and I said unto her, Let me drink, I pray thee."],
['Genesis','Gen','24','046','46',"And she made haste, and let down her pitcher from her \emph{shoulder}, and said, Drink, and I will give thy camels drink also: so I drank, and she made the camels drink also."],
['Genesis','Gen','24','047','47',"And I asked her, and said, Whose daughter \emph{art} thou? And she said, The daughter of Bethuel, Nahor's son, whom Milcah bare unto him: and I put the earring upon her face, and the bracelets upon her hands."],
['Genesis','Gen','24','048','48',"And I bowed down my head, and worshipped the LORD, and blessed the LORD God of my master Abraham, which had led me in the right way to take my master's brother's daughter unto his son."],
['Genesis','Gen','24','049','49',"And now if ye will deal kindly and truly with my master, tell me: and if not, tell me; that I may turn to the right hand, or to the left."],
['Genesis','Gen','24','050','50',"Then Laban and Bethuel answered and said, The thing proceedeth from the LORD: we cannot speak unto thee bad or good."],
['Genesis','Gen','24','051','51',"Behold, Rebekah \emph{is} before thee, take \emph{her}, and go, and let her be thy master's son's wife, as the LORD hath spoken."],
['Genesis','Gen','24','052','52',"And it came to pass, that, when Abraham's servant heard their words, he worshipped the LORD, \emph{bowing} \emph{himself} to the earth."],
['Genesis','Gen','24','053','53',"And the servant brought forth jewels of silver, and jewels of gold, and raiment, and gave \emph{them} to Rebekah: he gave also to her brother and to her mother precious things."],
['Genesis','Gen','24','054','54',"And they did eat and drink, he and the men that \emph{were} with him, and tarried all night; and they rose up in the morning, and he said, Send me away unto my master."],
['Genesis','Gen','24','055','55',"And her brother and her mother said, Let the damsel abide with us \emph{a} \emph{few} days, at the least ten; after that she shall go."],
['Genesis','Gen','24','056','56',"And he said unto them, Hinder me not, seeing the LORD hath prospered my way; send me away that I may go to my master."],
['Genesis','Gen','24','057','57',"And they said, We will call the damsel, and enquire at her mouth."],
['Genesis','Gen','24','058','58',"And they called Rebekah, and said unto her, Wilt thou go with this man? And she said, I will go."],
['Genesis','Gen','24','059','59',"And they sent away Rebekah their sister, and her nurse, and Abraham's servant, and his men."],
['Genesis','Gen','24','060','60',"And they blessed Rebekah, and said unto her, Thou \emph{art} our sister, be thou \emph{the} \emph{mother} of thousands of millions, and let thy seed possess the gate of those which hate them."],
['Genesis','Gen','24','061','61',"And Rebekah arose, and her damsels, and they rode upon the camels, and followed the man: and the servant took Rebekah, and went his way."],
['Genesis','Gen','24','062','62',"And Isaac came from the way of the well Lahai-roi; for he dwelt in the south country."],
['Genesis','Gen','24','063','63',"And Isaac went out to meditate in the field at the eventide: and he lifted up his eyes, and saw, and, behold, the camels \emph{were} coming."],
['Genesis','Gen','24','064','64',"And Rebekah lifted up her eyes, and when she saw Isaac, she lighted off the camel."],
['Genesis','Gen','24','065','65',"For she \emph{had} said unto the servant, What man \emph{is} this that walketh in the field to meet us? And the servant \emph{had} said, It \emph{is} my master: therefore she took a vail, and covered herself."],
['Genesis','Gen','24','066','66',"And the servant told Isaac all things that he had done."],
['Genesis','Gen','24','067','67',"And Isaac brought her into his mother Sarah's tent, and took Rebekah, and she became his wife; and he loved her: and Isaac was comforted after his mother's \emph{death}."]
]

Genesis25 = [
    ['Genesis','Gen','25','001','1',"Then again Abraham took a wife, and her name \emph{was} Keturah."],
['Genesis','Gen','25','002','2',"And she bare him Zimran, and Jokshan, and Medan, and Midian, and Ishbak, and Shuah."],
['Genesis','Gen','25','003','3',"And Jokshan begat Sheba, and Dedan. And the sons of Dedan were Asshurim, and Letushim, and Leummim."],
['Genesis','Gen','25','004','4',"And the sons of Midian; Ephah, and Epher, and Hanoch, and Abida, and Eldaah. All these \emph{were} the children of Keturah."],
['Genesis','Gen','25','005','5',"And Abraham gave all that he had unto Isaac."],
['Genesis','Gen','25','006','6',"But unto the sons of the concubines, which Abraham had, Abraham gave gifts, and sent them away from Isaac his son, while he yet lived, eastward, unto the east country."],
['Genesis','Gen','25','007','7',"And these \emph{are} the days of the years of Abraham's life which he lived, an hundred threescore and fifteen years."],
['Genesis','Gen','25','008','8',"Then Abraham gave up the ghost, and died in a good old age, an old man, and full \emph{of} \emph{years}; and was gathered to his people."],
['Genesis','Gen','25','009','9',"And his sons Isaac and Ishmael buried him in the cave of Machpelah, in the field of Ephron the son of Zohar the Hittite, which \emph{is} before Mamre;"],
['Genesis','Gen','25','010','10',"The field which Abraham purchased of the sons of Heth: there was Abraham buried, and Sarah his wife."],
['Genesis','Gen','25','011','11',"And it came to pass after the death of Abraham, that God blessed his son Isaac; and Isaac dwelt by the well Lahai-roi."],
['Genesis','Gen','25','012','12',"Now these \emph{are} the generations of Ishmael, Abraham's son, whom Hagar the Egyptian, Sarah's handmaid, bare unto Abraham:"],
['Genesis','Gen','25','013','13',"And these \emph{are} the names of the sons of Ishmael, by their names, according to their generations: the firstborn of Ishmael, Nebajoth; and Kedar, and Adbeel, and Mibsam,"],
['Genesis','Gen','25','014','14',"And Mishma, and Dumah, and Massa,"],
['Genesis','Gen','25','015','15',"Hadar, and Tema, Jetur, Naphish, and Kedemah:"],
['Genesis','Gen','25','016','16',"These \emph{are} the sons of Ishmael, and these \emph{are} their names, by their towns, and by their castles; twelve princes according to their nations."],
['Genesis','Gen','25','017','17',"And these \emph{are} the years of the life of Ishmael, an hundred and thirty and seven years: and he gave up the ghost and died; and was gathered unto his people."],
['Genesis','Gen','25','018','18',"And they dwelt from Havilah unto Shur, that \emph{is} before Egypt, as thou goest toward Assyria: \emph{and} he died in the presence of all his brethren."],
['Genesis','Gen','25','019','19',"And these \emph{are} the generations of Isaac, Abraham's son: Abraham begat Isaac:"],
['Genesis','Gen','25','020','20',"And Isaac was forty years old when he took Rebekah to wife, the daughter of Bethuel the Syrian of Padan-aram, the sister to Laban the Syrian."],
['Genesis','Gen','25','021','21',"And Isaac intreated the LORD for his wife, because she \emph{was} barren: and the LORD was intreated of him, and Rebekah his wife conceived."],
['Genesis','Gen','25','022','22',"And the children struggled together within her; and she said, If \emph{it} \emph{be} so, why \emph{am} I thus? And she went to enquire of the LORD."],
['Genesis','Gen','25','023','23',"And the LORD said unto her, Two nations \emph{are} in thy womb, and two manner of people shall be separated from thy bowels; and \emph{the} \emph{one} people shall be stronger than \emph{the} \emph{other} people; and the elder shall serve the younger."],
['Genesis','Gen','25','024','24',"And when her days to be delivered were fulfilled, behold, \emph{there} \emph{were} twins in her womb."],
['Genesis','Gen','25','025','25',"And the first came out red, all over like an hairy garment; and they called his name Esau."],
['Genesis','Gen','25','026','26',"And after that came his brother out, and his hand took hold on Esau's heel; and his name was called Jacob: and Isaac \emph{was} threescore years old when she bare them."],
['Genesis','Gen','25','027','27',"And the boys grew: and Esau was a cunning hunter, a man of the field; and Jacob \emph{was} a plain man, dwelling in tents."],
['Genesis','Gen','25','028','28',"And Isaac loved Esau, because he did eat of \emph{his} venison: but Rebekah loved Jacob."],
['Genesis','Gen','25','029','29',"And Jacob sod pottage: and Esau came from the field, and he \emph{was} faint:"],
['Genesis','Gen','25','030','30',"And Esau said to Jacob, Feed me, I pray thee, with that same red \emph{pottage}; for I \emph{am} faint: therefore was his name called Edom."],
['Genesis','Gen','25','031','31',"And Jacob said, Sell me this day thy birthright."],
['Genesis','Gen','25','032','32',"And Esau said, Behold, I \emph{am} at the point to die: and what profit shall this birthright do to me?"],
['Genesis','Gen','25','033','33',"And Jacob said, Swear to me this day; and he sware unto him: and he sold his birthright unto Jacob."],
['Genesis','Gen','25','034','34',"Then Jacob gave Esau bread and pottage of lentiles; and he did eat and drink, and rose up, and went his way: thus Esau despised \emph{his} birthright."]
]

Genesis26 = [
    ['Genesis','Gen','26','001','1',"And there was a famine in the land, beside the first famine that was in the days of Abraham. And Isaac went unto Abimelech king of the Philistines unto Gerar."],
['Genesis','Gen','26','002','2',"And the LORD appeared unto him, and said, Go not down into Egypt; dwell in the land which I shall tell thee of:"],
['Genesis','Gen','26','003','3',"Sojourn in this land, and I will be with thee, and will bless thee; for unto thee, and unto thy seed, I will give all these countries, and I will perform the oath which I sware unto Abraham thy father;"],
['Genesis','Gen','26','004','4',"And I will make thy seed to multiply as the stars of heaven, and will give unto thy seed all these countries; and in thy seed shall all the nations of the earth be blessed;"],
['Genesis','Gen','26','005','5',"Because that Abraham obeyed my voice, and kept my charge, my commandments, my statutes, and my laws."],
['Genesis','Gen','26','006','6',"And Isaac dwelt in Gerar:"],
['Genesis','Gen','26','007','7',"And the men of the place asked \emph{him} of his wife; and he said, She \emph{is} my sister: for he feared to say, \emph{She} \emph{is} my wife; lest, \emph{said} \emph{he}, the men of the place should kill me for Rebekah; because she \emph{was} fair to look upon."],
['Genesis','Gen','26','008','8',"And it came to pass, when he had been there a long time, that Abimelech king of the Philistines looked out at a window, and saw, and, behold, Isaac \emph{was} sporting with Rebekah his wife."],
['Genesis','Gen','26','009','9',"And Abimelech called Isaac, and said, Behold, of a surety she \emph{is} thy wife: and how saidst thou, She \emph{is} my sister? And Isaac said unto him, Because I said, Lest I die for her."],
['Genesis','Gen','26','010','10',"And Abimelech said, What \emph{is} this thou hast done unto us? one of the people might lightly have lien with thy wife, and thou shouldest have brought guiltiness upon us."],
['Genesis','Gen','26','011','11',"And Abimelech charged all \emph{his} people, saying, He that toucheth this man or his wife shall surely be put to death."],
['Genesis','Gen','26','012','12',"Then Isaac sowed in that land, and received in the same year an hundredfold: and the LORD blessed him."],
['Genesis','Gen','26','013','13',"And the man waxed great, and went forward, and grew until he became very great:"],
['Genesis','Gen','26','014','14',"For he had possession of flocks, and possession of herds, and great store of servants: and the Philistines envied him."],
['Genesis','Gen','26','015','15',"For all the wells which his father's servants had digged in the days of Abraham his father, the Philistines had stopped them, and filled them with earth."],
['Genesis','Gen','26','016','16',"And Abimelech said unto Isaac, Go from us; for thou art much mightier than we."],
['Genesis','Gen','26','017','17',"And Isaac departed thence, and pitched his tent in the valley of Gerar, and dwelt there."],
['Genesis','Gen','26','018','18',"And Isaac digged again the wells of water, which they had digged in the days of Abraham his father; for the Philistines had stopped them after the death of Abraham: and he called their names after the names by which his father had called them."],
['Genesis','Gen','26','019','19',"And Isaac's servants digged in the valley, and found there a well of springing water."],
['Genesis','Gen','26','020','20',"And the herdmen of Gerar did strive with Isaac's herdmen, saying, The water \emph{is} our's: and he called the name of the well Esek; because they strove with him."],
['Genesis','Gen','26','021','21',"And they digged another well, and strove for that also: and he called the name of it Sitnah."],
['Genesis','Gen','26','022','22',"And he removed from thence, and digged another well; and for that they strove not: and he called the name of it Rehoboth; and he said, For now the LORD hath made room for us, and we shall be fruitful in the land."],
['Genesis','Gen','26','023','23',"And he went up from thence to Beer-sheba."],
['Genesis','Gen','26','024','24',"And the LORD appeared unto him the same night, and said, I \emph{am} the God of Abraham thy father: fear not, for I \emph{am} with thee, and will bless thee, and multiply thy seed for my servant Abraham's sake."],
['Genesis','Gen','26','025','25',"And he builded an altar there, and called upon the name of the LORD, and pitched his tent there: and there Isaac's servants digged a well."],
['Genesis','Gen','26','026','26',"Then Abimelech went to him from Gerar, and Ahuzzath one of his friends, and Phichol the chief captain of his army."],
['Genesis','Gen','26','027','27',"And Isaac said unto them, Wherefore come ye to me, seeing ye hate me, and have sent me away from you?"],
['Genesis','Gen','26','028','28',"And they said, We saw certainly that the LORD was with thee: and we said, Let there be now an oath betwixt us, \emph{even} betwixt us and thee, and let us make a covenant with thee;"],
['Genesis','Gen','26','029','29',"That thou wilt do us no hurt, as we have not touched thee, and as we have done unto thee nothing but good, and have sent thee away in peace: thou \emph{art} now the blessed of the LORD."],
['Genesis','Gen','26','030','30',"And he made them a feast, and they did eat and drink."],
['Genesis','Gen','26','031','31',"And they rose up betimes in the morning, and sware one to another: and Isaac sent them away, and they departed from him in peace."],
['Genesis','Gen','26','032','32',"And it came to pass the same day, that Isaac's servants came, and told him concerning the well which they had digged, and said unto him, We have found water."],
['Genesis','Gen','26','033','33',"And he called it Shebah: therefore the name of the city \emph{is} Beer-sheba unto this day."],
['Genesis','Gen','26','034','34',"And Esau was forty years old when he took to wife Judith the daughter of Beeri the Hittite, and Bashemath the daughter of Elon the Hittite:"],
['Genesis','Gen','26','035','35',"Which were a grief of mind unto Isaac and to Rebekah."]
]

Genesis27 = [
    ['Genesis','Gen','27','001','1',"And it came to pass, that when Isaac was old, and his eyes were dim, so that he could not see, he called Esau his eldest son, and said unto him, My son: and he said unto him, Behold, \emph{here} \emph{am} I."],
['Genesis','Gen','27','002','2',"And he said, Behold now, I am old, I know not the day of my death:"],
['Genesis','Gen','27','003','3',"Now therefore take, I pray thee, thy weapons, thy quiver and thy bow, and go out to the field, and take me \emph{some} venison;"],
['Genesis','Gen','27','004','4',"And make me savoury meat, such as I love, and bring \emph{it} to me, that I may eat; that my soul may bless thee before I die."],
['Genesis','Gen','27','005','5',"And Rebekah heard when Isaac spake to Esau his son. And Esau went to the field to hunt \emph{for} venison, \emph{and} to bring \emph{it}."],
['Genesis','Gen','27','006','6',"And Rebekah spake unto Jacob her son, saying, Behold, I heard thy father speak unto Esau thy brother, saying,"],
['Genesis','Gen','27','007','7',"Bring me venison, and make me savoury meat, that I may eat, and bless thee before the LORD before my death."],
['Genesis','Gen','27','008','8',"Now therefore, my son, obey my voice according to that which I command thee."],
['Genesis','Gen','27','009','9',"Go now to the flock, and fetch me from thence two good kids of the goats; and I will make them savoury meat for thy father, such as he loveth:"],
['Genesis','Gen','27','010','10',"And thou shalt bring \emph{it} to thy father, that he may eat, and that he may bless thee before his death."],
['Genesis','Gen','27','011','11',"And Jacob said to Rebekah his mother, Behold, Esau my brother \emph{is} a hairy man, and I \emph{am} a smooth man:"],
['Genesis','Gen','27','012','12',"My father peradventure will feel me, and I shall seem to him as a deceiver; and I shall bring a curse upon me, and not a blessing."],
['Genesis','Gen','27','013','13',"And his mother said unto him, Upon me \emph{be} thy curse, my son: only obey my voice, and go fetch me \emph{them}."],
['Genesis','Gen','27','014','14',"And he went, and fetched, and brought \emph{them} to his mother: and his mother made savoury meat, such as his father loved."],
['Genesis','Gen','27','015','15',"And Rebekah took goodly raiment of her eldest son Esau, which \emph{were} with her in the house, and put them upon Jacob her younger son:"],
['Genesis','Gen','27','016','16',"And she put the skins of the kids of the goats upon his hands, and upon the smooth of his neck:"],
['Genesis','Gen','27','017','17',"And she gave the savoury meat and the bread, which she had prepared, into the hand of her son Jacob."],
['Genesis','Gen','27','018','18',"And he came unto his father, and said, My father: and he said, Here \emph{am} I; who \emph{art} thou, my son?"],
['Genesis','Gen','27','019','19',"And Jacob said unto his father, I \emph{am} Esau thy firstborn; I have done according as thou badest me: arise, I pray thee, sit and eat of my venison, that thy soul may bless me."],
['Genesis','Gen','27','020','20',"And Isaac said unto his son, How \emph{is} \emph{it} that thou hast found \emph{it} so quickly, my son? And he said, Because the LORD thy God brought \emph{it} to me."],
['Genesis','Gen','27','021','21',"And Isaac said unto Jacob, Come near, I pray thee, that I may feel thee, my son, whether thou \emph{be} my very son Esau or not."],
['Genesis','Gen','27','022','22',"And Jacob went near unto Isaac his father; and he felt him, and said, The voice \emph{is} Jacob's voice, but the hands \emph{are} the hands of Esau."],
['Genesis','Gen','27','023','23',"And he discerned him not, because his hands were hairy, as his brother Esau's hands: so he blessed him."],
['Genesis','Gen','27','024','24',"And he said, \emph{Art} thou my very son Esau? And he said, I \emph{am}."],
['Genesis','Gen','27','025','25',"And he said, Bring \emph{it} near to me, and I will eat of my son's venison, that my soul may bless thee. And he brought \emph{it} near to him, and he did eat: and he brought him wine, and he drank."],
['Genesis','Gen','27','026','26',"And his father Isaac said unto him, Come near now, and kiss me, my son."],
['Genesis','Gen','27','027','27',"And he came near, and kissed him: and he smelled the smell of his raiment, and blessed him, and said, See, the smell of my son \emph{is} as the smell of a field which the LORD hath blessed:"],
['Genesis','Gen','27','028','28',"Therefore God give thee of the dew of heaven, and the fatness of the earth, and plenty of corn and wine:"],
['Genesis','Gen','27','029','29',"Let people serve thee, and nations bow down to thee: be lord over thy brethren, and let thy mother's sons bow down to thee: cursed \emph{be} every one that curseth thee, and blessed \emph{be} he that blesseth thee."],
['Genesis','Gen','27','030','30',"And it came to pass, as soon as Isaac had made an end of blessing Jacob, and Jacob was yet scarce gone out from the presence of Isaac his father, that Esau his brother came in from his hunting."],
['Genesis','Gen','27','031','31',"And he also had made savoury meat, and brought it unto his father, and said unto his father, Let my father arise, and eat of his son's venison, that thy soul may bless me."],
['Genesis','Gen','27','032','32',"And Isaac his father said unto him, Who \emph{art} thou? And he said, I \emph{am} thy son, thy firstborn Esau."],
['Genesis','Gen','27','033','33',"And Isaac trembled very exceedingly, and said, Who? where \emph{is} he that hath taken venison, and brought \emph{it} me, and I have eaten of all before thou camest, and have blessed him? yea, \emph{and} he shall be blessed."],
['Genesis','Gen','27','034','34',"And when Esau heard the words of his father, he cried with a great and exceeding bitter cry, and said unto his father, Bless me, \emph{even} me also, O my father."],
['Genesis','Gen','27','035','35',"And he said, Thy brother came with subtilty, and hath taken away thy blessing."],
['Genesis','Gen','27','036','36',"And he said, Is not he rightly named Jacob? for he hath supplanted me these two times: he took away my birthright; and, behold, now he hath taken away my blessing. And he said, Hast thou not reserved a blessing for me?"],
['Genesis','Gen','27','037','37',"And Isaac answered and said unto Esau, Behold, I have made him thy lord, and all his brethren have I given to him for servants; and with corn and wine have I sustained him: and what shall I do now unto thee, my son?"],
['Genesis','Gen','27','038','38',"And Esau said unto his father, Hast thou but one blessing, my father? bless me, \emph{even} me also, O my father. And Esau lifted up his voice, and wept."],
['Genesis','Gen','27','039','39',"And Isaac his father answered and said unto him, Behold, thy dwelling shall be the fatness of the earth, and of the dew of heaven from above;"],
['Genesis','Gen','27','040','40',"And by thy sword shalt thou live, and shalt serve thy brother; and it shall come to pass when thou shalt have the dominion, that thou shalt break his yoke from off thy neck."],
['Genesis','Gen','27','041','41',"And Esau hated Jacob because of the blessing wherewith his father blessed him: and Esau said in his heart, The days of mourning for my father are at hand; then will I slay my brother Jacob."],
['Genesis','Gen','27','042','42',"And these words of Esau her elder son were told to Rebekah: and she sent and called Jacob her younger son, and said unto him, Behold, thy brother Esau, as touching thee, doth comfort himself, \emph{purposing} to kill thee."],
['Genesis','Gen','27','043','43',"Now therefore, my son, obey my voice; and arise, flee thou to Laban my brother to Haran;"],
['Genesis','Gen','27','044','44',"And tarry with him a few days, until thy brother's fury turn away;"],
['Genesis','Gen','27','045','45',"Until thy brother's anger turn away from thee, and he forget \emph{that} which thou hast done to him: then I will send, and fetch thee from thence: why should I be deprived also of you both in one day?"],
['Genesis','Gen','27','046','46',"And Rebekah said to Isaac, I am weary of my life because of the daughters of Heth: if Jacob take a wife of the daughters of Heth, such as these \emph{which} \emph{are} of the daughters of the land, what good shall my life do me?"]
]

Genesis28 = [
    ['Genesis','Gen','28','001','1',"And Isaac called Jacob, and blessed him, and charged him, and said unto him, Thou shalt not take a wife of the daughters of Canaan."],
    ['Genesis','Gen','28','002','2',"Arise, go to Padan-aram, to the house of Bethuel thy mother's father; and take thee a wife from thence of the daughters of Laban thy mother's brother."],
    ['Genesis','Gen','28','003','3',"And God Almighty bless thee, and make thee fruitful, and multiply thee, that thou mayest be a multitude of people;"],
    ['Genesis','Gen','28','004','4',"And give thee the blessing of Abraham, to thee, and to thy seed with thee; that thou mayest inherit the land wherein thou art a stranger, which God gave unto Abraham."],
    ['Genesis','Gen','28','005','5',"And Isaac sent away Jacob: and he went to Padan-aram unto Laban, son of Bethuel the Syrian, the brother of Rebekah, Jacob's and Esau's mother."],
    ['Genesis','Gen','28','006','6',"When Esau saw that Isaac had blessed Jacob, and sent him away to Padan-aram, to take him a wife from thence; and that as he blessed him he gave him a charge, saying, Thou shalt not take a wife of the daughters of Canaan;"],
    ['Genesis','Gen','28','007','7',"And that Jacob obeyed his father and his mother, and was gone to Padan-aram;"],
    ['Genesis','Gen','28','008','8',"And Esau seeing that the daughters of Canaan pleased not Isaac his father;"],
    ['Genesis','Gen','28','009','9',"Then went Esau unto Ishmael, and took unto the wives which he had Mahalath the daughter of Ishmael Abraham's son, the sister of Nebajoth, to be his wife."],
    ['Genesis','Gen','28','010','10',"And Jacob went out from Beer-sheba, and went toward Haran."],
    ['Genesis','Gen','28','011','11',"And he lighted upon a certain place, and tarried there all night, because the sun was set; and he took of the stones of that place, and put \emph{them} \emph{for} his pillows, and lay down in that place to sleep."],
    ['Genesis','Gen','28','012','12',"And he dreamed, and behold a ladder set up on the earth, and the top of it reached to heaven: and behold the angels of God ascending and descending on it."],
    ['Genesis','Gen','28','013','13',"And, behold, the LORD stood above it, and said, I \emph{am} the LORD God of Abraham thy father, and the God of Isaac: the land whereon thou liest, to thee will I give it, and to thy seed;"],
    ['Genesis','Gen','28','014','14',"And thy seed shall be as the dust of the earth, and thou shalt spread abroad to the west, and to the east, and to the north, and to the south: and in thee and in thy seed shall all the families of the earth be blessed."],
    ['Genesis','Gen','28','015','15',"And, behold, I \emph{am} with thee, and will keep thee in all \emph{places} whither thou goest, and will bring thee again into this land; for I will not leave thee, until I have done \emph{that} which I have spoken to thee of."],
    ['Genesis','Gen','28','016','16',"And Jacob awaked out of his sleep, and he said, Surely the LORD is in this place; and I knew \emph{it} not."],
    ['Genesis','Gen','28','017','17',"And he was afraid, and said, How dreadful \emph{is} this place! this \emph{is} none other but the house of God, and this \emph{is} the gate of heaven."],
    ['Genesis','Gen','28','018','18',"And Jacob rose up early in the morning, and took the stone that he had put \emph{for} his pillows, and set it up \emph{for} a pillar, and poured oil upon the top of it."],
    ['Genesis','Gen','28','019','19',"And he called the name of that place Beth-el: but the name of that city \emph{was} \emph{called} Luz at the first."],
    ['Genesis','Gen','28','020','20',"And Jacob vowed a vow, saying, If God will be with me, and will keep me in this way that I go, and will give me bread to eat, and raiment to put on,"],
    ['Genesis','Gen','28','021','21',"So that I come again to my father's house in peace; then shall the LORD be my God:"],
    ['Genesis','Gen','28','022','22',"And this stone, which I have set \emph{for} a pillar, shall be God's house: and of all that thou shalt give me I will surely give the tenth unto thee."]
]

Genesis29 = [
    ['Genesis','Gen','29','001','1',"Then Jacob went on his journey, and came into the land of the people of the east."],
['Genesis','Gen','29','002','2',"And he looked, and behold a well in the field, and, lo, there \emph{were} three flocks of sheep lying by it; for out of that well they watered the flocks: and a great stone \emph{was} upon the well's mouth."],
['Genesis','Gen','29','003','3',"And thither were all the flocks gathered: and they rolled the stone from the well's mouth, and watered the sheep, and put the stone again upon the well's mouth in his place."],
['Genesis','Gen','29','004','4',"And Jacob said unto them, My brethren, whence \emph{be} ye? And they said, Of Haran \emph{are} we."],
['Genesis','Gen','29','005','5',"And he said unto them, Know ye Laban the son of Nahor? And they said, We know \emph{him}."],
['Genesis','Gen','29','006','6',"And he said unto them, \emph{Is} he well? And they said, \emph{He} \emph{is} well: and, behold, Rachel his daughter cometh with the sheep."],
['Genesis','Gen','29','007','7',"And he said, Lo, \emph{it} \emph{is} yet high day, neither \emph{is} \emph{it} time that the cattle should be gathered together: water ye the sheep, and go \emph{and} feed \emph{them}."],
['Genesis','Gen','29','008','8',"And they said, We cannot, until all the flocks be gathered together, and \emph{till} they roll the stone from the well's mouth; then we water the sheep."],
['Genesis','Gen','29','009','9',"And while he yet spake with them, Rachel came with her father's sheep: for she kept them."],
['Genesis','Gen','29','010','10',"And it came to pass, when Jacob saw Rachel the daughter of Laban his mother's brother, and the sheep of Laban his mother's brother, that Jacob went near, and rolled the stone from the well's mouth, and watered the flock of Laban his mother's brother."],
['Genesis','Gen','29','011','11',"And Jacob kissed Rachel, and lifted up his voice, and wept."],
['Genesis','Gen','29','012','12',"And Jacob told Rachel that he \emph{was} her father's brother, and that he \emph{was} Rebekah's son: and she ran and told her father."],
['Genesis','Gen','29','013','13',"And it came to pass, when Laban heard the tidings of Jacob his sister's son, that he ran to meet him, and embraced him, and kissed him, and brought him to his house. And he told Laban all these things."],
['Genesis','Gen','29','014','14',"And Laban said to him, Surely thou \emph{art} my bone and my flesh. And he abode with him the space of a month."],
['Genesis','Gen','29','015','15',"And Laban said unto Jacob, Because thou \emph{art} my brother, shouldest thou therefore serve me for nought? tell me, what \emph{shall} thy wages \emph{be}?"],
['Genesis','Gen','29','016','16',"And Laban had two daughters: the name of the elder \emph{was} Leah, and the name of the younger \emph{was} Rachel."],
['Genesis','Gen','29','017','17',"Leah \emph{was} tender eyed; but Rachel was beautiful and well favoured."],
['Genesis','Gen','29','018','18',"And Jacob loved Rachel; and said, I will serve thee seven years for Rachel thy younger daughter."],
['Genesis','Gen','29','019','19',"And Laban said, \emph{It} \emph{is} better that I give her to thee, than that I should give her to another man: abide with me."],
['Genesis','Gen','29','020','20',"And Jacob served seven years for Rachel; and they seemed unto him \emph{but} a few days, for the love he had to her."],
['Genesis','Gen','29','021','21',"And Jacob said unto Laban, Give \emph{me} my wife, for my days are fulfilled, that I may go in unto her."],
['Genesis','Gen','29','022','22',"And Laban gathered together all the men of the place, and made a feast."],
['Genesis','Gen','29','023','23',"And it came to pass in the evening, that he took Leah his daughter, and brought her to him; and he went in unto her."],
['Genesis','Gen','29','024','24',"And Laban gave unto his daughter Leah Zilpah his maid \emph{for} an handmaid."],
['Genesis','Gen','29','025','25',"And it came to pass, that in the morning, behold, it \emph{was} Leah: and he said to Laban, What \emph{is} this thou hast done unto me? did not I serve with thee for Rachel? wherefore then hast thou beguiled me?"],
['Genesis','Gen','29','026','26',"And Laban said, It must not be so done in our country, to give the younger before the firstborn."],
['Genesis','Gen','29','027','27',"Fulfil her week, and we will give thee this also for the service which thou shalt serve with me yet seven other years."],
['Genesis','Gen','29','028','28',"And Jacob did so, and fulfilled her week: and he gave him Rachel his daughter to wife also."],
['Genesis','Gen','29','029','29',"And Laban gave to Rachel his daughter Bilhah his handmaid to be her maid."],
['Genesis','Gen','29','030','30',"And he went in also unto Rachel, and he loved also Rachel more than Leah, and served with him yet seven other years."],
['Genesis','Gen','29','031','31',"And when the LORD saw that Leah \emph{was} hated, he opened her womb: but Rachel \emph{was} barren."],
['Genesis','Gen','29','032','32',"And Leah conceived, and bare a son, and she called his name Reuben: for she said, Surely the LORD hath looked upon my affliction; now therefore my husband will love me."],
['Genesis','Gen','29','033','33',"And she conceived again, and bare a son; and said, Because the LORD hath heard that I \emph{was} hated, he hath therefore given me this \emph{son} also: and she called his name Simeon."],
['Genesis','Gen','29','034','34',"And she conceived again, and bare a son; and said, Now this time will my husband be joined unto me, because I have born him three sons: therefore was his name called Levi."],
['Genesis','Gen','29','035','35',"And she conceived again, and bare a son: and she said, Now will I praise the LORD: therefore she called his name Judah; and left bearing."]
]

Genesis30 = [
    ['Genesis','Gen','30','001','1',"And when Rachel saw that she bare Jacob no children, Rachel envied her sister; and said unto Jacob, Give me children, or else I die."],
['Genesis','Gen','30','002','2',"And Jacob's anger was kindled against Rachel: and he said, \emph{Am} I in God's stead, who hath withheld from thee the fruit of the womb?"],
['Genesis','Gen','30','003','3',"And she said, Behold my maid Bilhah, go in unto her; and she shall bear upon my knees, that I may also have children by her."],
['Genesis','Gen','30','004','4',"And she gave him Bilhah her handmaid to wife: and Jacob went in unto her."],
['Genesis','Gen','30','005','5',"And Bilhah conceived, and bare Jacob a son."],
['Genesis','Gen','30','006','6',"And Rachel said, God hath judged me, and hath also heard my voice, and hath given me a son: therefore called she his name Dan."],
['Genesis','Gen','30','007','7',"And Bilhah Rachel's maid conceived again, and bare Jacob a second son."],
['Genesis','Gen','30','008','8',"And Rachel said, With great wrestlings have I wrestled with my sister, and I have prevailed: and she called his name Naphtali."],
['Genesis','Gen','30','009','9',"When Leah saw that she had left bearing, she took Zilpah her maid, and gave her Jacob to wife."],
['Genesis','Gen','30','010','10',"And Zilpah Leah's maid bare Jacob a son."],
['Genesis','Gen','30','011','11',"And Leah said, A troop cometh: and she called his name Gad."],
['Genesis','Gen','30','012','12',"And Zilpah Leah's maid bare Jacob a second son."],
['Genesis','Gen','30','013','13',"And Leah said, Happy am I, for the daughters will call me blessed: and she called his name Asher."],
['Genesis','Gen','30','014','14',"And Reuben went in the days of wheat harvest, and found mandrakes in the field, and brought them unto his mother Leah. Then Rachel said to Leah, Give me, I pray thee, of thy son's mandrakes."],
['Genesis','Gen','30','015','15',"And she said unto her, \emph{Is} \emph{it} a small matter that thou hast taken my husband? and wouldest thou take away my son's mandrakes also? And Rachel said, Therefore he shall lie with thee to night for thy son's mandrakes."],
['Genesis','Gen','30','016','16',"And Jacob came out of the field in the evening, and Leah went out to meet him, and said, Thou must come in unto me; for surely I have hired thee with my son's mandrakes. And he lay with her that night."],
['Genesis','Gen','30','017','17',"And God hearkened unto Leah, and she conceived, and bare Jacob the fifth son."],
['Genesis','Gen','30','018','18',"And Leah said, God hath given me my hire, because I have given my maiden to my husband: and she called his name Issachar."],
['Genesis','Gen','30','019','19',"And Leah conceived again, and bare Jacob the sixth son."],
['Genesis','Gen','30','020','20',"And Leah said, God hath endued me \emph{with} a good dowry; now will my husband dwell with me, because I have born him six sons: and she called his name Zebulun."],
['Genesis','Gen','30','021','21',"And afterwards she bare a daughter, and called her name Dinah."],
['Genesis','Gen','30','022','22',"And God remembered Rachel, and God hearkened to her, and opened her womb."],
['Genesis','Gen','30','023','23',"And she conceived, and bare a son; and said, God hath taken away my reproach:"],
['Genesis','Gen','30','024','24',"And she called his name Joseph; and said, The LORD shall add to me another son."],
['Genesis','Gen','30','025','25',"And it came to pass, when Rachel had born Joseph, that Jacob said unto Laban, Send me away, that I may go unto mine own place, and to my country."],
['Genesis','Gen','30','026','26',"Give \emph{me} my wives and my children, for whom I have served thee, and let me go: for thou knowest my service which I have done thee."],
['Genesis','Gen','30','027','27',"And Laban said unto him, I pray thee, if I have found favour in thine eyes, \emph{tarry:} \emph{for} I have learned by experience that the LORD hath blessed me for thy sake."],
['Genesis','Gen','30','028','28',"And he said, Appoint me thy wages, and I will give \emph{it}."],
['Genesis','Gen','30','029','29',"And he said unto him, Thou knowest how I have served thee, and how thy cattle was with me."],
['Genesis','Gen','30','030','30',"For \emph{it} \emph{was} little which thou hadst before I \emph{came}, and it is \emph{now} increased unto a multitude; and the LORD hath blessed thee since my coming: and now when shall I provide for mine own house also?"],
['Genesis','Gen','30','031','31',"And he said, What shall I give thee? And Jacob said, Thou shalt not give me any thing: if thou wilt do this thing for me, I will again feed \emph{and} keep thy flock:"],
['Genesis','Gen','30','032','32',"I will pass through all thy flock to day, removing from thence all the speckled and spotted cattle, and all the brown cattle among the sheep, and the spotted and speckled among the goats: and \emph{of} \emph{such} shall be my hire."],
['Genesis','Gen','30','033','33',"So shall my righteousness answer for me in time to come, when it shall come for my hire before thy face: every one that \emph{is} not speckled and spotted among the goats, and brown among the sheep, that shall be counted stolen with me."],
['Genesis','Gen','30','034','34',"And Laban said, Behold, I would it might be according to thy word."],
['Genesis','Gen','30','035','35',"And he removed that day the he goats that were ringstraked and spotted, and all the she goats that were speckled and spotted, \emph{and} every one that had \emph{some} white in it, and all the brown among the sheep, and gave \emph{them} into the hand of his sons."],
['Genesis','Gen','30','036','36',"And he set three days' journey betwixt himself and Jacob: and Jacob fed the rest of Laban's flocks."],
['Genesis','Gen','30','037','37',"And Jacob took him rods of green poplar, and of the hazel and chesnut tree; and pilled white strakes in them, and made the white appear which \emph{was} in the rods."],
['Genesis','Gen','30','038','38',"And he set the rods which he had pilled before the flocks in the gutters in the watering troughs when the flocks came to drink, that they should conceive when they came to drink."],
['Genesis','Gen','30','039','39',"And the flocks conceived before the rods, and brought forth cattle ringstraked, speckled, and spotted."],
['Genesis','Gen','30','040','40',"And Jacob did separate the lambs, and set the faces of the flocks toward the ringstraked, and all the brown in the flock of Laban; and he put his own flocks by themselves, and put them not unto Laban's cattle."],
['Genesis','Gen','30','041','41',"And it came to pass, whensoever the stronger cattle did conceive, that Jacob laid the rods before the eyes of the cattle in the gutters, that they might conceive among the rods."],
['Genesis','Gen','30','042','42',"But when the cattle were feeble, he put \emph{them} not in: so the feebler were Laban's, and the stronger Jacob's."],
['Genesis','Gen','30','043','43',"And the man increased exceedingly, and had much cattle, and maidservants, and menservants, and camels, and asses."]
]

Genesis31 = [
    ['Genesis','Gen','31','001','1',"And he heard the words of Laban's sons, saying, Jacob hath taken away all that \emph{was} our father's; and of \emph{that} which \emph{was} our father's hath he gotten all this glory."],
['Genesis','Gen','31','002','2',"And Jacob beheld the countenance of Laban, and, behold, it \emph{was} not toward him as before."],
['Genesis','Gen','31','003','3',"And the LORD said unto Jacob, Return unto the land of thy fathers, and to thy kindred; and I will be with thee."],
['Genesis','Gen','31','004','4',"And Jacob sent and called Rachel and Leah to the field unto his flock,"],
['Genesis','Gen','31','005','5',"And said unto them, I see your father's countenance, that it \emph{is} not toward me as before; but the God of my father hath been with me."],
['Genesis','Gen','31','006','6',"And ye know that with all my power I have served your father."],
['Genesis','Gen','31','007','7',"And your father hath deceived me, and changed my wages ten times; but God suffered him not to hurt me."],
['Genesis','Gen','31','008','8',"If he said thus, The speckled shall be thy wages; then all the cattle bare speckled: and if he said thus, The ringstraked shall be thy hire; then bare all the cattle ringstraked."],
['Genesis','Gen','31','009','9',"Thus God hath taken away the cattle of your father, and given \emph{them} to me."],
['Genesis','Gen','31','010','10',"And it came to pass at the time that the cattle conceived, that I lifted up mine eyes, and saw in a dream, and, behold, the rams which leaped upon the cattle \emph{were} ringstraked, speckled, and grisled."],
['Genesis','Gen','31','011','11',"And the angel of God spake unto me in a dream, \emph{saying}, Jacob: And I said, Here \emph{am} I."],
['Genesis','Gen','31','012','12',"And he said, Lift up now thine eyes, and see, all the rams which leap upon the cattle \emph{are} ringstraked, speckled, and grisled: for I have seen all that Laban doeth unto thee."],
['Genesis','Gen','31','013','13',"I \emph{am} the God of Beth-el, where thou anointedst the pillar, \emph{and} where thou vowedst a vow unto me: now arise, get thee out from this land, and return unto the land of thy kindred."],
['Genesis','Gen','31','014','14',"And Rachel and Leah answered and said unto him, \emph{Is} \emph{there} yet any portion or inheritance for us in our father's house?"],
['Genesis','Gen','31','015','15',"Are we not counted of him strangers? for he hath sold us, and hath quite devoured also our money."],
['Genesis','Gen','31','016','16',"For all the riches which God hath taken from our father, that \emph{is} our's, and our children's: now then, whatsoever God hath said unto thee, do."],
['Genesis','Gen','31','017','17',"Then Jacob rose up, and set his sons and his wives upon camels;"],
['Genesis','Gen','31','018','18',"And he carried away all his cattle, and all his goods which he had gotten, the cattle of his getting, which he had gotten in Padan-aram, for to go to Isaac his father in the land of Canaan."],
['Genesis','Gen','31','019','19',"And Laban went to shear his sheep: and Rachel had stolen the images that \emph{were} her father's."],
['Genesis','Gen','31','020','20',"And Jacob stole away unawares to Laban the Syrian, in that he told him not that he fled."],
['Genesis','Gen','31','021','21',"So he fled with all that he had; and he rose up, and passed over the river, and set his face \emph{toward} the mount Gilead."],
['Genesis','Gen','31','022','22',"And it was told Laban on the third day that Jacob was fled."],
['Genesis','Gen','31','023','23',"And he took his brethren with him, and pursued after him seven days' journey; and they overtook him in the mount Gilead."],
['Genesis','Gen','31','024','24',"And God came to Laban the Syrian in a dream by night, and said unto him, Take heed that thou speak not to Jacob either good or bad."],
['Genesis','Gen','31','025','25',"Then Laban overtook Jacob. Now Jacob had pitched his tent in the mount: and Laban with his brethren pitched in the mount of Gilead."],
['Genesis','Gen','31','026','26',"And Laban said to Jacob, What hast thou done, that thou hast stolen away unawares to me, and carried away my daughters, as captives \emph{taken} with the sword?"],
['Genesis','Gen','31','027','27',"Wherefore didst thou flee away secretly, and steal away from me; and didst not tell me, that I might have sent thee away with mirth, and with songs, with tabret, and with harp?"],
['Genesis','Gen','31','028','28',"And hast not suffered me to kiss my sons and my daughters? thou hast now done foolishly in \emph{so} doing."],
['Genesis','Gen','31','029','29',"It is in the power of my hand to do you hurt: but the God of your father spake unto me yesternight, saying, Take thou heed that thou speak not to Jacob either good or bad."],
['Genesis','Gen','31','030','30',"And now, \emph{though} thou wouldest needs be gone, because thou sore longedst after thy father's house, \emph{yet} wherefore hast thou stolen my gods?"],
['Genesis','Gen','31','031','31',"And Jacob answered and said to Laban, Because I was afraid: for I said, Peradventure thou wouldest take by force thy daughters from me."],
['Genesis','Gen','31','032','32',"With whomsoever thou findest thy gods, let him not live: before our brethren discern thou what \emph{is} thine with me, and take \emph{it} to thee. For Jacob knew not that Rachel had stolen them."],
['Genesis','Gen','31','033','33',"And Laban went into Jacob's tent, and into Leah's tent, and into the two maidservants' tents; but he found \emph{them} not. Then went he out of Leah's tent, and entered into Rachel's tent."],
['Genesis','Gen','31','034','34',"Now Rachel had taken the images, and put them in the camel's furniture, and sat upon them. And Laban searched all the tent, but found \emph{them} not."],
['Genesis','Gen','31','035','35',"And she said to her father, Let it not displease my lord that I cannot rise up before thee; for the custom of women \emph{is} upon me. And he searched, but found not the images."],
['Genesis','Gen','31','036','36',"And Jacob was wroth, and chode with Laban: and Jacob answered and said to Laban, What \emph{is} my trespass? what \emph{is} my sin, that thou hast so hotly pursued after me?"],
['Genesis','Gen','31','037','37',"Whereas thou hast searched all my stuff, what hast thou found of all thy household stuff? set \emph{it} here before my brethren and thy brethren, that they may judge betwixt us both."],
['Genesis','Gen','31','038','38',"This twenty years \emph{have} I \emph{been} with thee; thy ewes and thy she goats have not cast their young, and the rams of thy flock have I not eaten."],
['Genesis','Gen','31','039','39',"That which was torn \emph{of} \emph{beasts} I brought not unto thee; I bare the loss of it; of my hand didst thou require it, \emph{whether} stolen by day, or stolen by night."],
['Genesis','Gen','31','040','40',"\emph{Thus} I was; in the day the drought consumed me, and the frost by night; and my sleep departed from mine eyes."],
['Genesis','Gen','31','041','41',"Thus have I been twenty years in thy house; I served thee fourteen years for thy two daughters, and six years for thy cattle: and thou hast changed my wages ten times."],
['Genesis','Gen','31','042','42',"Except the God of my father, the God of Abraham, and the fear of Isaac, had been with me, surely thou hadst sent me away now empty. God hath seen mine affliction and the labour of my hands, and rebuked \emph{thee} yesternight."],
['Genesis','Gen','31','043','43',"And Laban answered and said unto Jacob, \emph{These} daughters \emph{are} my daughters, and \emph{these} children \emph{are} my children, and \emph{these} cattle \emph{are} my cattle, and all that thou seest \emph{is} mine: and what can I do this day unto these my daughters, or unto their children which they have born?"],
['Genesis','Gen','31','044','44',"Now therefore come thou, let us make a covenant, I and thou; and let it be for a witness between me and thee."],
['Genesis','Gen','31','045','45',"And Jacob took a stone, and set it up \emph{for} a pillar."],
['Genesis','Gen','31','046','46',"And Jacob said unto his brethren, Gather stones; and they took stones, and made an heap: and they did eat there upon the heap."],
['Genesis','Gen','31','047','47',"And Laban called it Jegarsahadutha: but Jacob called it Galeed."],
['Genesis','Gen','31','048','48',"And Laban said, This heap \emph{is} a witness between me and thee this day. Therefore was the name of it called Galeed;"],
['Genesis','Gen','31','049','49',"And Mizpah; for he said, The LORD watch between me and thee, when we are absent one from another."],
['Genesis','Gen','31','050','50',"If thou shalt afflict my daughters, or if thou shalt take \emph{other} wives beside my daughters, no man \emph{is} with us; see, God \emph{is} witness betwixt me and thee."],
['Genesis','Gen','31','051','51',"And Laban said to Jacob, Behold this heap, and behold \emph{this} pillar, which I have cast betwixt me and thee;"],
['Genesis','Gen','31','052','52',"This heap \emph{be} witness, and \emph{this} pillar \emph{be} witness, that I will not pass over this heap to thee, and that thou shalt not pass over this heap and this pillar unto me, for harm."],
['Genesis','Gen','31','053','53',"The God of Abraham, and the God of Nahor, the God of their father, judge betwixt us. And Jacob sware by the fear of his father Isaac."],
['Genesis','Gen','31','054','54',"Then Jacob offered sacrifice upon the mount, and called his brethren to eat bread: and they did eat bread, and tarried all night in the mount."],
['Genesis','Gen','31','055','55',"And early in the morning Laban rose up, and kissed his sons and his daughters, and blessed them: and Laban departed, and returned unto his place."]
]

Genesis32 = [
    ['Genesis','Gen','32','001','1',"And Jacob went on his way, and the angels of God met him."],
['Genesis','Gen','32','002','2',"And when Jacob saw them, he said, This \emph{is} God's host: and he called the name of that place Mahanaim."],
['Genesis','Gen','32','003','3',"And Jacob sent messengers before him to Esau his brother unto the land of Seir, the country of Edom."],
['Genesis','Gen','32','004','4',"And he commanded them, saying, Thus shall ye speak unto my lord Esau; Thy servant Jacob saith thus, I have sojourned with Laban, and stayed there until now:"],
['Genesis','Gen','32','005','5',"And I have oxen, and asses, flocks, and menservants, and womenservants: and I have sent to tell my lord, that I may find grace in thy sight."],
['Genesis','Gen','32','006','6',"And the messengers returned to Jacob, saying, We came to thy brother Esau, and also he cometh to meet thee, and four hundred men with him."],
['Genesis','Gen','32','007','7',"Then Jacob was greatly afraid and distressed: and he divided the people that \emph{was} with him, and the flocks, and herds, and the camels, into two bands;"],
['Genesis','Gen','32','008','8',"And said, If Esau come to the one company, and smite it, then the other company which is left shall escape."],
['Genesis','Gen','32','009','9',"And Jacob said, O God of my father Abraham, and God of my father Isaac, the LORD which saidst unto me, Return unto thy country, and to thy kindred, and I will deal well with thee:"],
['Genesis','Gen','32','010','10',"I am not worthy of the least of all the mercies, and of all the truth, which thou hast shewed unto thy servant; for with my staff I passed over this Jordan; and now I am become two bands."],
['Genesis','Gen','32','011','11',"Deliver me, I pray thee, from the hand of my brother, from the hand of Esau: for I fear him, lest he will come and smite me, \emph{and} the mother with the children."],
['Genesis','Gen','32','012','12',"And thou saidst, I will surely do thee good, and make thy seed as the sand of the sea, which cannot be numbered for multitude."],
['Genesis','Gen','32','013','13',"And he lodged there that same night; and took of that which came to his hand a present for Esau his brother;"],
['Genesis','Gen','32','014','14',"Two hundred she goats, and twenty he goats, two hundred ewes, and twenty rams,"],
['Genesis','Gen','32','015','15',"Thirty milch camels with their colts, forty kine, and ten bulls, twenty she asses, and ten foals."],
['Genesis','Gen','32','016','16',"And he delivered \emph{them} into the hand of his servants, every drove by themselves; and said unto his servants, Pass over before me, and put a space betwixt drove and drove."],
['Genesis','Gen','32','017','17',"And he commanded the foremost, saying, When Esau my brother meeteth thee, and asketh thee, saying, Whose \emph{art} thou? and whither goest thou? and whose \emph{are} these before thee?"],
['Genesis','Gen','32','018','18',"Then thou shalt say, \emph{They} \emph{be} thy servant Jacob's; it \emph{is} a present sent unto my lord Esau: and, behold, also he \emph{is} behind us."],
['Genesis','Gen','32','019','19',"And so commanded he the second, and the third, and all that followed the droves, saying, On this manner shall ye speak unto Esau, when ye find him."],
['Genesis','Gen','32','020','20',"And say ye moreover, Behold, thy servant Jacob \emph{is} behind us. For he said, I will appease him with the present that goeth before me, and afterward I will see his face; peradventure he will accept of me."],
['Genesis','Gen','32','021','21',"So went the present over before him: and himself lodged that night in the company."],
['Genesis','Gen','32','022','22',"And he rose up that night, and took his two wives, and his two womenservants, and his eleven sons, and passed over the ford Jabbok."],
['Genesis','Gen','32','023','23',"And he took them, and sent them over the brook, and sent over that he had."],
['Genesis','Gen','32','024','24',"And Jacob was left alone; and there wrestled a man with him until the breaking of the day."],
['Genesis','Gen','32','025','25',"And when he saw that he prevailed not against him, he touched the hollow of his thigh; and the hollow of Jacob's thigh was out of joint, as he wrestled with him."],
['Genesis','Gen','32','026','26',"And he said, Let me go, for the day breaketh. And he said, I will not let thee go, except thou bless me."],
['Genesis','Gen','32','027','27',"And he said unto him, What \emph{is} thy name? And he said, Jacob."],
['Genesis','Gen','32','028','28',"And he said, Thy name shall be called no more Jacob, but Israel: for as a prince hast thou power with God and with men, and hast prevailed."],
['Genesis','Gen','32','029','29',"And Jacob asked \emph{him}, and said, Tell \emph{me}, I pray thee, thy name. And he said, Wherefore \emph{is} it \emph{that} thou dost ask after my name? And he blessed him there."],
['Genesis','Gen','32','030','30',"And Jacob called the name of the place Peniel: for I have seen God face to face, and my life is preserved."],
['Genesis','Gen','32','031','31',"And as he passed over Penuel the sun rose upon him, and he halted upon his thigh."],
['Genesis','Gen','32','032','32',"Therefore the children of Israel eat not \emph{of} the sinew which shrank, which \emph{is} upon the hollow of the thigh, unto this day: because he touched the hollow of Jacob's thigh in the sinew that shrank."],
]

Genesis33 = [
    ['Genesis','Gen','33','001','1',"And Jacob lifted up his eyes, and looked, and, behold, Esau came, and with him four hundred men. And he divided the children unto Leah, and unto Rachel, and unto the two handmaids."],
['Genesis','Gen','33','002','2',"And he put the handmaids and their children foremost, and Leah and her children after, and Rachel and Joseph hindermost."],
['Genesis','Gen','33','003','3',"And he passed over before them, and bowed himself to the ground seven times, until he came near to his brother."],
['Genesis','Gen','33','004','4',"And Esau ran to meet him, and embraced him, and fell on his neck, and kissed him: and they wept."],
['Genesis','Gen','33','005','5',"And he lifted up his eyes, and saw the women and the children; and said, Who \emph{are} those with thee? And he said, The children which God hath graciously given thy servant."],
['Genesis','Gen','33','006','6',"Then the handmaidens came near, they and their children, and they bowed themselves."],
['Genesis','Gen','33','007','7',"And Leah also with her children came near, and bowed themselves: and after came Joseph near and Rachel, and they bowed themselves."],
['Genesis','Gen','33','008','8',"And he said, What \emph{meanest} thou by all this drove which I met? And he said, \emph{These} \emph{are} to find grace in the sight of my lord."],
['Genesis','Gen','33','009','9',"And Esau said, I have enough, my brother; keep that thou hast unto thyself."],
['Genesis','Gen','33','010','10',"And Jacob said, Nay, I pray thee, if now I have found grace in thy sight, then receive my present at my hand: for therefore I have seen thy face, as though I had seen the face of God, and thou wast pleased with me."],
['Genesis','Gen','33','011','11',"Take, I pray thee, my blessing that is brought to thee; because God hath dealt graciously with me, and because I have enough. And he urged him, and he took \emph{it}."],
['Genesis','Gen','33','012','12',"And he said, Let us take our journey, and let us go, and I will go before thee."],
['Genesis','Gen','33','013','13',"And he said unto him, My lord knoweth that the children \emph{are} tender, and the flocks and herds with young \emph{are} with me: and if men should overdrive them one day, all the flock will die."],
['Genesis','Gen','33','014','14',"Let my lord, I pray thee, pass over before his servant: and I will lead on softly, according as the cattle that goeth before me and the children be able to endure, until I come unto my lord unto Seir."],
['Genesis','Gen','33','015','15',"And Esau said, Let me now leave with thee \emph{some} of the folk that \emph{are} with me. And he said, What needeth it? let me find grace in the sight of my lord."],
['Genesis','Gen','33','016','16',"So Esau returned that day on his way unto Seir."],
['Genesis','Gen','33','017','17',"And Jacob journeyed to Succoth, and built him an house, and made booths for his cattle: therefore the name of the place is called Succoth."],
['Genesis','Gen','33','018','18',"And Jacob came to Shalem, a city of Shechem, which \emph{is} in the land of Canaan, when he came from Padan-aram; and pitched his tent before the city."],
['Genesis','Gen','33','019','19',"And he bought a parcel of a field, where he had spread his tent, at the hand of the children of Hamor, Shechem's father, for an hundred pieces of money."],
['Genesis','Gen','33','020','20',"And he erected there an altar, and called it El-elohe-Israel."]
]

Genesis34 = [
    ['Genesis','Gen','34','001','1',"And Dinah the daughter of Leah, which she bare unto Jacob, went out to see the daughters of the land."],
['Genesis','Gen','34','002','2',"And when Shechem the son of Hamor the Hivite, prince of the country, saw her, he took her, and lay with her, and defiled her."],
['Genesis','Gen','34','003','3',"And his soul clave unto Dinah the daughter of Jacob, and he loved the damsel, and spake kindly unto the damsel."],
['Genesis','Gen','34','004','4',"And Shechem spake unto his father Hamor, saying, Get me this damsel to wife."],
['Genesis','Gen','34','005','5',"And Jacob heard that he had defiled Dinah his daughter: now his sons were with his cattle in the field: and Jacob held his peace until they were come."],
['Genesis','Gen','34','006','6',"And Hamor the father of Shechem went out unto Jacob to commune with him."],
['Genesis','Gen','34','007','7',"And the sons of Jacob came out of the field when they heard \emph{it}: and the men were grieved, and they were very wroth, because he had wrought folly in Israel in lying with Jacob's daughter; which thing ought not to be done."],
['Genesis','Gen','34','008','8',"And Hamor communed with them, saying, The soul of my son Shechem longeth for your daughter: I pray you give her him to wife."],
['Genesis','Gen','34','009','9',"And make ye marriages with us, \emph{and} give your daughters unto us, and take our daughters unto you."],
['Genesis','Gen','34','010','10',"And ye shall dwell with us: and the land shall be before you; dwell and trade ye therein, and get you possessions therein."],
['Genesis','Gen','34','011','11',"And Shechem said unto her father and unto her brethren, Let me find grace in your eyes, and what ye shall say unto me I will give."],
['Genesis','Gen','34','012','12',"Ask me never so much dowry and gift, and I will give according as ye shall say unto me: but give me the damsel to wife."],
['Genesis','Gen','34','013','13',"And the sons of Jacob answered Shechem and Hamor his father deceitfully, and said, because he had defiled Dinah their sister:"],
['Genesis','Gen','34','014','14',"And they said unto them, We cannot do this thing, to give our sister to one that is uncircumcised; for that \emph{were} a reproach unto us:"],
['Genesis','Gen','34','015','15',"But in this will we consent unto you: If ye will be as we \emph{be}, that every male of you be circumcised;"],
['Genesis','Gen','34','016','16',"Then will we give our daughters unto you, and we will take your daughters to us, and we will dwell with you, and we will become one people."],
['Genesis','Gen','34','017','17',"But if ye will not hearken unto us, to be circumcised; then will we take our daughter, and we will be gone."],
['Genesis','Gen','34','018','18',"And their words pleased Hamor, and Shechem Hamor's son."],
['Genesis','Gen','34','019','19',"And the young man deferred not to do the thing, because he had delight in Jacob's daughter: and he \emph{was} more honourable than all the house of his father."],
['Genesis','Gen','34','020','20',"And Hamor and Shechem his son came unto the gate of their city, and communed with the men of their city, saying,"],
['Genesis','Gen','34','021','21',"These men \emph{are} peaceable with us; therefore let them dwell in the land, and trade therein; for the land, behold, \emph{it} \emph{is} large enough for them; let us take their daughters to us for wives, and let us give them our daughters."],
['Genesis','Gen','34','022','22',"Only herein will the men consent unto us for to dwell with us, to be one people, if every male among us be circumcised, as they \emph{are} circumcised."],
['Genesis','Gen','34','023','23',"\emph{Shall} not their cattle and their substance and every beast of their's \emph{be} our's? only let us consent unto them, and they will dwell with us."],
['Genesis','Gen','34','024','24',"And unto Hamor and unto Shechem his son hearkened all that went out of the gate of his city; and every male was circumcised, all that went out of the gate of his city."],
['Genesis','Gen','34','025','25',"And it came to pass on the third day, when they were sore, that two of the sons of Jacob, Simeon and Levi, Dinah's brethren, took each man his sword, and came upon the city boldly, and slew all the males."],
['Genesis','Gen','34','026','26',"And they slew Hamor and Shechem his son with the edge of the sword, and took Dinah out of Shechem's house, and went out."],
['Genesis','Gen','34','027','27',"The sons of Jacob came upon the slain, and spoiled the city, because they had defiled their sister."],
['Genesis','Gen','34','028','28',"They took their sheep, and their oxen, and their asses, and that which \emph{was} in the city, and that which \emph{was} in the field,"],
['Genesis','Gen','34','029','29',"And all their wealth, and all their little ones, and their wives took they captive, and spoiled even all that \emph{was} in the house."],
['Genesis','Gen','34','030','30',"And Jacob said to Simeon and Levi, Ye have troubled me to make me to stink among the inhabitants of the land, among the Canaanites and the Perizzites: and I \emph{being} few in number, they shall gather themselves together against me, and slay me; and I shall be destroyed, I and my house."],
['Genesis','Gen','34','031','31',"And they said, Should he deal with our sister as with an harlot?"]
]

Genesis35 = [
    ['Genesis','Gen','35','001','1',"And God said unto Jacob, Arise, go up to Beth-el, and dwell there: and make there an altar unto God, that appeared unto thee when thou fleddest from the face of Esau thy brother."],
['Genesis','Gen','35','002','2',"Then Jacob said unto his household, and to all that \emph{were} with him, Put away the strange gods that \emph{are} among you, and be clean, and change your garments:"],
['Genesis','Gen','35','003','3',"And let us arise, and go up to Beth-el; and I will make there an altar unto God, who answered me in the day of my distress, and was with me in the way which I went."],
['Genesis','Gen','35','004','4',"And they gave unto Jacob all the strange gods which \emph{were} in their hand, and \emph{all} \emph{their} earrings which \emph{were} in their ears; and Jacob hid them under the oak which \emph{was} by Shechem."],
['Genesis','Gen','35','005','5',"And they journeyed: and the terror of God was upon the cities that \emph{were} round about them, and they did not pursue after the sons of Jacob."],
['Genesis','Gen','35','006','6',"So Jacob came to Luz, which \emph{is} in the land of Canaan, that \emph{is}, Beth-el, he and all the people that \emph{were} with him."],
['Genesis','Gen','35','007','7',"And he built there an altar, and called the place El-beth-el: because there God appeared unto him, when he fled from the face of his brother."],
['Genesis','Gen','35','008','8',"But Deborah Rebekah's nurse died, and she was buried beneath Beth-el under an oak: and the name of it was called Allon-bachuth."],
['Genesis','Gen','35','009','9',"And God appeared unto Jacob again, when he came out of Padan-aram, and blessed him."],
['Genesis','Gen','35','010','10',"And God said unto him, Thy name \emph{is} Jacob: thy name shall not be called any more Jacob, but Israel shall be thy name: and he called his name Israel."],
['Genesis','Gen','35','011','11',"And God said unto him, I \emph{am} God Almighty: be fruitful and multiply; a nation and a company of nations shall be of thee, and kings shall come out of thy loins;"],
['Genesis','Gen','35','012','12',"And the land which I gave Abraham and Isaac, to thee I will give it, and to thy seed after thee will I give the land."],
['Genesis','Gen','35','013','13',"And God went up from him in the place where he talked with him."],
['Genesis','Gen','35','014','14',"And Jacob set up a pillar in the place where he talked with him, \emph{even} a pillar of stone: and he poured a drink offering thereon, and he poured oil thereon."],
['Genesis','Gen','35','015','15',"And Jacob called the name of the place where God spake with him, Beth-el."],
['Genesis','Gen','35','016','16',"And they journeyed from Beth-el; and there was but a little way to come to Ephrath: and Rachel travailed, and she had hard labour."],
['Genesis','Gen','35','017','17',"And it came to pass, when she was in hard labour, that the midwife said unto her, Fear not; thou shalt have this son also."],
['Genesis','Gen','35','018','18',"And it came to pass, as her soul was in departing, (for she died) that she called his name Ben-oni: but his father called him Benjamin."],
['Genesis','Gen','35','019','19',"And Rachel died, and was buried in the way to Ephrath, which \emph{is} Beth-lehem."],
['Genesis','Gen','35','020','20',"And Jacob set a pillar upon her grave: that \emph{is} the pillar of Rachel's grave unto this day."],
['Genesis','Gen','35','021','21',"And Israel journeyed, and spread his tent beyond the tower of Edar."],
['Genesis','Gen','35','022','22',"And it came to pass, when Israel dwelt in that land, that Reuben went and lay with Bilhah his father's concubine: and Israel heard \emph{it}. Now the sons of Jacob were twelve:"],
['Genesis','Gen','35','023','23',"The sons of Leah; Reuben, Jacob's firstborn, and Simeon, and Levi, and Judah, and Issachar, and Zebulun:"],
['Genesis','Gen','35','024','24',"The sons of Rachel; Joseph, and Benjamin:"],
['Genesis','Gen','35','025','25',"And the sons of Bilhah, Rachel's handmaid; Dan, and Naphtali:"],
['Genesis','Gen','35','026','26',"And the sons of Zilpah, Leah's handmaid; Gad, and Asher: these \emph{are} the sons of Jacob, which were born to him in Padan-aram."],
['Genesis','Gen','35','027','27',"And Jacob came unto Isaac his father unto Mamre, unto the city of Arbah, which \emph{is} Hebron, where Abraham and Isaac sojourned."],
['Genesis','Gen','35','028','28',"And the days of Isaac were an hundred and fourscore years."],
['Genesis','Gen','35','029','29',"And Isaac gave up the ghost, and died, and was gathered unto his people, \emph{being} old and full of days: and his sons Esau and Jacob buried him."]
]

Genesis36 = [
    ['Genesis','Gen','36','001','1',"Now these \emph{are} the generations of Esau, who \emph{is} Edom."],
['Genesis','Gen','36','002','2',"Esau took his wives of the daughters of Canaan; Adah the daughter of Elon the Hittite, and Aholibamah the daughter of Anah the daughter of Zibeon the Hivite;"],
['Genesis','Gen','36','003','3',"And Bashemath Ishmael's daughter, sister of Nebajoth."],
['Genesis','Gen','36','004','4',"And Adah bare to Esau Eliphaz; and Bashemath bare Reuel;"],
['Genesis','Gen','36','005','5',"And Aholibamah bare Jeush, and Jaalam, and Korah: these \emph{are} the sons of Esau, which were born unto him in the land of Canaan."],
['Genesis','Gen','36','006','6',"And Esau took his wives, and his sons, and his daughters, and all the persons of his house, and his cattle, and all his beasts, and all his substance, which he had got in the land of Canaan; and went into the country from the face of his brother Jacob."],
['Genesis','Gen','36','007','7',"For their riches were more than that they might dwell together; and the land wherein they were strangers could not bear them because of their cattle."],
['Genesis','Gen','36','008','8',"Thus dwelt Esau in mount Seir: Esau \emph{is} Edom."],
['Genesis','Gen','36','009','9',"And these \emph{are} the generations of Esau the father of the Edomites in mount Seir:"],
['Genesis','Gen','36','010','10',"These \emph{are} the names of Esau's sons; Eliphaz the son of Adah the wife of Esau, Reuel the son of Bashemath the wife of Esau."],
['Genesis','Gen','36','011','11',"And the sons of Eliphaz were Teman, Omar, Zepho, and Gatam, and Kenaz."],
['Genesis','Gen','36','012','12',"And Timna was concubine to Eliphaz Esau's son; and she bare to Eliphaz Amalek: these \emph{were} the sons of Adah Esau's wife."],
['Genesis','Gen','36','013','13',"And these \emph{are} the sons of Reuel; Nahath, and Zerah, Shammah, and Mizzah: these were the sons of Bashemath Esau's wife."],
['Genesis','Gen','36','014','14',"And these were the sons of Aholibamah, the daughter of Anah the daughter of Zibeon, Esau's wife: and she bare to Esau Jeush, and Jaalam, and Korah."],
['Genesis','Gen','36','015','15',"These \emph{were} dukes of the sons of Esau: the sons of Eliphaz the firstborn \emph{son} of Esau; duke Teman, duke Omar, duke Zepho, duke Kenaz,"],
['Genesis','Gen','36','016','16',"Duke Korah, duke Gatam, \emph{and} duke Amalek: these \emph{are} the dukes \emph{that} \emph{came} of Eliphaz in the land of Edom; these \emph{were} the sons of Adah."],
['Genesis','Gen','36','017','17',"And these \emph{are} the sons of Reuel Esau's son; duke Nahath, duke Zerah, duke Shammah, duke Mizzah: these \emph{are} the dukes \emph{that} \emph{came} of Reuel in the land of Edom; these \emph{are} the sons of Bashemath Esau's wife."],
['Genesis','Gen','36','018','18',"And these \emph{are} the sons of Aholibamah Esau's wife; duke Jeush, duke Jaalam, duke Korah: these \emph{were} the dukes \emph{that} \emph{came} of Aholibamah the daughter of Anah, Esau's wife."],
['Genesis','Gen','36','019','19',"These \emph{are} the sons of Esau, who \emph{is} Edom, and these \emph{are} their dukes."],
['Genesis','Gen','36','020','20',"These \emph{are} the sons of Seir the Horite, who inhabited the land; Lotan, and Shobal, and Zibeon, and Anah,"],
['Genesis','Gen','36','021','21',"And Dishon, and Ezer, and Dishan: these \emph{are} the dukes of the Horites, the children of Seir in the land of Edom."],
['Genesis','Gen','36','022','22',"And the children of Lotan were Hori and Hemam; and Lotan's sister \emph{was} Timna."],
['Genesis','Gen','36','023','23',"And the children of Shobal \emph{were} these; Alvan, and Manahath, and Ebal, Shepho, and Onam."],
['Genesis','Gen','36','024','24',"And these \emph{are} the children of Zibeon; both Ajah, and Anah: this \emph{was} \emph{that} Anah that found the mules in the wilderness, as he fed the asses of Zibeon his father."],
['Genesis','Gen','36','025','25',"And the children of Anah \emph{were} these; Dishon, and Aholibamah the daughter of Anah."],
['Genesis','Gen','36','026','26',"And these \emph{are} the children of Dishon; Hemdan, and Eshban, and Ithran, and Cheran."],
['Genesis','Gen','36','027','27',"The children of Ezer \emph{are} these; Bilhan, and Zaavan, and Akan."],
['Genesis','Gen','36','028','28',"The children of Dishan \emph{are} these; Uz, and Aran."],
['Genesis','Gen','36','029','29',"These \emph{are} the dukes \emph{that} \emph{came} of the Horites; duke Lotan, duke Shobal, duke Zibeon, duke Anah,"],
['Genesis','Gen','36','030','30',"Duke Dishon, duke Ezer, duke Dishan: these \emph{are} the dukes \emph{that} \emph{came} of Hori, among their dukes in the land of Seir."],
['Genesis','Gen','36','031','31',"And these \emph{are} the kings that reigned in the land of Edom, before there reigned any king over the children of Israel."],
['Genesis','Gen','36','032','32',"And Bela the son of Beor reigned in Edom: and the name of his city \emph{was} Dinhabah."],
['Genesis','Gen','36','033','33',"And Bela died, and Jobab the son of Zerah of Bozrah reigned in his stead."],
['Genesis','Gen','36','034','34',"And Jobab died, and Husham of the land of Temani reigned in his stead."],
['Genesis','Gen','36','035','35',"And Husham died, and Hadad the son of Bedad, who smote Midian in the field of Moab, reigned in his stead: and the name of his city \emph{was} Avith."],
['Genesis','Gen','36','036','36',"And Hadad died, and Samlah of Masrekah reigned in his stead."],
['Genesis','Gen','36','037','37',"And Samlah died, and Saul of Rehoboth \emph{by} the river reigned in his stead."],
['Genesis','Gen','36','038','38',"And Saul died, and Baal-hanan the son of Achbor reigned in his stead."],
['Genesis','Gen','36','039','39',"And Baal-hanan the son of Achbor died, and Hadar reigned in his stead: and the name of his city \emph{was} Pau; and his wife's name \emph{was} Mehetabel, the daughter of Matred, the daughter of Mezahab."],
['Genesis','Gen','36','040','40',"And these \emph{are} the names of the dukes \emph{that} \emph{came} of Esau, according to their families, after their places, by their names; duke Timnah, duke Alvah, duke Jetheth,"],
['Genesis','Gen','36','041','41',"Duke Aholibamah, duke Elah, duke Pinon,"],
['Genesis','Gen','36','042','42',"Duke Kenaz, duke Teman, duke Mibzar,"],
['Genesis','Gen','36','043','43',"Duke Magdiel, duke Iram: these \emph{be} the dukes of Edom, according to their habitations in the land of their possession: he \emph{is} Esau the father of the Edomites."]
]

Genesis37 = [
    ['Genesis','Gen','37','001','1',"And Jacob dwelt in the land wherein his father was a stranger, in the land of Canaan."],
['Genesis','Gen','37','002','2',"These \emph{are} the generations of Jacob. Joseph, \emph{being} seventeen years old, was feeding the flock with his brethren; and the lad \emph{was} with the sons of Bilhah, and with the sons of Zilpah, his father's wives: and Joseph brought unto his father their evil report."],
['Genesis','Gen','37','003','3',"Now Israel loved Joseph more than all his children, because he \emph{was} the son of his old age: and he made him a coat of \emph{many} colours."],
['Genesis','Gen','37','004','4',"And when his brethren saw that their father loved him more than all his brethren, they hated him, and could not speak peaceably unto him."],
['Genesis','Gen','37','005','5',"And Joseph dreamed a dream, and he told \emph{it} his brethren: and they hated him yet the more."],
['Genesis','Gen','37','006','6',"And he said unto them, Hear, I pray you, this dream which I have dreamed:"],
['Genesis','Gen','37','007','7',"For, behold, we \emph{were} binding sheaves in the field, and, lo, my sheaf arose, and also stood upright; and, behold, your sheaves stood round about, and made obeisance to my sheaf."],
['Genesis','Gen','37','008','8',"And his brethren said to him, Shalt thou indeed reign over us? or shalt thou indeed have dominion over us? And they hated him yet the more for his dreams, and for his words."],
['Genesis','Gen','37','009','9',"And he dreamed yet another dream, and told it his brethren, and said, Behold, I have dreamed a dream more; and, behold, the sun and the moon and the eleven stars made obeisance to me."],
['Genesis','Gen','37','010','10',"And he told \emph{it} to his father, and to his brethren: and his father rebuked him, and said unto him, What \emph{is} this dream that thou hast dreamed? Shall I and thy mother and thy brethren indeed come to bow down ourselves to thee to the earth?"],
['Genesis','Gen','37','011','11',"And his brethren envied him; but his father observed the saying."],
['Genesis','Gen','37','012','12',"And his brethren went to feed their father's flock in Shechem."],
['Genesis','Gen','37','013','13',"And Israel said unto Joseph, Do not thy brethren feed \emph{the} \emph{flock} in Shechem? come, and I will send thee unto them. And he said to him, Here \emph{am} \emph{I}."],
['Genesis','Gen','37','014','14',"And he said to him, Go, I pray thee, see whether it be well with thy brethren, and well with the flocks; and bring me word again. So he sent him out of the vale of Hebron, and he came to Shechem."],
['Genesis','Gen','37','015','15',"And a certain man found him, and, behold, \emph{he} \emph{was} wandering in the field: and the man asked him, saying, What seekest thou?"],
['Genesis','Gen','37','016','16',"And he said, I seek my brethren: tell me, I pray thee, where they feed \emph{their} \emph{flocks}."],
['Genesis','Gen','37','017','17',"And the man said, They are departed hence; for I heard them say, Let us go to Dothan. And Joseph went after his brethren, and found them in Dothan."],
['Genesis','Gen','37','018','18',"And when they saw him afar off, even before he came near unto them, they conspired against him to slay him."],
['Genesis','Gen','37','019','19',"And they said one to another, Behold, this dreamer cometh."],
['Genesis','Gen','37','020','20',"Come now therefore, and let us slay him, and cast him into some pit, and we will say, Some evil beast hath devoured him: and we shall see what will become of his dreams."],
['Genesis','Gen','37','021','21',"And Reuben heard \emph{it}, and he delivered him out of their hands; and said, Let us not kill him."],
['Genesis','Gen','37','022','22',"And Reuben said unto them, Shed no blood, \emph{but} cast him into this pit that \emph{is} in the wilderness, and lay no hand upon him; that he might rid him out of their hands, to deliver him to his father again."],
['Genesis','Gen','37','023','23',"And it came to pass, when Joseph was come unto his brethren, that they stript Joseph out of his coat, \emph{his} coat of \emph{many} colours that \emph{was} on him;"],
['Genesis','Gen','37','024','24',"And they took him, and cast him into a pit: and the pit \emph{was} empty, \emph{there} \emph{was} no water in it."],
['Genesis','Gen','37','025','25',"And they sat down to eat bread: and they lifted up their eyes and looked, and, behold, a company of Ishmeelites came from Gilead with their camels bearing spicery and balm and myrrh, going to carry \emph{it} down to Egypt."],
['Genesis','Gen','37','026','26',"And Judah said unto his brethren, What profit \emph{is} \emph{it} if we slay our brother, and conceal his blood?"],
['Genesis','Gen','37','027','27',"Come, and let us sell him to the Ishmeelites, and let not our hand be upon him; for he \emph{is} our brother \emph{and} our flesh. And his brethren were content."],
['Genesis','Gen','37','028','28',"Then there passed by Midianites merchantmen; and they drew and lifted up Joseph out of the pit, and sold Joseph to the Ishmeelites for twenty \emph{pieces} of silver: and they brought Joseph into Egypt."],
['Genesis','Gen','37','029','29',"And Reuben returned unto the pit; and, behold, Joseph \emph{was} not in the pit; and he rent his clothes."],
['Genesis','Gen','37','030','30',"And he returned unto his brethren, and said, The child \emph{is} not; and I, whither shall I go?"],
['Genesis','Gen','37','031','31',"And they took Joseph's coat, and killed a kid of the goats, and dipped the coat in the blood;"],
['Genesis','Gen','37','032','32',"And they sent the coat of \emph{many} colours, and they brought \emph{it} to their father; and said, This have we found: know now whether it \emph{be} thy son's coat or no."],
['Genesis','Gen','37','033','33',"And he knew it, and said, \emph{It} \emph{is} my son's coat; an evil beast hath devoured him; Joseph is without doubt rent in pieces."],
['Genesis','Gen','37','034','34',"And Jacob rent his clothes, and put sackcloth upon his loins, and mourned for his son many days."],
['Genesis','Gen','37','035','35',"And all his sons and all his daughters rose up to comfort him; but he refused to be comforted; and he said, For I will go down into the grave unto my son mourning. Thus his father wept for him."],
['Genesis','Gen','37','036','36',"And the Midianites sold him into Egypt unto Potiphar, an officer of Pharaoh's, \emph{and} captain of the guard."]
]

Genesis38 = [
    ['Genesis','Gen','38','001','1',"And it came to pass at that time, that Judah went down from his brethren, and turned in to a certain Adullamite, whose name \emph{was} Hirah."],
['Genesis','Gen','38','002','2',"And Judah saw there a daughter of a certain Canaanite, whose name \emph{was} Shuah; and he took her, and went in unto her."],
['Genesis','Gen','38','003','3',"And she conceived, and bare a son; and he called his name Er."],
['Genesis','Gen','38','004','4',"And she conceived again, and bare a son; and she called his name Onan."],
['Genesis','Gen','38','005','5',"And she yet again conceived, and bare a son; and called his name Shelah: and he was at Chezib, when she bare him."],
['Genesis','Gen','38','006','6',"And Judah took a wife for Er his firstborn, whose name \emph{was} Tamar."],
['Genesis','Gen','38','007','7',"And Er, Judah's firstborn, was wicked in the sight of the LORD; and the LORD slew him."],
['Genesis','Gen','38','008','8',"And Judah said unto Onan, Go in unto thy brother's wife, and marry her, and raise up seed to thy brother."],
['Genesis','Gen','38','009','9',"And Onan knew that the seed should not be his; and it came to pass, when he went in unto his brother's wife, that he spilled \emph{it} on the ground, lest that he should give seed to his brother."],
['Genesis','Gen','38','010','10',"And the thing which he did displeased the LORD: wherefore he slew him also."],
['Genesis','Gen','38','011','11',"Then said Judah to Tamar his daughter in law, Remain a widow at thy father's house, till Shelah my son be grown: for he said, Lest peradventure he die also, as his brethren \emph{did}. And Tamar went and dwelt in her father's house."],
['Genesis','Gen','38','012','12',"And in process of time the daughter of Shuah Judah's wife died; and Judah was comforted, and went up unto his sheepshearers to Timnath, he and his friend Hirah the Adullamite."],
['Genesis','Gen','38','013','13',"And it was told Tamar, saying, Behold thy father in law goeth up to Timnath to shear his sheep."],
['Genesis','Gen','38','014','14',"And she put her widow's garments off from her, and covered her with a vail, and wrapped herself, and sat in an open place, which \emph{is} by the way to Timnath; for she saw that Shelah was grown, and she was not given unto him to wife."],
['Genesis','Gen','38','015','15',"When Judah saw her, he thought her \emph{to} \emph{be} an harlot; because she had covered her face."],
['Genesis','Gen','38','016','16',"And he turned unto her by the way, and said, Go to, I pray thee, let me come in unto thee; (for he knew not that she \emph{was} his daughter in law.) And she said, What wilt thou give me, that thou mayest come in unto me?"],
['Genesis','Gen','38','017','17',"And he said, I will send \emph{thee} a kid from the flock. And she said, Wilt thou give \emph{me} a pledge, till thou send \emph{it}?"],
['Genesis','Gen','38','018','18',"And he said, What pledge shall I give thee? And she said, Thy signet, and thy bracelets, and thy staff that \emph{is} in thine hand. And he gave \emph{it} her, and came in unto her, and she conceived by him."],
['Genesis','Gen','38','019','19',"And she arose, and went away, and laid by her vail from her, and put on the garments of her widowhood."],
['Genesis','Gen','38','020','20',"And Judah sent the kid by the hand of his friend the Adullamite, to receive \emph{his} pledge from the woman's hand: but he found her not."],
['Genesis','Gen','38','021','21',"Then he asked the men of that place, saying, Where \emph{is} the harlot, that \emph{was} openly by the way side? And they said, There was no harlot in this \emph{place}."],
['Genesis','Gen','38','022','22',"And he returned to Judah, and said, I cannot find her; and also the men of the place said, \emph{that} there was no harlot in this \emph{place}."],
['Genesis','Gen','38','023','23',"And Judah said, Let her take \emph{it} to her, lest we be shamed: behold, I sent this kid, and thou hast not found her."],
['Genesis','Gen','38','024','24',"And it came to pass about three months after, that it was told Judah, saying, Tamar thy daughter in law hath played the harlot; and also, behold, she \emph{is} with child by whoredom. And Judah said, Bring her forth, and let her be burnt."],
['Genesis','Gen','38','025','25',"When she \emph{was} brought forth, she sent to her father in law, saying, By the man, whose these \emph{are,} \emph{am} I with child: and she said, Discern, I pray thee, whose \emph{are} these, the signet, and bracelets, and staff."],
['Genesis','Gen','38','026','26',"And Judah acknowledged \emph{them}, and said, She hath been more righteous than I; because that I gave her not to Shelah my son. And he knew her again no more."],
['Genesis','Gen','38','027','27',"And it came to pass in the time of her travail, that, behold, twins \emph{were} in her womb."],
['Genesis','Gen','38','028','28',"And it came to pass, when she travailed, that \emph{the} \emph{one} put out \emph{his} hand: and the midwife took and bound upon his hand a scarlet thread, saying, This came out first."],
['Genesis','Gen','38','029','29',"And it came to pass, as he drew back his hand, that, behold, his brother came out: and she said, How hast thou broken forth? \emph{this} breach \emph{be} upon thee: therefore his name was called Pharez."],
['Genesis','Gen','38','030','30',"And afterward came out his brother, that had the scarlet thread upon his hand: and his name was called Zarah."]
]

Genesis39 = [
    ['Genesis','Gen','39','001','1',"And Joseph was brought down to Egypt; and Potiphar, an officer of Pharaoh, captain of the guard, an Egyptian, bought him of the hands of the Ishmeelites, which had brought him down thither."],
['Genesis','Gen','39','002','2',"And the LORD was with Joseph, and he was a prosperous man; and he was in the house of his master the Egyptian."],
['Genesis','Gen','39','003','3',"And his master saw that the LORD \emph{was} with him, and that the LORD made all that he did to prosper in his hand."],
['Genesis','Gen','39','004','4',"And Joseph found grace in his sight, and he served him: and he made him overseer over his house, and all \emph{that} he had he put into his hand."],
['Genesis','Gen','39','005','5',"And it came to pass from the time \emph{that} he had made him overseer in his house, and over all that he had, that the LORD blessed the Egyptian's house for Joseph's sake; and the blessing of the LORD was upon all that he had in the house, and in the field."],
['Genesis','Gen','39','006','6',"And he left all that he had in Joseph's hand; and he knew not ought he had, save the bread which he did eat. And Joseph was \emph{a} goodly \emph{person}, and well favoured."],
['Genesis','Gen','39','007','7',"And it came to pass after these things, that his master's wife cast her eyes upon Joseph; and she said, Lie with me."],
['Genesis','Gen','39','008','8',"But he refused, and said unto his master's wife, Behold, my master wotteth not what \emph{is} with me in the house, and he hath committed all that he hath to my hand;"],
['Genesis','Gen','39','009','9',"\emph{There} \emph{is} none greater in this house than I; neither hath he kept back any thing from me but thee, because thou \emph{art} his wife: how then can I do this great wickedness, and sin against God?"],
['Genesis','Gen','39','010','10',"And it came to pass, as she spake to Joseph day by day, that he hearkened not unto her, to lie by her, \emph{or} to be with her."],
['Genesis','Gen','39','011','11',"And it came to pass about this time, that \emph{Joseph} went into the house to do his business; and \emph{there} \emph{was} none of the men of the house there within."],
['Genesis','Gen','39','012','12',"And she caught him by his garment, saying, Lie with me: and he left his garment in her hand, and fled, and got him out."],
['Genesis','Gen','39','013','13',"And it came to pass, when she saw that he had left his garment in her hand, and was fled forth,"],
['Genesis','Gen','39','014','14',"That she called unto the men of her house, and spake unto them, saying, See, he hath brought in an Hebrew unto us to mock us; he came in unto me to lie with me, and I cried with a loud voice:"],
['Genesis','Gen','39','015','15',"And it came to pass, when he heard that I lifted up my voice and cried, that he left his garment with me, and fled, and got him out."],
['Genesis','Gen','39','016','16',"And she laid up his garment by her, until his lord came home."],
['Genesis','Gen','39','017','17',"And she spake unto him according to these words, saying, The Hebrew servant, which thou hast brought unto us, came in unto me to mock me:"],
['Genesis','Gen','39','018','18',"And it came to pass, as I lifted up my voice and cried, that he left his garment with me, and fled out."],
['Genesis','Gen','39','019','19',"And it came to pass, when his master heard the words of his wife, which she spake unto him, saying, After this manner did thy servant to me; that his wrath was kindled."],
['Genesis','Gen','39','020','20',"And Joseph's master took him, and put him into the prison, a place where the king's prisoners \emph{were} bound: and he was there in the prison."],
['Genesis','Gen','39','021','21',"But the LORD was with Joseph, and shewed him mercy, and gave him favour in the sight of the keeper of the prison."],
['Genesis','Gen','39','022','22',"And the keeper of the prison committed to Joseph's hand all the prisoners that \emph{were} in the prison; and whatsoever they did there, he was the doer \emph{of} \emph{it}."],
['Genesis','Gen','39','023','23',"The keeper of the prison looked not to any thing \emph{that} \emph{was} under his hand; because the LORD was with him, and \emph{that} which he did, the LORD made \emph{it} to prosper."]
]

Genesis40 = [
    ['Genesis','Gen','40','001','1',"And it came to pass after these things, \emph{that} the butler of the king of Egypt and \emph{his} baker had offended their lord the king of Egypt."],
['Genesis','Gen','40','002','2',"And Pharaoh was wroth against two \emph{of} his officers, against the chief of the butlers, and against the chief of the bakers."],
['Genesis','Gen','40','003','3',"And he put them in ward in the house of the captain of the guard, into the prison, the place where Joseph \emph{was} bound."],
['Genesis','Gen','40','004','4',"And the captain of the guard charged Joseph with them, and he served them: and they continued a season in ward."],
['Genesis','Gen','40','005','5',"And they dreamed a dream both of them, each man his dream in one night, each man according to the interpretation of his dream, the butler and the baker of the king of Egypt, which \emph{were} bound in the prison."],
['Genesis','Gen','40','006','6',"And Joseph came in unto them in the morning, and looked upon them, and, behold, they \emph{were} sad."],
['Genesis','Gen','40','007','7',"And he asked Pharaoh's officers that \emph{were} with him in the ward of his lord's house, saying, Wherefore look ye \emph{so} sadly to day?"],
['Genesis','Gen','40','008','8',"And they said unto him, We have dreamed a dream, and \emph{there} \emph{is} no interpreter of it. And Joseph said unto them, \emph{Do} not interpretations \emph{belong} to God? tell me \emph{them}, I pray you."],
['Genesis','Gen','40','009','9',"And the chief butler told his dream to Joseph, and said to him, In my dream, behold, a vine \emph{was} before me;"],
['Genesis','Gen','40','010','10',"And in the vine \emph{were} three branches: and it \emph{was} as though it budded, \emph{and} her blossoms shot forth; and the clusters thereof brought forth ripe grapes:"],
['Genesis','Gen','40','011','11',"And Pharaoh's cup \emph{was} in my hand: and I took the grapes, and pressed them into Pharaoh's cup, and I gave the cup into Pharaoh's hand."],
['Genesis','Gen','40','012','12',"And Joseph said unto him, This \emph{is} the interpretation of it: The three branches \emph{are} three days:"],
['Genesis','Gen','40','013','13',"Yet within three days shall Pharaoh lift up thine head, and restore thee unto thy place: and thou shalt deliver Pharaoh's cup into his hand, after the former manner when thou wast his butler."],
['Genesis','Gen','40','014','14',"But think on me when it shall be well with thee, and shew kindness, I pray thee, unto me, and make mention of me unto Pharaoh, and bring me out of this house:"],
['Genesis','Gen','40','015','15',"For indeed I was stolen away out of the land of the Hebrews: and here also have I done nothing that they should put me into the dungeon."],
['Genesis','Gen','40','016','16',"When the chief baker saw that the interpretation was good, he said unto Joseph, I also \emph{was} in my dream, and, behold, \emph{I} \emph{had} three white baskets on my head:"],
['Genesis','Gen','40','017','17',"And in the uppermost basket \emph{there} \emph{was} of all manner of bakemeats for Pharaoh; and the birds did eat them out of the basket upon my head."],
['Genesis','Gen','40','018','18',"And Joseph answered and said, This \emph{is} the interpretation thereof: The three baskets \emph{are} three days:"],
['Genesis','Gen','40','019','19',"Yet within three days shall Pharaoh lift up thy head from off thee, and shall hang thee on a tree; and the birds shall eat thy flesh from off thee."],
['Genesis','Gen','40','020','20',"And it came to pass the third day, \emph{which} \emph{was} Pharaoh's birthday, that he made a feast unto all his servants: and he lifted up the head of the chief butler and of the chief baker among his servants."],
['Genesis','Gen','40','021','21',"And he restored the chief butler unto his butlership again; and he gave the cup into Pharaoh's hand:"],
['Genesis','Gen','40','022','22',"But he hanged the chief baker: as Joseph had interpreted to them."],
['Genesis','Gen','40','023','23',"Yet did not the chief butler remember Joseph, but forgat him."]
]
    
Genesis41 = [
    ['Genesis','Gen','41','001','1',"And it came to pass at the end of two full years, that Pharaoh dreamed: and, behold, he stood by the river."],
['Genesis','Gen','41','002','2',"And, behold, there came up out of the river seven well favoured kine and fatfleshed; and they fed in a meadow."],
['Genesis','Gen','41','003','3',"And, behold, seven other kine came up after them out of the river, ill favoured and leanfleshed; and stood by the \emph{other} kine upon the brink of the river."],
['Genesis','Gen','41','004','4',"And the ill favoured and leanfleshed kine did eat up the seven well favoured and fat kine. So Pharaoh awoke."],
['Genesis','Gen','41','005','5',"And he slept and dreamed the second time: and, behold, seven ears of corn came up upon one stalk, rank and good."],
['Genesis','Gen','41','006','6',"And, behold, seven thin ears and blasted with the east wind sprung up after them."],
['Genesis','Gen','41','007','7',"And the seven thin ears devoured the seven rank and full ears. And Pharaoh awoke, and, behold, \emph{it} \emph{was} a dream."],
['Genesis','Gen','41','008','8',"And it came to pass in the morning that his spirit was troubled; and he sent and called for all the magicians of Egypt, and all the wise men thereof: and Pharaoh told them his dream; but \emph{there} \emph{was} none that could interpret them unto Pharaoh."],
['Genesis','Gen','41','009','9',"Then spake the chief butler unto Pharaoh, saying, I do remember my faults this day:"],
['Genesis','Gen','41','010','10',"Pharaoh was wroth with his servants, and put me in ward in the captain of the guard's house, \emph{both} me and the chief baker:"],
['Genesis','Gen','41','011','11',"And we dreamed a dream in one night, I and he; we dreamed each man according to the interpretation of his dream."],
['Genesis','Gen','41','012','12',"And \emph{there} \emph{was} there with us a young man, an Hebrew, servant to the captain of the guard; and we told him, and he interpreted to us our dreams; to each man according to his dream he did interpret."],
['Genesis','Gen','41','013','13',"And it came to pass, as he interpreted to us, so it was; me he restored unto mine office, and him he hanged."],
['Genesis','Gen','41','014','14',"Then Pharaoh sent and called Joseph, and they brought him hastily out of the dungeon: and he shaved \emph{himself}, and changed his raiment, and came in unto Pharaoh."],
['Genesis','Gen','41','015','15',"And Pharaoh said unto Joseph, I have dreamed a dream, and \emph{there} \emph{is} none that can interpret it: and I have heard say of thee, \emph{that} thou canst understand a dream to interpret it."],
['Genesis','Gen','41','016','16',"And Joseph answered Pharaoh, saying, \emph{It} \emph{is} not in me: God shall give Pharaoh an answer of peace."],
['Genesis','Gen','41','017','17',"And Pharaoh said unto Joseph, In my dream, behold, I stood upon the bank of the river:"],
['Genesis','Gen','41','018','18',"And, behold, there came up out of the river seven kine, fatfleshed and well favoured; and they fed in a meadow:"],
['Genesis','Gen','41','019','19',"And, behold, seven other kine came up after them, poor and very ill favoured and leanfleshed, such as I never saw in all the land of Egypt for badness:"],
['Genesis','Gen','41','020','20',"And the lean and the ill favoured kine did eat up the first seven fat kine:"],
['Genesis','Gen','41','021','21',"And when they had eaten them up, it could not be known that they had eaten them; but they \emph{were} still ill favoured, as at the beginning. So I awoke."],
['Genesis','Gen','41','022','22',"And I saw in my dream, and, behold, seven ears came up in one stalk, full and good:"],
['Genesis','Gen','41','023','23',"And, behold, seven ears, withered, thin, \emph{and} blasted with the east wind, sprung up after them:"],
['Genesis','Gen','41','024','24',"And the thin ears devoured the seven good ears: and I told \emph{this} unto the magicians; but \emph{there} \emph{was} none that could declare \emph{it} to me."],
['Genesis','Gen','41','025','25',"And Joseph said unto Pharaoh, The dream of Pharaoh \emph{is} one: God hath shewed Pharaoh what he \emph{is} about to do."],
['Genesis','Gen','41','026','26',"The seven good kine \emph{are} seven years; and the seven good ears \emph{are} seven years: the dream \emph{is} one."],
['Genesis','Gen','41','027','27',"And the seven thin and ill favoured kine that came up after them \emph{are} seven years; and the seven empty ears blasted with the east wind shall be seven years of famine."],
['Genesis','Gen','41','028','28',"This \emph{is} the thing which I have spoken unto Pharaoh: What God \emph{is} about to do he sheweth unto Pharaoh."],
['Genesis','Gen','41','029','29',"Behold, there come seven years of great plenty throughout all the land of Egypt:"],
['Genesis','Gen','41','030','30',"And there shall arise after them seven years of famine; and all the plenty shall be forgotten in the land of Egypt; and the famine shall consume the land;"],
['Genesis','Gen','41','031','31',"And the plenty shall not be known in the land by reason of that famine following; for it \emph{shall} \emph{be} very grievous."],
['Genesis','Gen','41','032','32',"And for that the dream was doubled unto Pharaoh twice; \emph{it} \emph{is} because the thing \emph{is} established by God, and God will shortly bring it to pass."],
['Genesis','Gen','41','033','33',"Now therefore let Pharaoh look out a man discreet and wise, and set him over the land of Egypt."],
['Genesis','Gen','41','034','34',"Let Pharaoh do \emph{this}, and let him appoint officers over the land, and take up the fifth part of the land of Egypt in the seven plenteous years."],
['Genesis','Gen','41','035','35',"And let them gather all the food of those good years that come, and lay up corn under the hand of Pharaoh, and let them keep food in the cities."],
['Genesis','Gen','41','036','36',"And that food shall be for store to the land against the seven years of famine, which shall be in the land of Egypt; that the land perish not through the famine."],
['Genesis','Gen','41','037','37',"And the thing was good in the eyes of Pharaoh, and in the eyes of all his servants."],
['Genesis','Gen','41','038','38',"And Pharaoh said unto his servants, Can we find \emph{such} \emph{a} \emph{one} as this \emph{is}, a man in whom the Spirit of God \emph{is}?"],
['Genesis','Gen','41','039','39',"And Pharaoh said unto Joseph, Forasmuch as God hath shewed thee all this, \emph{there} \emph{is} none so discreet and wise as thou \emph{art}:"],
['Genesis','Gen','41','040','40',"Thou shalt be over my house, and according unto thy word shall all my people be ruled: only in the throne will I be greater than thou."],
['Genesis','Gen','41','041','41',"And Pharaoh said unto Joseph, See, I have set thee over all the land of Egypt."],
['Genesis','Gen','41','042','42',"And Pharaoh took off his ring from his hand, and put it upon Joseph's hand, and arrayed him in vestures of fine linen, and put a gold chain about his neck;"],
['Genesis','Gen','41','043','43',"And he made him to ride in the second chariot which he had; and they cried before him, Bow the knee: and he made him \emph{ruler} over all the land of Egypt."],
['Genesis','Gen','41','044','44',"And Pharaoh said unto Joseph, I \emph{am} Pharaoh, and without thee shall no man lift up his hand or foot in all the land of Egypt."],
['Genesis','Gen','41','045','45',"And Pharaoh called Joseph's name Zaphnath-paaneah; and he gave him to wife Asenath the daughter of Poti-pherah priest of On. And Joseph went out over \emph{all} the land of Egypt."],
['Genesis','Gen','41','046','46',"And Joseph \emph{was} thirty years old when he stood before Pharaoh king of Egypt. And Joseph went out from the presence of Pharaoh, and went throughout all the land of Egypt."],
['Genesis','Gen','41','047','47',"And in the seven plenteous years the earth brought forth by handfuls."],
['Genesis','Gen','41','048','48',"And he gathered up all the food of the seven years, which were in the land of Egypt, and laid up the food in the cities: the food of the field, which \emph{was} round about every city, laid he up in the same."],
['Genesis','Gen','41','049','49',"And Joseph gathered corn as the sand of the sea, very much, until he left numbering; for \emph{it} \emph{was} without number."],
['Genesis','Gen','41','050','50',"And unto Joseph were born two sons before the years of famine came, which Asenath the daughter of Poti-pherah priest of On bare unto him."],
['Genesis','Gen','41','051','51',"And Joseph called the name of the firstborn Manasseh: For God, \emph{said} \emph{he}, hath made me forget all my toil, and all my father's house."],
['Genesis','Gen','41','052','52',"And the name of the second called he Ephraim: For God hath caused me to be fruitful in the land of my affliction."],
['Genesis','Gen','41','053','53',"And the seven years of plenteousness, that was in the land of Egypt, were ended."],
['Genesis','Gen','41','054','54',"And the seven years of dearth began to come, according as Joseph had said: and the dearth was in all lands; but in all the land of Egypt there was bread."],
['Genesis','Gen','41','055','55',"And when all the land of Egypt was famished, the people cried to Pharaoh for bread: and Pharaoh said unto all the Egyptians, Go unto Joseph; what he saith to you, do."],
['Genesis','Gen','41','056','56',"And the famine was over all the face of the earth: And Joseph opened all the storehouses, and sold unto the Egyptians; and the famine waxed sore in the land of Egypt."],
['Genesis','Gen','41','057','57',"And all countries came into Egypt to Joseph for to buy \emph{corn}; because that the famine was \emph{so} sore in all lands."]
]
    
Genesis42 = [
    ['Genesis','Gen','42','001','1',"Now when Jacob saw that there was corn in Egypt, Jacob said unto his sons, Why do ye look one upon another?"],
['Genesis','Gen','42','002','2',"And he said, Behold, I have heard that there is corn in Egypt: get you down thither, and buy for us from thence; that we may live, and not die."],
['Genesis','Gen','42','003','3',"And Joseph's ten brethren went down to buy corn in Egypt."],
['Genesis','Gen','42','004','4',"But Benjamin, Joseph's brother, Jacob sent not with his brethren; for he said, Lest peradventure mischief befall him."],
['Genesis','Gen','42','005','5',"And the sons of Israel came to buy \emph{corn} among those that came: for the famine was in the land of Canaan."],
['Genesis','Gen','42','006','6',"And Joseph \emph{was} the governor over the land, \emph{and} he \emph{it} \emph{was} that sold to all the people of the land: and Joseph's brethren came, and bowed down themselves before him \emph{with} their faces to the earth."],
['Genesis','Gen','42','007','7',"And Joseph saw his brethren, and he knew them, but made himself strange unto them, and spake roughly unto them; and he said unto them, Whence come ye? And they said, From the land of Canaan to buy food."],
['Genesis','Gen','42','008','8',"And Joseph knew his brethren, but they knew not him."],
['Genesis','Gen','42','009','9',"And Joseph remembered the dreams which he dreamed of them, and said unto them, Ye \emph{are} spies; to see the nakedness of the land ye are come."],
['Genesis','Gen','42','010','10',"And they said unto him, Nay, my lord, but to buy food are thy servants come."],
['Genesis','Gen','42','011','11',"We \emph{are} all one man's sons; we \emph{are} true \emph{men}, thy servants are no spies."],
['Genesis','Gen','42','012','12',"And he said unto them, Nay, but to see the nakedness of the land ye are come."],
['Genesis','Gen','42','013','13',"And they said, Thy servants \emph{are} twelve brethren, the sons of one man in the land of Canaan; and, behold, the youngest \emph{is} this day with our father, and one \emph{is} not."],
['Genesis','Gen','42','014','14',"And Joseph said unto them, That \emph{is} \emph{it} that I spake unto you, saying, Ye \emph{are} spies:"],
['Genesis','Gen','42','015','15',"Hereby ye shall be proved: By the life of Pharaoh ye shall not go forth hence, except your youngest brother come hither."],
['Genesis','Gen','42','016','16',"Send one of you, and let him fetch your brother, and ye shall be kept in prison, that your words may be proved, whether \emph{there} \emph{be} \emph{any} truth in you: or else by the life of Pharaoh surely ye \emph{are} spies."],
['Genesis','Gen','42','017','17',"And he put them all together into ward three days."],
['Genesis','Gen','42','018','18',"And Joseph said unto them the third day, This do, and live; \emph{for} I fear God:"],
['Genesis','Gen','42','019','19',"If ye \emph{be} true \emph{men}, let one of your brethren be bound in the house of your prison: go ye, carry corn for the famine of your houses:"],
['Genesis','Gen','42','020','20',"But bring your youngest brother unto me; so shall your words be verified, and ye shall not die. And they did so."],
['Genesis','Gen','42','021','21',"And they said one to another, We \emph{are} verily guilty concerning our brother, in that we saw the anguish of his soul, when he besought us, and we would not hear; therefore is this distress come upon us."],
['Genesis','Gen','42','022','22',"And Reuben answered them, saying, Spake I not unto you, saying, Do not sin against the child; and ye would not hear? therefore, behold, also his blood is required."],
['Genesis','Gen','42','023','23',"And they knew not that Joseph understood \emph{them}; for he spake unto them by an interpreter."],
['Genesis','Gen','42','024','24',"And he turned himself about from them, and wept; and returned to them again, and communed with them, and took from them Simeon, and bound him before their eyes."],
['Genesis','Gen','42','025','25',"Then Joseph commanded to fill their sacks with corn, and to restore every man's money into his sack, and to give them provision for the way: and thus did he unto them."],
['Genesis','Gen','42','026','26',"And they laded their asses with the corn, and departed thence."],
['Genesis','Gen','42','027','27',"And as one of them opened his sack to give his ass provender in the inn, he espied his money; for, behold, it \emph{was} in his sack's mouth."],
['Genesis','Gen','42','028','28',"And he said unto his brethren, My money is restored; and, lo, \emph{it} \emph{is} even in my sack: and their heart failed \emph{them}, and they were afraid, saying one to another, What \emph{is} this \emph{that} God hath done unto us?"],
['Genesis','Gen','42','029','29',"And they came unto Jacob their father unto the land of Canaan, and told him all that befell unto them; saying,"],
['Genesis','Gen','42','030','30',"The man, \emph{who} \emph{is} the lord of the land, spake roughly to us, and took us for spies of the country."],
['Genesis','Gen','42','031','31',"And we said unto him, We \emph{are} true \emph{men}; we are no spies:"],
['Genesis','Gen','42','032','32',"We \emph{be} twelve brethren, sons of our father; one \emph{is} not, and the youngest \emph{is} this day with our father in the land of Canaan."],
['Genesis','Gen','42','033','33',"And the man, the lord of the country, said unto us, Hereby shall I know that ye \emph{are} true \emph{men}; leave one of your brethren \emph{here} with me, and take \emph{food} \emph{for} the famine of your households, and be gone:"],
['Genesis','Gen','42','034','34',"And bring your youngest brother unto me: then shall I know that ye \emph{are} no spies, but \emph{that} ye \emph{are} true \emph{men:} \emph{so} will I deliver you your brother, and ye shall traffick in the land."],
['Genesis','Gen','42','035','35',"And it came to pass as they emptied their sacks, that, behold, every man's bundle of money \emph{was} in his sack: and when \emph{both} they and their father saw the bundles of money, they were afraid."],
['Genesis','Gen','42','036','36',"And Jacob their father said unto them, Me have ye bereaved \emph{of} \emph{my} \emph{children}: Joseph \emph{is} not, and Simeon \emph{is} not, and ye will take Benjamin \emph{away}: all these things are against me."],
['Genesis','Gen','42','037','37',"And Reuben spake unto his father, saying, Slay my two sons, if I bring him not to thee: deliver him into my hand, and I will bring him to thee again."],
['Genesis','Gen','42','038','38',"And he said, My son shall not go down with you; for his brother is dead, and he is left alone: if mischief befall him by the way in the which ye go, then shall ye bring down my gray hairs with sorrow to the grave."]
]

Genesis43 = [
    ['Genesis','Gen','43','001','1',"And the famine \emph{was} sore in the land."],
['Genesis','Gen','43','002','2',"And it came to pass, when they had eaten up the corn which they had brought out of Egypt, their father said unto them, Go again, buy us a little food."],
['Genesis','Gen','43','003','3',"And Judah spake unto him, saying, The man did solemnly protest unto us, saying, Ye shall not see my face, except your brother \emph{be} with you."],
['Genesis','Gen','43','004','4',"If thou wilt send our brother with us, we will go down and buy thee food:"],
['Genesis','Gen','43','005','5',"But if thou wilt not send \emph{him}, we will not go down: for the man said unto us, Ye shall not see my face, except your brother \emph{be} with you."],
['Genesis','Gen','43','006','6',"And Israel said, Wherefore dealt ye \emph{so} ill with me, \emph{as} to tell the man whether ye had yet a brother?"],
['Genesis','Gen','43','007','7',"And they said, The man asked us straitly of our state, and of our kindred, saying, \emph{Is} your father yet alive? have ye \emph{another} brother? and we told him according to the tenor of these words: could we certainly know that he would say, Bring your brother down?"],
['Genesis','Gen','43','008','8',"And Judah said unto Israel his father, Send the lad with me, and we will arise and go; that we may live, and not die, both we, and thou, \emph{and} also our little ones."],
['Genesis','Gen','43','009','9',"I will be surety for him; of my hand shalt thou require him: if I bring him not unto thee, and set him before thee, then let me bear the blame for ever:"],
['Genesis','Gen','43','010','10',"For except we had lingered, surely now we had returned this second time."],
['Genesis','Gen','43','011','11',"And their father Israel said unto them, If \emph{it} \emph{must} \emph{be} so now, do this; take of the best fruits in the land in your vessels, and carry down the man a present, a little balm, and a little honey, spices, and myrrh, nuts, and almonds:"],
['Genesis','Gen','43','012','12',"And take double money in your hand; and the money that was brought again in the mouth of your sacks, carry \emph{it} again in your hand; peradventure it \emph{was} an oversight:"],
['Genesis','Gen','43','013','13',"Take also your brother, and arise, go again unto the man:"],
['Genesis','Gen','43','014','14',"And God Almighty give you mercy before the man, that he may send away your other brother, and Benjamin. If I be bereaved \emph{of} \emph{my} \emph{children}, I am bereaved."],
['Genesis','Gen','43','015','15',"And the men took that present, and they took double money in their hand, and Benjamin; and rose up, and went down to Egypt, and stood before Joseph."],
['Genesis','Gen','43','016','16',"And when Joseph saw Benjamin with them, he said to the ruler of his house, Bring \emph{these} men home, and slay, and make ready; for \emph{these} men shall dine with me at noon."],
['Genesis','Gen','43','017','17',"And the man did as Joseph bade; and the man brought the men into Joseph's house."],
['Genesis','Gen','43','018','18',"And the men were afraid, because they were brought into Joseph's house; and they said, Because of the money that was returned in our sacks at the first time are we brought in; that he may seek occasion against us, and fall upon us, and take us for bondmen, and our asses."],
['Genesis','Gen','43','019','19',"And they came near to the steward of Joseph's house, and they communed with him at the door of the house,"],
['Genesis','Gen','43','020','20',"And said, O sir, we came indeed down at the first time to buy food:"],
['Genesis','Gen','43','021','21',"And it came to pass, when we came to the inn, that we opened our sacks, and, behold, \emph{every} man's money \emph{was} in the mouth of his sack, our money in full weight: and we have brought it again in our hand."],
['Genesis','Gen','43','022','22',"And other money have we brought down in our hands to buy food: we cannot tell who put our money in our sacks."],
['Genesis','Gen','43','023','23',"And he said, Peace \emph{be} to you, fear not: your God, and the God of your father, hath given you treasure in your sacks: I had your money. And he brought Simeon out unto them."],
['Genesis','Gen','43','024','24',"And the man brought the men into Joseph's house, and gave \emph{them} water, and they washed their feet; and he gave their asses provender."],
['Genesis','Gen','43','025','25',"And they made ready the present against Joseph came at noon: for they heard that they should eat bread there."],
['Genesis','Gen','43','026','26',"And when Joseph came home, they brought him the present which \emph{was} in their hand into the house, and bowed themselves to him to the earth."],
['Genesis','Gen','43','027','27',"And he asked them of \emph{their} welfare, and said, \emph{Is} your father well, the old man of whom ye spake? \emph{Is} he yet alive?"],
['Genesis','Gen','43','028','28',"And they answered, Thy servant our father \emph{is} in good health, he \emph{is} yet alive. And they bowed down their heads, and made obeisance."],
['Genesis','Gen','43','029','29',"And he lifted up his eyes, and saw his brother Benjamin, his mother's son, and said, \emph{Is} this your younger brother, of whom ye spake unto me? And he said, God be gracious unto thee, my son."],
['Genesis','Gen','43','030','30',"And Joseph made haste; for his bowels did yearn upon his brother: and he sought \emph{where} to weep; and he entered into \emph{his} chamber, and wept there."],
['Genesis','Gen','43','031','31',"And he washed his face, and went out, and refrained himself, and said, Set on bread."],
['Genesis','Gen','43','032','32',"And they set on for him by himself, and for them by themselves, and for the Egyptians, which did eat with him, by themselves: because the Egyptians might not eat bread with the Hebrews; for that \emph{is} an abomination unto the Egyptians."],
    ['Genesis','Gen','43','033','33',"And they sat before him, the firstborn according to his birthright, and the youngest according to his youth: and the men marvelled one at another."],
    ['Genesis','Gen','43','034','34',"And he took \emph{and} \emph{sent} messes unto them from before him: but Benjamin's mess was five times so much as any of their's. And they drank, and were merry with him."]
]
    
Genesis44 = [
    ['Genesis','Gen','44','001','1',"And he commanded the steward of his house, saying, Fill the men's sacks \emph{with} food, as much as they can carry, and put every man's money in his sack's mouth."],
['Genesis','Gen','44','002','2',"And put my cup, the silver cup, in the sack's mouth of the youngest, and his corn money. And he did according to the word that Joseph had spoken."],
['Genesis','Gen','44','003','3',"As soon as the morning was light, the men were sent away, they and their asses."],
['Genesis','Gen','44','004','4',"\emph{And} when they were gone out of the city, \emph{and} not \emph{yet} far off, Joseph said unto his steward, Up, follow after the men; and when thou dost overtake them, say unto them, Wherefore have ye rewarded evil for good?"],
['Genesis','Gen','44','005','5',"\emph{Is} not this \emph{it} in which my lord drinketh, and whereby indeed he divineth? ye have done evil in so doing."],
['Genesis','Gen','44','006','6',"And he overtook them, and he spake unto them these same words."],
['Genesis','Gen','44','007','7',"And they said unto him, Wherefore saith my lord these words? God forbid that thy servants should do according to this thing:"],
['Genesis','Gen','44','008','8',"Behold, the money, which we found in our sacks' mouths, we brought again unto thee out of the land of Canaan: how then should we steal out of thy lord's house silver or gold?"],
['Genesis','Gen','44','009','9',"With whomsoever of thy servants it be found, both let him die, and we also will be my lord's bondmen."],
['Genesis','Gen','44','010','10',"And he said, Now also \emph{let} it \emph{be} according unto your words: he with whom it is found shall be my servant; and ye shall be blameless."],
['Genesis','Gen','44','011','11',"Then they speedily took down every man his sack to the ground, and opened every man his sack."],
['Genesis','Gen','44','012','12',"And he searched, \emph{and} began at the eldest, and left at the youngest: and the cup was found in Benjamin's sack."],
['Genesis','Gen','44','013','13',"Then they rent their clothes, and laded every man his ass, and returned to the city."],
['Genesis','Gen','44','014','14',"And Judah and his brethren came to Joseph's house; for he \emph{was} yet there: and they fell before him on the ground."],
['Genesis','Gen','44','015','15',"And Joseph said unto them, What deed \emph{is} this that ye have done? wot ye not that such a man as I can certainly divine?"],
['Genesis','Gen','44','016','16',"And Judah said, What shall we say unto my lord? what shall we speak? or how shall we clear ourselves? God hath found out the iniquity of thy servants: behold, we \emph{are} my lord's servants, both we, and \emph{he} also with whom the cup is found."],
['Genesis','Gen','44','017','17',"And he said, God forbid that I should do so: \emph{but} the man in whose hand the cup is found, he shall be my servant; and as for you, get you up in peace unto your father."],
['Genesis','Gen','44','018','18',"Then Judah came near unto him, and said, Oh my lord, let thy servant, I pray thee, speak a word in my lord's ears, and let not thine anger burn against thy servant: for thou \emph{art} even as Pharaoh."],
['Genesis','Gen','44','019','19',"My lord asked his servants, saying, Have ye a father, or a brother?"],
['Genesis','Gen','44','020','20',"And we said unto my lord, We have a father, an old man, and a child of his old age, a little one; and his brother is dead, and he alone is left of his mother, and his father loveth him."],
['Genesis','Gen','44','021','21',"And thou saidst unto thy servants, Bring him down unto me, that I may set mine eyes upon him."],
['Genesis','Gen','44','022','22',"And we said unto my lord, The lad cannot leave his father: for \emph{if} he should leave his father, \emph{his} \emph{father} would die."],
['Genesis','Gen','44','023','23',"And thou saidst unto thy servants, Except your youngest brother come down with you, ye shall see my face no more."],
['Genesis','Gen','44','024','24',"And it came to pass when we came up unto thy servant my father, we told him the words of my lord."],
['Genesis','Gen','44','025','25',"And our father said, Go again, \emph{and} buy us a little food."],
['Genesis','Gen','44','026','26',"And we said, We cannot go down: if our youngest brother be with us, then will we go down: for we may not see the man's face, except our youngest brother \emph{be} with us."],
['Genesis','Gen','44','027','27',"And thy servant my father said unto us, Ye know that my wife bare me two \emph{sons}:"],
['Genesis','Gen','44','028','28',"And the one went out from me, and I said, Surely he is torn in pieces; and I saw him not since:"],
['Genesis','Gen','44','029','29',"And if ye take this also from me, and mischief befall him, ye shall bring down my gray hairs with sorrow to the grave."],
['Genesis','Gen','44','030','30',"Now therefore when I come to thy servant my father, and the lad \emph{be} not with us; seeing that his life is bound up in the lad's life;"],
['Genesis','Gen','44','031','31',"It shall come to pass, when he seeth that the lad \emph{is} not \emph{with} \emph{us}, that he will die: and thy servants shall bring down the gray hairs of thy servant our father with sorrow to the grave."],
['Genesis','Gen','44','032','32',"For thy servant became surety for the lad unto my father, saying, If I bring him not unto thee, then I shall bear the blame to my father for ever."],
['Genesis','Gen','44','033','33',"Now therefore, I pray thee, let thy servant abide instead of the lad a bondman to my lord; and let the lad go up with his brethren."],
['Genesis','Gen','44','034','34',"For how shall I go up to my father, and the lad \emph{be} not with me? lest peradventure I see the evil that shall come on my father."]
]
     
Genesis45 = [
    ['Genesis','Gen','45','001','1',"Then Joseph could not refrain himself before all them that stood by him; and he cried, Cause every man to go out from me. And there stood no man with him, while Joseph made himself known unto his brethren."],
['Genesis','Gen','45','002','2',"And he wept aloud: and the Egyptians and the house of Pharaoh heard."],
['Genesis','Gen','45','003','3',"And Joseph said unto his brethren, I \emph{am} Joseph; doth my father yet live? And his brethren could not answer him; for they were troubled at his presence."],
['Genesis','Gen','45','004','4',"And Joseph said unto his brethren, Come near to me, I pray you. And they came near. And he said, I \emph{am} Joseph your brother, whom ye sold into Egypt."],
['Genesis','Gen','45','005','5',"Now therefore be not grieved, nor angry with yourselves, that ye sold me hither: for God did send me before you to preserve life."],
['Genesis','Gen','45','006','6',"For these two years \emph{hath} the famine \emph{been} in the land: and yet \emph{there} \emph{are} five years, in the which \emph{there} \emph{shall} neither \emph{be} earing nor harvest."],
['Genesis','Gen','45','007','7',"And God sent me before you to preserve you a posterity in the earth, and to save your lives by a great deliverance."],
['Genesis','Gen','45','008','8',"So now \emph{it} \emph{was} not you \emph{that} sent me hither, but God: and he hath made me a father to Pharaoh, and lord of all his house, and a ruler throughout all the land of Egypt."],
['Genesis','Gen','45','009','9',"Haste ye, and go up to my father, and say unto him, Thus saith thy son Joseph, God hath made me lord of all Egypt: come down unto me, tarry not:"],
['Genesis','Gen','45','010','10',"And thou shalt dwell in the land of Goshen, and thou shalt be near unto me, thou, and thy children, and thy children's children, and thy flocks, and thy herds, and all that thou hast:"],
['Genesis','Gen','45','011','11',"And there will I nourish thee; for yet \emph{there} \emph{are} five years of famine; lest thou, and thy household, and all that thou hast, come to poverty."],
['Genesis','Gen','45','012','12',"And, behold, your eyes see, and the eyes of my brother Benjamin, that \emph{it} \emph{is} my mouth that speaketh unto you."],
['Genesis','Gen','45','013','13',"And ye shall tell my father of all my glory in Egypt, and of all that ye have seen; and ye shall haste and bring down my father hither."],
['Genesis','Gen','45','014','14',"And he fell upon his brother Benjamin's neck, and wept; and Benjamin wept upon his neck."],
['Genesis','Gen','45','015','15',"Moreover he kissed all his brethren, and wept upon them: and after that his brethren talked with him."],
['Genesis','Gen','45','016','16',"And the fame thereof was heard in Pharaoh's house, saying, Joseph's brethren are come: and it pleased Pharaoh well, and his servants."],
['Genesis','Gen','45','017','17',"And Pharaoh said unto Joseph, Say unto thy brethren, This do ye; lade your beasts, and go, get you unto the land of Canaan;"],
['Genesis','Gen','45','018','18',"And take your father and your households, and come unto me: and I will give you the good of the land of Egypt, and ye shall eat the fat of the land."],
['Genesis','Gen','45','019','19',"Now thou art commanded, this do ye; take you wagons out of the land of Egypt for your little ones, and for your wives, and bring your father, and come."],
['Genesis','Gen','45','020','20',"Also regard not your stuff; for the good of all the land of Egypt \emph{is} your's."],
['Genesis','Gen','45','021','21',"And the children of Israel did so: and Joseph gave them wagons, according to the commandment of Pharaoh, and gave them provision for the way."],
['Genesis','Gen','45','022','22',"To all of them he gave each man changes of raiment; but to Benjamin he gave three hundred \emph{pieces} of silver, and five changes of raiment."],
['Genesis','Gen','45','023','23',"And to his father he sent after this \emph{manner}; ten asses laden with the good things of Egypt, and ten she asses laden with corn and bread and meat for his father by the way."],
['Genesis','Gen','45','024','24',"So he sent his brethren away, and they departed: and he said unto them, See that ye fall not out by the way."],
['Genesis','Gen','45','025','25',"And they went up out of Egypt, and came into the land of Canaan unto Jacob their father,"],
['Genesis','Gen','45','026','26',"And told him, saying, Joseph \emph{is} yet alive, and he \emph{is} governor over all the land of Egypt. And Jacob's heart fainted, for he believed them not."],
['Genesis','Gen','45','027','27',"And they told him all the words of Joseph, which he had said unto them: and when he saw the wagons which Joseph had sent to carry him, the spirit of Jacob their father revived:"],
['Genesis','Gen','45','028','28',"And Israel said, \emph{It} \emph{is} enough; Joseph my son \emph{is} yet alive: I will go and see him before I die."]
]
     
Genesis46 = [
    ['Genesis','Gen','46','001','1',"And Israel took his journey with all that he had, and came to Beer-sheba, and offered sacrifices unto the God of his father Isaac."],
    ['Genesis','Gen','46','002','2',"And God spake unto Israel in the visions of the night, and said, Jacob, Jacob. And he said, Here \emph{am} I."],
['Genesis','Gen','46','003','3',"And he said, I \emph{am} God, the God of thy father: fear not to go down into Egypt; for I will there make of thee a great nation:"],
['Genesis','Gen','46','004','4',"I will go down with thee into Egypt; and I will also surely bring thee up \emph{again}: and Joseph shall put his hand upon thine eyes."],
['Genesis','Gen','46','005','5',"And Jacob rose up from Beer-sheba: and the sons of Israel carried Jacob their father, and their little ones, and their wives, in the wagons which Pharaoh had sent to carry him."],
['Genesis','Gen','46','006','6',"And they took their cattle, and their goods, which they had gotten in the land of Canaan, and came into Egypt, Jacob, and all his seed with him:"],
['Genesis','Gen','46','007','7',"His sons, and his sons' sons with him, his daughters, and his sons' daughters, and all his seed brought he with him into Egypt."],
['Genesis','Gen','46','008','8',"And these \emph{are} the names of the children of Israel, which came into Egypt, Jacob and his sons: Reuben, Jacob's firstborn."],
['Genesis','Gen','46','009','9',"And the sons of Reuben; Hanoch, and Phallu, and Hezron, and Carmi."],
['Genesis','Gen','46','010','10',"And the sons of Simeon; Jemuel, and Jamin, and Ohad, and Jachin, and Zohar, and Shaul the son of a Canaanitish woman."],
['Genesis','Gen','46','011','11',"And the sons of Levi; Gershon, Kohath, and Merari."],
['Genesis','Gen','46','012','12',"And the sons of Judah; Er, and Onan, and Shelah, and Pharez, and Zerah: but Er and Onan died in the land of Canaan. And the sons of Pharez were Hezron and Hamul."],
['Genesis','Gen','46','013','13',"And the sons of Issachar; Tola, and Phuvah, and Job, and Shimron."],
['Genesis','Gen','46','014','14',"And the sons of Zebulun; Sered, and Elon, and Jahleel."],
['Genesis','Gen','46','015','15',"These \emph{be} the sons of Leah, which she bare unto Jacob in Padan-aram, with his daughter Dinah: all the souls of his sons and his daughters \emph{were} thirty and three."],
['Genesis','Gen','46','016','16',"And the sons of Gad; Ziphion, and Haggi, Shuni, and Ezbon, Eri, and Arodi, and Areli."],
['Genesis','Gen','46','017','17',"And the sons of Asher; Jimnah, and Ishuah, and Isui, and Beriah, and Serah their sister: and the sons of Beriah; Heber, and Malchiel."],
['Genesis','Gen','46','018','18',"These \emph{are} the sons of Zilpah, whom Laban gave to Leah his daughter, and these she bare unto Jacob, \emph{even} sixteen souls."],
['Genesis','Gen','46','019','19',"The sons of Rachel Jacob's wife; Joseph, and Benjamin."],
['Genesis','Gen','46','020','20',"And unto Joseph in the land of Egypt were born Manasseh and Ephraim, which Asenath the daughter of Poti-pherah priest of On bare unto him."],
['Genesis','Gen','46','021','21',"And the sons of Benjamin \emph{were} Belah, and Becher, and Ashbel, Gera, and Naaman, Ehi, and Rosh, Muppim, and Huppim, and Ard."],
['Genesis','Gen','46','022','22',"These \emph{are} the sons of Rachel, which were born to Jacob: all the souls \emph{were} fourteen."],
['Genesis','Gen','46','023','23',"And the sons of Dan; Hushim."],
['Genesis','Gen','46','024','24',"And the sons of Naphtali; Jahzeel, and Guni, and Jezer, and Shillem."],
['Genesis','Gen','46','025','25',"These \emph{are} the sons of Bilhah, which Laban gave unto Rachel his daughter, and she bare these unto Jacob: all the souls \emph{were} seven."],
['Genesis','Gen','46','026','26',"All the souls that came with Jacob into Egypt, which came out of his loins, besides Jacob's sons' wives, all the souls \emph{were} threescore and six;"],
['Genesis','Gen','46','027','27',"And the sons of Joseph, which were born him in Egypt, \emph{were} two souls: all the souls of the house of Jacob, which came into Egypt, \emph{were} threescore and ten."],
['Genesis','Gen','46','028','28',"And he sent Judah before him unto Joseph, to direct his face unto Goshen; and they came into the land of Goshen."],
['Genesis','Gen','46','029','29',"And Joseph made ready his chariot, and went up to meet Israel his father, to Goshen, and presented himself unto him; and he fell on his neck, and wept on his neck a good while."],
    ['Genesis','Gen','46','030','30',"And Israel said unto Joseph, Now let me die, since I have seen thy face, because thou \emph{art} yet alive."],
    ['Genesis','Gen','46','031','31',"And Joseph said unto his brethren, and unto his father's house, I will go up, and shew Pharaoh, and say unto him, My brethren, and my father's house, which \emph{were} in the land of Canaan, are come unto me;"],
    ['Genesis','Gen','46','032','32',"And the men \emph{are} shepherds, for their trade hath been to feed cattle; and they have brought their flocks, and their herds, and all that they have."],
    ['Genesis','Gen','46','033','33',"And it shall come to pass, when Pharaoh shall call you, and shall say, What \emph{is} your occupation?"],
    ['Genesis','Gen','46','034','34',"That ye shall say, Thy servants' trade hath been about cattle from our youth even until now, both we, \emph{and} also our fathers: that ye may dwell in the land of Goshen; for every shepherd \emph{is} an abomination unto the Egyptians."]
] 
     
Genesis47 = [
    ['Genesis','Gen','47','001','1',"Then Joseph came and told Pharaoh, and said, My father and my brethren, and their flocks, and their herds, and all that they have, are come out of the land of Canaan; and, behold, they \emph{are} in the land of Goshen."],
    ['Genesis','Gen','47','002','2',"And he took some of his brethren, \emph{even} five men, and presented them unto Pharaoh."],
    ['Genesis','Gen','47','003','3',"And Pharaoh said unto his brethren, What \emph{is} your occupation? And they said unto Pharaoh, Thy servants \emph{are} shepherds, both we, \emph{and} also our fathers."],
['Genesis','Gen','47','004','4',"They said moreover unto Pharaoh, For to sojourn in the land are we come; for thy servants have no pasture for their flocks; for the famine \emph{is} sore in the land of Canaan: now therefore, we pray thee, let thy servants dwell in the land of Goshen."],
['Genesis','Gen','47','005','5',"And Pharaoh spake unto Joseph, saying, Thy father and thy brethren are come unto thee:"],
['Genesis','Gen','47','006','6',"The land of Egypt \emph{is} before thee; in the best of the land make thy father and brethren to dwell; in the land of Goshen let them dwell: and if thou knowest \emph{any} men of activity among them, then make them rulers over my cattle."],
['Genesis','Gen','47','007','7',"And Joseph brought in Jacob his father, and set him before Pharaoh: and Jacob blessed Pharaoh."],
['Genesis','Gen','47','008','8',"And Pharaoh said unto Jacob, How old \emph{art} thou?"],
['Genesis','Gen','47','009','9',"And Jacob said unto Pharaoh, The days of the years of my pilgrimage \emph{are} an hundred and thirty years: few and evil have the days of the years of my life been, and have not attained unto the days of the years of the life of my fathers in the days of their pilgrimage."],
['Genesis','Gen','47','010','10',"And Jacob blessed Pharaoh, and went out from before Pharaoh."],
['Genesis','Gen','47','011','11',"And Joseph placed his father and his brethren, and gave them a possession in the land of Egypt, in the best of the land, in the land of Rameses, as Pharaoh had commanded."],
['Genesis','Gen','47','012','12',"And Joseph nourished his father, and his brethren, and all his father's household, with bread, according to \emph{their} families."],
['Genesis','Gen','47','013','13',"And \emph{there} \emph{was} no bread in all the land; for the famine \emph{was} very sore, so that the land of Egypt and \emph{all} the land of Canaan fainted by reason of the famine."],
['Genesis','Gen','47','014','14',"And Joseph gathered up all the money that was found in the land of Egypt, and in the land of Canaan, for the corn which they bought: and Joseph brought the money into Pharaoh's house."],
['Genesis','Gen','47','015','15',"And when money failed in the land of Egypt, and in the land of Canaan, all the Egyptians came unto Joseph, and said, Give us bread: for why should we die in thy presence? for the money faileth."],
['Genesis','Gen','47','016','16',"And Joseph said, Give your cattle; and I will give you for your cattle, if money fail."],
['Genesis','Gen','47','017','17',"And they brought their cattle unto Joseph: and Joseph gave them bread \emph{in} \emph{exchange} for horses, and for the flocks, and for the cattle of the herds, and for the asses: and he fed them with bread for all their cattle for that year."],
['Genesis','Gen','47','018','18',"When that year was ended, they came unto him the second year, and said unto him, We will not hide \emph{it} from my lord, how that our money is spent; my lord also hath our herds of cattle; there is not ought left in the sight of my lord, but our bodies, and our lands:"],
['Genesis','Gen','47','019','19',"Wherefore shall we die before thine eyes, both we and our land? buy us and our land for bread, and we and our land will be servants unto Pharaoh: and give \emph{us} seed, that we may live, and not die, that the land be not desolate."],
['Genesis','Gen','47','020','20',"And Joseph bought all the land of Egypt for Pharaoh; for the Egyptians sold every man his field, because the famine prevailed over them: so the land became Pharaoh's."],
['Genesis','Gen','47','021','21',"And as for the people, he removed them to cities from \emph{one} end of the borders of Egypt even to the \emph{other} end thereof."],
['Genesis','Gen','47','022','22',"Only the land of the priests bought he not; for the priests had a portion \emph{assigned} \emph{them} of Pharaoh, and did eat their portion which Pharaoh gave them: wherefore they sold not their lands."],
['Genesis','Gen','47','023','23',"Then Joseph said unto the people, Behold, I have bought you this day and your land for Pharaoh: lo, \emph{here} \emph{is} seed for you, and ye shall sow the land."],
['Genesis','Gen','47','024','24',"And it shall come to pass in the increase, that ye shall give the fifth \emph{part} unto Pharaoh, and four parts shall be your own, for seed of the field, and for your food, and for them of your households, and for food for your little ones."],
['Genesis','Gen','47','025','25',"And they said, Thou hast saved our lives: let us find grace in the sight of my lord, and we will be Pharaoh's servants."],
['Genesis','Gen','47','026','26',"And Joseph made it a law over the land of Egypt unto this day, \emph{that} Pharaoh should have the fifth \emph{part}; except the land of the priests only, \emph{which} became not Pharaoh's."],
    ['Genesis','Gen','47','027','27',"And Israel dwelt in the land of Egypt, in the country of Goshen; and they had possessions therein, and grew, and multiplied exceedingly."],
    ['Genesis','Gen','47','028','28',"And Jacob lived in the land of Egypt seventeen years: so the whole age of Jacob was an hundred forty and seven years."],
    ['Genesis','Gen','47','029','29',"And the time drew nigh that Israel must die: and he called his son Joseph, and said unto him, If now I have found grace in thy sight, put, I pray thee, thy hand under my thigh, and deal kindly and truly with me; bury me not, I pray thee, in Egypt:"],
    ['Genesis','Gen','47','030','30',"But I will lie with my fathers, and thou shalt carry me out of Egypt, and bury me in their buryingplace. And he said, I will do as thou hast said."],
    ['Genesis','Gen','47','031','31',"And he said, Swear unto me. And he sware unto him. And Israel bowed himself upon the bed's head."]
]
     
Genesis48 = [
    ['Genesis','Gen','48','001','1',"And it came to pass after these things, that \emph{one} told Joseph, Behold, thy father \emph{is} sick: and he took with him his two sons, Manasseh and Ephraim."],
['Genesis','Gen','48','002','2',"And \emph{one} told Jacob, and said, Behold, thy son Joseph cometh unto thee: and Israel strengthened himself, and sat upon the bed."],
['Genesis','Gen','48','003','3',"And Jacob said unto Joseph, God Almighty appeared unto me at Luz in the land of Canaan, and blessed me,"],
['Genesis','Gen','48','004','4',"And said unto me, Behold, I will make thee fruitful, and multiply thee, and I will make of thee a multitude of people; and will give this land to thy seed after thee \emph{for} an everlasting possession."],
['Genesis','Gen','48','005','5',"And now thy two sons, Ephraim and Manasseh, which were born unto thee in the land of Egypt before I came unto thee into Egypt, \emph{are} mine; as Reuben and Simeon, they shall be mine."],
['Genesis','Gen','48','006','6',"And thy issue, which thou begettest after them, shall be thine, \emph{and} shall be called after the name of their brethren in their inheritance."],
['Genesis','Gen','48','007','7',"And as for me, when I came from Padan, Rachel died by me in the land of Canaan in the way, when yet \emph{there} \emph{was} but a little way to come unto Ephrath: and I buried her there in the way of Ephrath; the same \emph{is} Beth-lehem."],
['Genesis','Gen','48','008','8',"And Israel beheld Joseph's sons, and said, Who \emph{are} these?"],
['Genesis','Gen','48','009','9',"And Joseph said unto his father, They \emph{are} my sons, whom God hath given me in this \emph{place}. And he said, Bring them, I pray thee, unto me, and I will bless them."],
['Genesis','Gen','48','010','10',"Now the eyes of Israel were dim for age, \emph{so} \emph{that} he could not see. And he brought them near unto him; and he kissed them, and embraced them."],
['Genesis','Gen','48','011','11',"And Israel said unto Joseph, I had not thought to see thy face: and, lo, God hath shewed me also thy seed."],
['Genesis','Gen','48','012','12',"And Joseph brought them out from between his knees, and he bowed himself with his face to the earth."],
['Genesis','Gen','48','013','13',"And Joseph took them both, Ephraim in his right hand toward Israel's left hand, and Manasseh in his left hand toward Israel's right hand, and brought \emph{them} near unto him."],
['Genesis','Gen','48','014','14',"And Israel stretched out his right hand, and laid \emph{it} upon Ephraim's head, who \emph{was} the younger, and his left hand upon Manasseh's head, guiding his hands wittingly; for Manasseh \emph{was} the firstborn."],
['Genesis','Gen','48','015','15',"And he blessed Joseph, and said, God, before whom my fathers Abraham and Isaac did walk, the God which fed me all my life long unto this day,"],
['Genesis','Gen','48','016','16',"The Angel which redeemed me from all evil, bless the lads; and let my name be named on them, and the name of my fathers Abraham and Isaac; and let them grow into a multitude in the midst of the earth."],
['Genesis','Gen','48','017','17',"And when Joseph saw that his father laid his right hand upon the head of Ephraim, it displeased him: and he held up his father's hand, to remove it from Ephraim's head unto Manasseh's head."],
['Genesis','Gen','48','018','18',"And Joseph said unto his father, Not so, my father: for this \emph{is} the firstborn; put thy right hand upon his head."],
['Genesis','Gen','48','019','19',"And his father refused, and said, I know \emph{it}, my son, I know \emph{it}: he also shall become a people, and he also shall be great: but truly his younger brother shall be greater than he, and his seed shall become a multitude of nations."],
['Genesis','Gen','48','020','20',"And he blessed them that day, saying, In thee shall Israel bless, saying, God make thee as Ephraim and as Manasseh: and he set Ephraim before Manasseh."],
    ['Genesis','Gen','48','021','21',"And Israel said unto Joseph, Behold, I die: but God shall be with you, and bring you again unto the land of your fathers."],
    ['Genesis','Gen','48','022','22',"Moreover I have given to thee one portion above thy brethren, which I took out of the hand of the Amorite with my sword and with my bow."]
]
    
Genesis49 = [
    ['Genesis','Gen','49','001','1',"And Jacob called unto his sons, and said, Gather yourselves together, that I may tell you \emph{that} which shall befall you in the last days."],
['Genesis','Gen','49','002','2',"Gather yourselves together, and hear, ye sons of Jacob; and hearken unto Israel your father."],
['Genesis','Gen','49','003','3',"Reuben, thou \emph{art} my firstborn, my might, and the beginning of my strength, the excellency of dignity, and the excellency of power:"],
['Genesis','Gen','49','004','4',"Unstable as water, thou shalt not excel; because thou wentest up to thy father's bed; then defiledst thou \emph{it}: he went up to my couch."],
['Genesis','Gen','49','005','5',"Simeon and Levi \emph{are} brethren; instruments of cruelty \emph{are} \emph{in} their habitations."],
['Genesis','Gen','49','006','6',"O my soul, come not thou into their secret; unto their assembly, mine honour, be not thou united: for in their anger they slew a man, and in their selfwill they digged down a wall."],
['Genesis','Gen','49','007','7',"Cursed \emph{be} their anger, for \emph{it} \emph{was} fierce; and their wrath, for it was cruel: I will divide them in Jacob, and scatter them in Israel."],
['Genesis','Gen','49','008','8',"Judah, thou \emph{art} \emph{he} whom thy brethren shall praise: thy hand \emph{shall} \emph{be} in the neck of thine enemies; thy father's children shall bow down before thee."],
['Genesis','Gen','49','009','9',"Judah \emph{is} a lion's whelp: from the prey, my son, thou art gone up: he stooped down, he couched as a lion, and as an old lion; who shall rouse him up?"],
['Genesis','Gen','49','010','10',"The sceptre shall not depart from Judah, nor a lawgiver from between his feet, until Shiloh come; and unto him \emph{shall} the gathering of the people \emph{be}."],
['Genesis','Gen','49','011','11',"Binding his foal unto the vine, and his ass's colt unto the choice vine; he washed his garments in wine, and his clothes in the blood of grapes:"],
['Genesis','Gen','49','012','12',"His eyes \emph{shall} \emph{be} red with wine, and his teeth white with milk."],
['Genesis','Gen','49','013','13',"Zebulun shall dwell at the haven of the sea; and he \emph{shall} \emph{be} for an haven of ships; and his border \emph{shall} \emph{be} unto Zidon."],
['Genesis','Gen','49','014','14',"Issachar \emph{is} a strong ass couching down between two burdens:"],
['Genesis','Gen','49','015','15',"And he saw that rest \emph{was} good, and the land that \emph{it} \emph{was} pleasant; and bowed his shoulder to bear, and became a servant unto tribute."],
['Genesis','Gen','49','016','16',"Dan shall judge his people, as one of the tribes of Israel."],
['Genesis','Gen','49','017','17',"Dan shall be a serpent by the way, an adder in the path, that biteth the horse heels, so that his rider shall fall backward."],
['Genesis','Gen','49','018','18',"I have waited for thy salvation, O LORD."],
['Genesis','Gen','49','019','19',"Gad, a troop shall overcome him: but he shall overcome at the last."],
['Genesis','Gen','49','020','20',"Out of Asher his bread \emph{shall} \emph{be} fat, and he shall yield royal dainties."],
['Genesis','Gen','49','021','21',"Naphtali \emph{is} a hind let loose: he giveth goodly words."],
['Genesis','Gen','49','022','22',"Joseph \emph{is} a fruitful bough, \emph{even} a fruitful bough by a well; \emph{whose} branches run over the wall:"],
['Genesis','Gen','49','023','23',"The archers have sorely grieved him, and shot \emph{at} \emph{him}, and hated him:"],
['Genesis','Gen','49','024','24',"But his bow abode in strength, and the arms of his hands were made strong by the hands of the mighty \emph{God} of Jacob; (from thence \emph{is} the shepherd, the stone of Israel:)"],
['Genesis','Gen','49','025','25',"\emph{Even} by the God of thy father, who shall help thee; and by the Almighty, who shall bless thee with blessings of heaven above, blessings of the deep that lieth under, blessings of the breasts, and of the womb:"],
['Genesis','Gen','49','026','26',"The blessings of thy father have prevailed above the blessings of my progenitors unto the utmost bound of the everlasting hills: they shall be on the head of Joseph, and on the crown of the head of him that was separate from his brethren."],
['Genesis','Gen','49','027','27',"Benjamin shall ravin \emph{as} a wolf: in the morning he shall devour the prey, and at night he shall divide the spoil."],
['Genesis','Gen','49','028','28',"All these \emph{are} the twelve tribes of Israel: and this \emph{is} \emph{it} that their father spake unto them, and blessed them; every one according to his blessing he blessed them."],
['Genesis','Gen','49','029','29',"And he charged them, and said unto them, I am to be gathered unto my people: bury me with my fathers in the cave that \emph{is} in the field of Ephron the Hittite,"],
['Genesis','Gen','49','030','30',"In the cave that \emph{is} in the field of Machpelah, which \emph{is} before Mamre, in the land of Canaan, which Abraham bought with the field of Ephron the Hittite for a possession of a buryingplace."],
['Genesis','Gen','49','031','31',"There they buried Abraham and Sarah his wife; there they buried Isaac and Rebekah his wife; and there I buried Leah."],
['Genesis','Gen','49','032','32',"The purchase of the field and of the cave that \emph{is} therein \emph{was} from the children of Heth."],
    ['Genesis','Gen','49','033','33',"And when Jacob had made an end of commanding his sons, he gathered up his feet into the bed, and yielded up the ghost, and was gathered unto his people."]
]
    
Genesis50 = [
    ['Genesis','Gen','50','001','1',"And Joseph fell upon his father's face, and wept upon him, and kissed him."],
['Genesis','Gen','50','002','2',"And Joseph commanded his servants the physicians to embalm his father: and the physicians embalmed Israel."],
['Genesis','Gen','50','003','3',"And forty days were fulfilled for him; for so are fulfilled the days of those which are embalmed: and the Egyptians mourned for him threescore and ten days."],
['Genesis','Gen','50','004','4',"And when the days of his mourning were past, Joseph spake unto the house of Pharaoh, saying, If now I have found grace in your eyes, speak, I pray you, in the ears of Pharaoh, saying,"],
['Genesis','Gen','50','005','5',"My father made me swear, saying, Lo, I die: in my grave which I have digged for me in the land of Canaan, there shalt thou bury me. Now therefore let me go up, I pray thee, and bury my father, and I will come again."],
['Genesis','Gen','50','006','6',"And Pharaoh said, Go up, and bury thy father, according as he made thee swear."],
['Genesis','Gen','50','007','7',"And Joseph went up to bury his father: and with him went up all the servants of Pharaoh, the elders of his house, and all the elders of the land of Egypt,"],
['Genesis','Gen','50','008','8',"And all the house of Joseph, and his brethren, and his father's house: only their little ones, and their flocks, and their herds, they left in the land of Goshen."],
['Genesis','Gen','50','009','9',"And there went up with him both chariots and horsemen: and it was a very great company."],
['Genesis','Gen','50','010','10',"And they came to the threshingfloor of Atad, which \emph{is} beyond Jordan, and there they mourned with a great and very sore lamentation: and he made a mourning for his father seven days."],
['Genesis','Gen','50','011','11',"And when the inhabitants of the land, the Canaanites, saw the mourning in the floor of Atad, they said, This \emph{is} a grievous mourning to the Egyptians: wherefore the name of it was called Abel-mizraim, which \emph{is} beyond Jordan."],
['Genesis','Gen','50','012','12',"And his sons did unto him according as he commanded them:"],
['Genesis','Gen','50','013','13',"For his sons carried him into the land of Canaan, and buried him in the cave of the field of Machpelah, which Abraham bought with the field for a possession of a buryingplace of Ephron the Hittite, before Mamre."],
['Genesis','Gen','50','014','14',"And Joseph returned into Egypt, he, and his brethren, and all that went up with him to bury his father, after he had buried his father."],
['Genesis','Gen','50','015','15',"And when Joseph's brethren saw that their father was dead, they said, Joseph will peradventure hate us, and will certainly requite us all the evil which we did unto him."],
['Genesis','Gen','50','016','16',"And they sent a messenger unto Joseph, saying, Thy father did command before he died, saying,"],
['Genesis','Gen','50','017','17',"So shall ye say unto Joseph, Forgive, I pray thee now, the trespass of thy brethren, and their sin; for they did unto thee evil: and now, we pray thee, forgive the trespass of the servants of the God of thy father. And Joseph wept when they spake unto him."],
['Genesis','Gen','50','018','18',"And his brethren also went and fell down before his face; and they said, Behold, we \emph{be} thy servants."],
['Genesis','Gen','50','019','19',"And Joseph said unto them, Fear not: for \emph{am} I in the place of God?"],
['Genesis','Gen','50','020','20',"But as for you, ye thought evil against me; \emph{but} God meant it unto good, to bring to pass, as \emph{it} \emph{is} this day, to save much people alive."],
['Genesis','Gen','50','021','21',"Now therefore fear ye not: I will nourish you, and your little ones. And he comforted them, and spake kindly unto them."],
['Genesis','Gen','50','022','22',"And Joseph dwelt in Egypt, he, and his father's house: and Joseph lived an hundred and ten years."],
['Genesis','Gen','50','023','23',"And Joseph saw Ephraim's children of the third \emph{generation}: the children also of Machir the son of Manasseh were brought up upon Joseph's knees."],
['Genesis','Gen','50','024','24',"And Joseph said unto his brethren, I die: and God will surely visit you, and bring you out of this land unto the land which he sware to Abraham, to Isaac, and to Jacob."],
['Genesis','Gen','50','025','25',"And Joseph took an oath of the children of Israel, saying, God will surely visit you, and ye shall carry up my bones from hence."],
    ['Genesis','Gen','50','026','26',"So Joseph died, \emph{being} an hundred and ten years old: and they embalmed him, and he was put in a coffin in Egypt."]
]

def main():
    ProcessVerses('Genesis50',Genesis50)

if __name__ == "__main__":
    main()



%%%%%%%%%%%%%%%%%%%%%%%%%%%
Chapter with Latex format
%%%%%%%%%%%%%%%%%%%%%%%%%%%


\footnote{\textcolor[cmyk]{0.99998,1,0,0}{\hyperlink{TOC}{Return to end of Table of Contents.}}}\textcolor[cmyk]{0.99998,1,0,0}{And Joseph fell upon his father's face, and wept upon him, and kissed him.}
[2] \textcolor[cmyk]{0.99998,1,0,0}{And Joseph commanded his servants the physicians to embalm his father: and the physicians embalmed Israel.}
[3] \textcolor[cmyk]{0.99998,1,0,0}{And forty days were fulfilled for him; for so are fulfilled the days of those which are embalmed: and the Egyptians mourned for him threescore and ten days.}
[4] \textcolor[cmyk]{0.99998,1,0,0}{And when the days of his mourning were past, Joseph spake unto the house of Pharaoh, saying, If now I have found grace in your eyes, speak, I pray you, in the ears of Pharaoh, saying,}
[5] \textcolor[cmyk]{0.99998,1,0,0}{My father made me swear, saying, Lo, I die: in my grave which I have digged for me in the land of Canaan, there sha

father's & 3 \\ \hline
fear & 1 \\ \hline
fell & 2 \\ \hline
field & 2 \\ \hline
flocks & 1 \\ \hline
floor & 1 \\ \hline
for & 9 \\ \hline
forgive & 1 \\ \hline
forty & 1 \\ \hline
found & 1 \\ \hline
from & 1 \\ \hline
fulfilled & 2 \\ \hline
go & 1 \\ \hline
good & 1 \\ \hline
grace & 1 \\ \hline
grave & 1 \\ \hline
great & 2 \\ \hline
grievous & 1 \\ \hline
had & 1 \\ \hline
hate & 1 \\ \hline
have & 2 \\ \hline
he & 10 \\ \hline
hence & 1 \\ \hline
herds & 1 \\ \hline
him & 13 \\ \hline
his & 18 \\ \hline
horsemen & 1 \\ \hline
house & 5 \\ \hline
hundred & 2 \\ \hline
in & 11 \\ \hline
inhabitants & 1 \\ \hline
into & 2 \\ \hline
it & 3 \\ \hline
kindly & 1 \\ \hline
kissed & 1 \\ \hline
knees & 1 \\ \hline
lamentation & 1 \\ \hline
land & 7 \\ \hline
left & 1 \\ \hline
let & 1 \\ \hline
little & 2 \\ \hline
lived & 1 \\ \hline
made & 3 \\ \hline
me & 5 \\ \hline
meant & 1 \\ \hline
messenger & 1 \\ \hline
mourned & 2 \\ \hline
mourning & 4 \\ \hline
much & 1 \\ \hline
my & 3 \\ 